In [1]:
import pandas as pd
import numpy as np
from plotnine import ggplot, geom_histogram, aes, geom_tile, theme_minimal, xlab, ylab, ylim, xlim
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from functools import reduce
%matplotlib inline

In [2]:
data_19_1 = pd.read_excel('2019 County Health Rankings Missouri Data - v1_0.xls',sheet_name = 'Outcomes & Factors Rankings', header = [0,1])
data_19_2 = pd.read_excel('2019 County Health Rankings Missouri Data - v1_0.xls',sheet_name = 'Additional Measure Data', header = [0,1])
data_19_3 = pd.read_excel('2019 County Health Rankings Missouri Data - v1_0.xls',sheet_name = 'Ranked Measure Data', header = [0,1])
data_19_4 = pd.read_excel('2019 County Health Rankings Missouri Data - v1_0.xls',sheet_name = 'Outcomes & Factors SubRankings', header = [0,1])

In [3]:
data_19_1

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Health Outcomes  \
                  FIPS              State             County         Z-Score   
0                29000           Missouri                NaN             NaN   
1                29001           Missouri              Adair        0.185230   
2                29003           Missouri             Andrew       -1.220680   
3                29005           Missouri           Atchison       -0.644169   
4                29007           Missouri            Audrain       -0.244042   
..                 ...                ...                ...             ...   
111              29223           Missouri              Wayne        0.987880   
112              29225           Missouri            Webster       -0.350411   
113              29227           Missouri              Worth        0.206137   
114              29229           Missouri             Wright        0.797923   
115              29510           Missouri     St. Louis City        2.084426   

           Health Factors         
      Rank        Z-Score   Rank  
0      NaN            NaN    NaN  
1     71.0      -0.238906   36.0  
2      8.0      -0.743436    6.0  
3     24.0      -0.632420    9.0  
4     50.0      -0.030719   58.0  
..     ...            ...    ...  
111  103.0       0.711992  108.0  
112   43.0       0.113857   74.0  
113   72.0      -0.515757   11.0  
114   97.0       0.413333  100.0  
115  112.0       1.197659  113.0  

[116 rows x 7 columns]

In [4]:
data_19_1.columns

MultiIndex([('Unnamed: 0_level_0',    'FIPS'),
            ('Unnamed: 1_level_0',   'State'),
            ('Unnamed: 2_level_0',  'County'),
            (   'Health Outcomes', 'Z-Score'),
            (   'Health Outcomes',    'Rank'),
            (    'Health Factors', 'Z-Score'),
            (    'Health Factors',    'Rank')],
           )

In [5]:
def add_index_prefix(df):
    new_columns = []
    for level0, level1 in df.columns:
        if level0.startswith('Unnamed') or level0.startswith("""*""") :
            new_columns.append(f"{level1}")
        else:
            new_columns.append(f"{level0}_{level1}")
    df.columns = new_columns
    return df

# Apply the function to the DataFrame
data_19_1 = add_index_prefix(data_19_1)

In [6]:
data_19_1.columns

Index(['FIPS', 'State', 'County', 'Health Outcomes_Z-Score',
       'Health Outcomes_Rank', 'Health Factors_Z-Score',
       'Health Factors_Rank'],
      dtype='object')

In [7]:
data_19_2

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Life expectancy  \
                  FIPS              State             County Life Expectancy   
0                29000           Missouri                NaN       77.403222   
1                29001           Missouri              Adair       77.290893   
2                29003           Missouri             Andrew       79.482247   
3                29005           Missouri           Atchison       77.575026   
4                29007           Missouri            Audrain       76.848305   
..                 ...                ...                ...             ...   
111              29223           Missouri              Wayne       74.692964   
112              29225           Missouri            Webster       77.388763   
113              29227           Missouri              Worth       71.766790   
114              29229           Missouri             Wright       74.689066   
115              29510           Missouri     St. Louis City       73.941362   

                                                        \
    95% CI - Low 95% CI - High Life Expectancy (Black)   
0      77.329386     77.477059                     NaN   
1      75.984695     78.597091                     NaN   
2      78.082630     80.881864                     NaN   
3      74.968435     80.181617                     NaN   
4      75.813400     77.883211               75.778083   
..           ...           ...                     ...   
111    73.156515     76.229414                     NaN   
112    76.475134     78.302393                     NaN   
113    66.051351     77.482229                     NaN   
114    73.355217     76.022914                     NaN   
115    73.569993     74.312731               69.870441   

                                                        \
    Life Expectancy (Hispanic) Life Expectancy (White)   
0                          NaN                     NaN   
1                          NaN                     NaN   
2                          NaN                     NaN   
3                          NaN                     NaN   
4                          NaN               76.894503   
..                         ...                     ...   
111                        NaN                     NaN   
112                        NaN                     NaN   
113                        NaN                     NaN   
114                        NaN                     NaN   
115                  87.512175               77.058679   

    Premature age-adjusted mortality  ... Demographics                       \
                            # Deaths  ...   % Hispanic # Non-Hispanic White   
0                              83522  ...     4.239023              4859227   
1                                309  ...     2.529850                22808   
2                                211  ...     2.449445                16617   
3                                 89  ...     1.345972                 5069   
4                                355  ...     3.092703                22369   
..                               ...  ...          ...                  ...   
111                              292  ...     1.947954                12605   
112                              483  ...     2.136299                36455   
113                               41  ...     1.409820                 1987   
114                              335  ...     1.903879                17440   
115                             5546  ...     4.033037               135641   

                                                      \
    % Non-Hispanic White # Not Proficient in English   
0              79.483137                       54975   
1              89.876660                         256   
2              94.656793                           0   
3              96.094787                           6   
4              87.239187                         133   
..                   ...         

In [8]:
data_19_2.columns.to_list()

[('Unnamed: 0_level_0', 'FIPS'),
 ('Unnamed: 1_level_0', 'State'),
 ('Unnamed: 2_level_0', 'County'),
 ('Life expectancy', 'Life Expectancy'),
 ('Life expectancy', '95% CI - Low'),
 ('Life expectancy', '95% CI - High'),
 ('Life expectancy', 'Life Expectancy (Black)'),
 ('Life expectancy', 'Life Expectancy (Hispanic)'),
 ('Life expectancy', 'Life Expectancy (White)'),
 ('Premature age-adjusted mortality', '# Deaths'),
 ('Premature age-adjusted mortality', 'Age-Adjusted Mortality'),
 ('Premature age-adjusted mortality', '95% CI - Low'),
 ('Premature age-adjusted mortality', '95% CI - High'),
 ('Premature age-adjusted mortality', 'Age-Adjusted Mortality (Black)'),
 ('Premature age-adjusted mortality', 'Age-Adjusted Mortality (Hispanic)'),
 ('Premature age-adjusted mortality', 'Age-Adjusted Mortality (White)'),
 ('Child mortality', '# Deaths'),
 ('Child mortality', 'Child Mortality Rate'),
 ('Child mortality', '95% CI - Low'),
 ('Child mortality', '95% CI - High'),
 ('Child mortality', 'Ch

In [9]:
data_19_2 = add_index_prefix(data_19_2)

In [10]:
data_19_2.columns

Index(['FIPS', 'State', 'County', 'Life expectancy_Life Expectancy',
       'Life expectancy_95% CI - Low', 'Life expectancy_95% CI - High',
       'Life expectancy_Life Expectancy (Black)',
       'Life expectancy_Life Expectancy (Hispanic)',
       'Life expectancy_Life Expectancy (White)',
       'Premature age-adjusted mortality_# Deaths',
       ...
       'Demographics_% Hispanic', 'Demographics_# Non-Hispanic White',
       'Demographics_% Non-Hispanic White',
       'Demographics_# Not Proficient in English',
       'Demographics_% Not Proficient in English', 'Demographics_95% CI - Low',
       'Demographics_95% CI - High', 'Demographics_% Female',
       'Demographics_# Rural', 'Demographics_% Rural'],
      dtype='object', length=111)

In [11]:
data_19_3

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  FIPS              State             County   
0                29000           Missouri                NaN   
1                29001           Missouri              Adair   
2                29003           Missouri             Andrew   
3                29005           Missouri           Atchison   
4                29007           Missouri            Audrain   
..                 ...                ...                ...   
111              29223           Missouri              Wayne   
112              29225           Missouri            Webster   
113              29227           Missouri              Worth   
114              29229           Missouri             Wright   
115              29510           Missouri     St. Louis City   

                      Premature death                                        \
    Years of Potential Life Lost Rate  95% CI - Low 95% CI - High   Z-Score   
0                         8190.098356   8107.350274   8272.846439       NaN   
1                         8362.577513   6916.682651   9808.472375 -0.204258   
2                         6425.364529   5001.266073   7849.462986 -1.205514   
3                         8293.721570   6660.539451  10206.138061 -0.239846   
4                         7428.216494   6206.127514   8650.305473 -0.687186   
..                                ...           ...           ...       ...   
111                      10338.815424   8323.805743  12353.825105  0.817169   
112                       7747.667445   6743.022451   8752.312440 -0.522076   
113                               NaN           NaN           NaN  0.000000   
114                      10453.327288   8655.263155  12251.391421  0.876355   
115                      12680.579721  12227.531070  13133.628371  2.027520   

                                                              ...  \
    YPLL Rate (Black) YPLL Rate (Hispanic) YPLL Rate (White)  ...   
0                 NaN                  NaN               NaN  ...   
1                 NaN                  NaN               NaN  ...   
2                 NaN                  NaN               NaN  ...   
3                 NaN                  NaN               NaN  ...   
4        10308.442690                  NaN       7247.911997  ...   
..                ...                  ...               ...  ...   
111               NaN                  NaN               NaN  ...   
112               NaN                  NaN               NaN  ...   
113               NaN                  NaN               NaN  ...   
114               NaN                  NaN               NaN  ...   
115      18360.707387          3894.228692       8430.159818  ...   

    Driving alone to work                                  \
            95% CI - High   Z-Score % Drive Alone (Black)   
0               81.978845       NaN                   NaN   
1               79.676223 -0.621011                   NaN   
2               89.629591  1.535234                   NaN   
3               87.325366  0.888771                   NaN   
4               84.542749  0.187616                   NaN   
..                    ...       ...                   ...   
111             83.618990 -0.259435                   NaN   
112             83.401811  0.321110                   NaN   
113             81.900943 -0.782063                   NaN   
114             83.268323 -0.576362                   NaN   
115             72.859674 -1.850562             66.825083   

                                                    \
    % Drive Alone (Hispanic) % Drive Alone (White)   
0                        NaN                   NaN   
1                        NaN                   NaN   
2                        NaN                   NaN   
3                        NaN                   NaN   
4                        NaN                   NaN   
..                       ...                   ...   
111                

In [12]:
data_19_3.columns.to_list()

[('Unnamed: 0_level_0', 'FIPS'),
 ('Unnamed: 1_level_0', 'State'),
 ('Unnamed: 2_level_0', 'County'),
 ('Premature death', 'Years of Potential Life Lost Rate'),
 ('Premature death', '95% CI - Low'),
 ('Premature death', '95% CI - High'),
 ('Premature death', 'Z-Score'),
 ('Premature death', 'YPLL Rate (Black)'),
 ('Premature death', 'YPLL Rate (Hispanic)'),
 ('Premature death', 'YPLL Rate (White)'),
 ('Poor or fair health', '% Fair/Poor'),
 ('Poor or fair health', '95% CI - Low'),
 ('Poor or fair health', '95% CI - High'),
 ('Poor or fair health', 'Z-Score'),
 ('Poor physical health days', 'Physically Unhealthy Days'),
 ('Poor physical health days', '95% CI - Low'),
 ('Poor physical health days', '95% CI - High'),
 ('Poor physical health days', 'Z-Score'),
 ('Poor mental health days', 'Mentally Unhealthy Days'),
 ('Poor mental health days', '95% CI - Low'),
 ('Poor mental health days', '95% CI - High'),
 ('Poor mental health days', 'Z-Score'),
 ('Low birthweight', 'Unreliable'),
 ('Low

In [13]:
data_19_3 = add_index_prefix(data_19_3)

In [14]:
data_19_3.columns

Index(['FIPS', 'State', 'County',
       'Premature death_Years of Potential Life Lost Rate',
       'Premature death_95% CI - Low', 'Premature death_95% CI - High',
       'Premature death_Z-Score', 'Premature death_YPLL Rate (Black)',
       'Premature death_YPLL Rate (Hispanic)',
       'Premature death_YPLL Rate (White)',
       ...
       'Driving alone to work_95% CI - High', 'Driving alone to work_Z-Score',
       'Driving alone to work_% Drive Alone (Black)',
       'Driving alone to work_% Drive Alone (Hispanic)',
       'Driving alone to work_% Drive Alone (White)',
       'Long commute - driving alone_# Workers who Drive Alone',
       'Long commute - driving alone_% Long Commute - Drives Alone',
       'Long commute - driving alone_95% CI - Low',
       'Long commute - driving alone_95% CI - High',
       'Long commute - driving alone_Z-Score'],
      dtype='object', length=162)

In [15]:
data_19_4

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Length of Life  \
                  FIPS              State             County        Z-Score   
0                29000           Missouri                NaN            NaN   
1                29001           Missouri              Adair      -0.102129   
2                29003           Missouri             Andrew      -0.602757   
3                29005           Missouri           Atchison      -0.119923   
4                29007           Missouri            Audrain      -0.343593   
..                 ...                ...                ...            ...   
111              29223           Missouri              Wayne       0.408585   
112              29225           Missouri            Webster      -0.261038   
113              29227           Missouri              Worth       0.000000   
114              29229           Missouri             Wright       0.438177   
115              29510           Missouri     St. Louis City       1.013760   

           Quality of Life        Health Behaviors        Clinical Care  \
      Rank         Z-Score   Rank          Z-Score   Rank       Z-Score   
0      NaN             NaN    NaN              NaN    NaN           NaN   
1     46.0        0.287358   94.0        -0.078726   36.0     -0.262958   
2     15.0       -0.617923    7.0        -0.265267    4.0     -0.098576   
3     44.0       -0.524246    9.0        -0.248227    6.0      0.039977   
4     28.0        0.099551   72.0         0.053242   79.0     -0.065380   
..     ...             ...    ...              ...    ...           ...   
111   95.0        0.579296  105.0         0.203282  105.0      0.182511   
112   35.0       -0.089373   53.0         0.065420   82.0     -0.011315   
113   59.0        0.206137   86.0        -0.177652   15.0      0.013301   
114   97.0        0.359745   98.0         0.094470   92.0      0.116022   
115  112.0        1.070666  112.0         0.369348  112.0     -0.038655   

           Social & Economic Factors        Physical Environment         
      Rank                   Z-Score   Rank              Z-Score   Rank  
0      NaN                       NaN    NaN                  NaN    NaN  
1      5.0                  0.073525   80.0             0.029252   83.0  
2     19.0                 -0.359558    6.0            -0.020035   44.0  
3     75.0                 -0.353550    7.0            -0.070620   11.0  
4     27.0                  0.025836   66.0            -0.044417   20.0  
..     ...                       ...    ...                  ...    ...  
111  109.0                  0.341436  108.0            -0.015238   48.0  
112   49.0                  0.002218   62.0             0.057534  102.0  
113   61.0                 -0.293687   11.0            -0.057718   16.0  
114   98.0                  0.237221  100.0            -0.034381   30.0  
115   38.0                  0.800198  115.0             0.066767  105.0  

[116 rows x 15 columns]

In [16]:
data_19_4.columns.to_list()

[('Unnamed: 0_level_0', 'FIPS'),
 ('Unnamed: 1_level_0', 'State'),
 ('Unnamed: 2_level_0', 'County'),
 ('Length of Life', 'Z-Score'),
 ('Length of Life', 'Rank'),
 ('Quality of Life', 'Z-Score'),
 ('Quality of Life', 'Rank'),
 ('Health Behaviors', 'Z-Score'),
 ('Health Behaviors', 'Rank'),
 ('Clinical Care', 'Z-Score'),
 ('Clinical Care', 'Rank'),
 ('Social & Economic Factors', 'Z-Score'),
 ('Social & Economic Factors', 'Rank'),
 ('Physical Environment', 'Z-Score'),
 ('Physical Environment', 'Rank')]

In [17]:
data_19_4 = add_index_prefix(data_19_4)

In [18]:
data_19_4.columns

Index(['FIPS', 'State', 'County', 'Length of Life_Z-Score',
       'Length of Life_Rank', 'Quality of Life_Z-Score',
       'Quality of Life_Rank', 'Health Behaviors_Z-Score',
       'Health Behaviors_Rank', 'Clinical Care_Z-Score', 'Clinical Care_Rank',
       'Social & Economic Factors_Z-Score', 'Social & Economic Factors_Rank',
       'Physical Environment_Z-Score', 'Physical Environment_Rank'],
      dtype='object')

In [19]:
chr19 = pd.merge(data_19_1 , data_19_2 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr19 = pd.merge(chr19 , data_19_3 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr19 = pd.merge(chr19 , data_19_4 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr19

,FIPS,State,County,Health Outcomes_Z-Score,Health Outcomes_Rank,Health Factors_Z-Score,Health Factors_Rank,Life expectancy_Life Expectancy,Life expectancy_95% CI - Low,Life expectancy_95% CI - High,...,Quality of Life_Z-Score,Quality of Life_Rank,Health Behaviors_Z-Score,Health Behaviors_Rank,Clinical Care_Z-Score,Clinical Care_Rank,Social & Economic Factors_Z-Score,Social & Economic Factors_Rank,Physical Environment_Z-Score,Physical Environment_Rank
0,29000,Missouri,NaN,NaN,NaN,NaN,NaN,77.403222,77.329386,77.477059,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29001,Missouri,Adair,0.185230,71.0,-0.238906,36.0,77.290893,75.984695,78.597091,...,0.287358,94.0,-0.078726,36.0,-0.262958,5.0,0.073525,80.0,0.029252,83.0
2,29003,Missouri,Andrew,-1.220680,8.0,-0.743436,6.0,79.482247,78.082630,80.881864,...,-0.617923,7.0,-0.265267,4.0,-0.098576,19.0,-0.359558,6.0,-0.020035,44.0
3,29005,Missouri,Atchison,-0.644169,24.0,-0.632420,9.0,77.575026,74.968435,80.181617,...,-0.524246,9.0,-0.248227,6.0,0.039977,75.0,-0.353550,7.0,-0.070620,11.0
4,29007,Missouri,Audrain,-0.244042,50.0,-0.030719,58.0,76.848305,75.813400,77.883211,...,0.099551,72.0,0.053242,79.0,-0.065380,27.0,0.025836,66.0,-0.044417,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,29223,Missouri,Wayne,0.987880,103.0,0.711992,108.0,74.692964,73.156515,76.229414,...,0.579296,105.0,0.203282,105.0,0.182511,109.0,0.341436,108.0,-0.015238,48.0
112,29225,Missouri,Webster,-0.350411,43.0,0.113857,74.0,77.388763,76.475134,78.302393,...,-0.089373,53.0,0.065420,82.0,-0.011315,49.0,0.002218,62.0,0.057534,102.0
113,29227,Missouri,Worth,0.206137,72.0,-0.515757,11.0,71.766790,66.051351,77.482229,...,0.206137,86.0,-0.177652,15.0,0.013301,61.0,-0.293687,11.0,-0.057718,16.0
114,29229,Missouri,Wright,0.797923,97.0,0.413333,100.0,74.689066,73.355217,76.022914,...,0.359745,98.0,0.094470,92.0,0.116022,98.0,0.237221,100.0,-0.034381,30.0


In [20]:
chr19['year'] = 2019
chr19

,FIPS,State,County,Health Outcomes_Z-Score,Health Outcomes_Rank,Health Factors_Z-Score,Health Factors_Rank,Life expectancy_Life Expectancy,Life expectancy_95% CI - Low,Life expectancy_95% CI - High,...,Quality of Life_Rank,Health Behaviors_Z-Score,Health Behaviors_Rank,Clinical Care_Z-Score,Clinical Care_Rank,Social & Economic Factors_Z-Score,Social & Economic Factors_Rank,Physical Environment_Z-Score,Physical Environment_Rank,year
0,29000,Missouri,NaN,NaN,NaN,NaN,NaN,77.403222,77.329386,77.477059,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
1,29001,Missouri,Adair,0.185230,71.0,-0.238906,36.0,77.290893,75.984695,78.597091,...,94.0,-0.078726,36.0,-0.262958,5.0,0.073525,80.0,0.029252,83.0,2019
2,29003,Missouri,Andrew,-1.220680,8.0,-0.743436,6.0,79.482247,78.082630,80.881864,...,7.0,-0.265267,4.0,-0.098576,19.0,-0.359558,6.0,-0.020035,44.0,2019
3,29005,Missouri,Atchison,-0.644169,24.0,-0.632420,9.0,77.575026,74.968435,80.181617,...,9.0,-0.248227,6.0,0.039977,75.0,-0.353550,7.0,-0.070620,11.0,2019
4,29007,Missouri,Audrain,-0.244042,50.0,-0.030719,58.0,76.848305,75.813400,77.883211,...,72.0,0.053242,79.0,-0.065380,27.0,0.025836,66.0,-0.044417,20.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,29223,Missouri,Wayne,0.987880,103.0,0.711992,108.0,74.692964,73.156515,76.229414,...,105.0,0.203282,105.0,0.182511,109.0,0.341436,108.0,-0.015238,48.0,2019
112,29225,Missouri,Webster,-0.350411,43.0,0.113857,74.0,77.388763,76.475134,78.302393,...,53.0,0.065420,82.0,-0.011315,49.0,0.002218,62.0,0.057534,102.0,2019
113,29227,Missouri,Worth,0.206137,72.0,-0.515757,11.0,71.766790,66.051351,77.482229,...,86.0,-0.177652,15.0,0.013301,61.0,-0.293687,11.0,-0.057718,16.0,2019
114,29229,Missouri,Wright,0.797923,97.0,0.413333,100.0,74.689066,73.355217,76.022914,...,98.0,0.094470,92.0,0.116022,98.0,0.237221,100.0,-0.034381,30.0,2019


In [21]:
chr19.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Z-Score',
 'Health Outcomes_Rank',
 'Health Factors_Z-Score',
 'Health Factors_Rank',
 'Life expectancy_Life Expectancy',
 'Life expectancy_95% CI - Low',
 'Life expectancy_95% CI - High',
 'Life expectancy_Life Expectancy (Black)',
 'Life expectancy_Life Expectancy (Hispanic)',
 'Life expectancy_Life Expectancy (White)',
 'Premature age-adjusted mortality_# Deaths',
 'Premature age-adjusted mortality_Age-Adjusted Mortality',
 'Premature age-adjusted mortality_95% CI - Low',
 'Premature age-adjusted mortality_95% CI - High',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Black)',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic)',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (White)',
 'Child mortality_# Deaths',
 'Child mortality_Child Mortality Rate',
 'Child mortality_95% CI - Low',
 'Child mortality_95% CI - High',
 'Child mortality_Child Mortality Rate (Black)',
 'Child mortality_Child Mort

In [23]:
# Columns to be dropped
columns_to_drop = ['Health Outcomes_Z-Score',
 'Health Factors_Z-Score',
 'Life expectancy_95% CI - Low',
 'Life expectancy_95% CI - High',
 'Life expectancy_Life Expectancy (Black)',
 'Life expectancy_Life Expectancy (Hispanic)',
 'Life expectancy_Life Expectancy (White)',
 'Premature age-adjusted mortality_95% CI - Low',
 'Premature age-adjusted mortality_95% CI - High',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Black)',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic)',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (White)',
 'Child mortality_# Deaths',
 'Child mortality_Child Mortality Rate',
 'Child mortality_95% CI - Low',
 'Child mortality_95% CI - High',
 'Child mortality_Child Mortality Rate (Black)',
 'Child mortality_Child Mortality Rate (Hispanic)',
 'Child mortality_Child Mortality Rate (White)',
 'Infant mortality_# Deaths',
 'Infant mortality_Infant Mortality Rate',
 'Infant mortality_95% CI - Low',
 'Infant mortality_95% CI - High',
 'Infant mortality_Infant Mortality Rate (Black)',
 'Infant mortality_Infant Mortality Rate (Hispanic)',
 'Infant mortality_Infant Mortality Rate (White)',
 'Frequent physical distress_95% CI - Low',
 'Frequent physical distress_95% CI - High',
 'Frequent mental distress_95% CI - Low',
 'Frequent mental distress_95% CI - High',
 'Diabetes prevalence_95% CI - Low',
 'Diabetes prevalence_95% CI - High',
 'HIV prevalence_# HIV Cases',
 'HIV prevalence_HIV Prevalence Rate',
 'Drug overdose deaths_# Drug Overdose Deaths',
 'Drug overdose deaths_Drug Overdose Mortality Rate',
 'Motor vehicle crash deaths_# Motor Vehicle Deaths',
 'Motor vehicle crash deaths_MV Mortality Rate',
 'Motor vehicle crash deaths_95% CI - Low',
 'Motor vehicle crash deaths_95% CI - High',
 'Insufficient sleep_% Insufficient Sleep',
 'Insufficient sleep_95% CI - Low',
 'Insufficient sleep_95% CI - High',
 'Uninsured adults_95% CI - Low',
 'Uninsured adults_95% CI - High',
 'Uninsured children_# Uninsured',
 'Uninsured children_% Uninsured',
 'Uninsured children_95% CI - Low',
 'Uninsured children_95% CI - High',
 'Disconnected youth_% Disconnected Youth',
 'Median household income_95% CI - Low',
 'Median household income_95% CI - High',
 'Median household income_Household income (Black)',
 'Median household income_Household income (Hispanic)',
 'Median household income_Household income (White)',
 'Children eligible for free or reduced price lunch_% Free or Reduced Lunch',
 'Residential segregation - black/white_Segregation index',
 'Residential segregation - non-white/white_Segregation Index',
 'Homicides_Homicide Rate',
 'Homicides_95% CI - Low',
 'Homicides_95% CI - High',
 'Firearm fatalities_# Firearm Fatalities',
 'Firearm fatalities_Firearm Fatalities Rate',
 'Firearm fatalities_95% CI - Low',
 'Firearm fatalities_95% CI - High',
 'Homeownership_# Homeowners',
 'Homeownership_% Homeowners',
 'Homeownership_95% CI - Low',
 'Homeownership_95% CI - High',
 'Severe housing cost burden_# Households with Severe Cost Burden',
 'Severe housing cost burden_% Severe Housing Cost Burden',
 'Severe housing cost burden_95% CI - Low',
 'Severe housing cost burden_95% CI - High',
 'Demographics_% < 18',
 'Demographics_# African American',
 'Demographics_% African American',
 'Demographics_# American Indian/Alaskan Native',
 'Demographics_% American Indian/Alaskan Native',
 'Demographics_# Asian',
 'Demographics_% Asian',
 'Demographics_# Native Hawaiian/Other Pacific Islander',
 'Demographics_% Native Hawaiian/Other Pacific Islander',
 'Demographics_# Hispanic',
 'Demographics_% Hispanic',
 'Demographics_# Non-Hispanic White',
 'Demographics_% Non-Hispanic White',
 'Demographics_# Not Proficient in English',
 'Demographics_% Not Proficient in English',
 'Demographics_95% CI - Low',
 'Demographics_95% CI - High',
 'Demographics_% Female',
 'Premature death_Years of Potential Life Lost Rate',
 'Premature death_95% CI - Low',
 'Premature death_95% CI - High',
 'Premature death_Z-Score',
 'Premature death_YPLL Rate (Black)',
 'Premature death_YPLL Rate (Hispanic)',
 'Premature death_YPLL Rate (White)',
 'Poor or fair health_95% CI - Low',
 'Poor or fair health_95% CI - High',
 'Poor or fair health_Z-Score',
 'Poor physical health days_95% CI - Low',
 'Poor physical health days_95% CI - High',
 'Poor physical health days_Z-Score',
 'Poor mental health days_95% CI - Low',
 'Poor mental health days_95% CI - High',
 'Poor mental health days_Z-Score',
 'Low birthweight_Unreliable',
 'Low birthweight_% LBW',
 'Low birthweight_95% CI - Low',
 'Low birthweight_95% CI - High',
 'Low birthweight_Z-Score',
 'Low birthweight_% LBW (Black)',
 'Low birthweight_% LBW (Hispanic)',
 'Low birthweight_% LBW (White)',
 'Adult smoking_95% CI - Low',
 'Adult smoking_95% CI - High',
 'Adult smoking_Z-Score',
 'Adult obesity_95% CI - Low',
 'Adult obesity_95% CI - High',
 'Adult obesity_Z-Score',
 'Food environment index_Z-Score',
 'Physical inactivity_% Physically Inactive',
 'Physical inactivity_95% CI - Low',
 'Physical inactivity_95% CI - High',
 'Physical inactivity_Z-Score',
 'Access to exercise opportunities_Z-Score',
 'Excessive drinking_% Excessive Drinking',
 'Excessive drinking_95% CI - Low',
 'Excessive drinking_95% CI - High',
 'Excessive drinking_Z-Score',
 'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths',
 'Alcohol-impaired driving deaths_# Driving Deaths',
 'Alcohol-impaired driving deaths_% Alcohol-Impaired',
 'Alcohol-impaired driving deaths_95% CI - Low',
 'Alcohol-impaired driving deaths_95% CI - High',
 'Alcohol-impaired driving deaths_Z-Score',
 'Sexually transmitted infections_# Chlamydia Cases',
 'Sexually transmitted infections_Chlamydia Rate',
 'Sexually transmitted infections_Z-Score',
 'Teen births_Teen Birth Rate',
 'Teen births_95% CI - Low',
 'Teen births_95% CI - High',
 'Teen births_Z-Score',
 'Teen births_Teen Birth Rate (Black)',
 'Teen births_Teen Birth Rate (Hispanic)',
 'Teen births_Teen Birth Rate (White)',
 'Uninsured_95% CI - Low',
 'Uninsured_95% CI - High',
 'Uninsured_Z-Score',
 'Primary care physicians_Z-Score',
 'Dentists_# Dentists',
 'Dentists_Dentist Rate',
 'Dentists_Dentist Ratio',
 'Dentists_Z-Score',
 'Mental health providers_Z-Score',
 'Preventable hospital stays_Preventable Hosp. Rate',
 'Preventable hospital stays_Z-Score',
 'Preventable hospital stays_Preventable Hosp. Rate (Black)',
 'Preventable hospital stays_Preventable Hosp. Rate (Hispanic)',
 'Preventable hospital stays_Preventable Hosp. Rate (White)',
 'Mammography screening_% Screened',
 'Mammography screening_Z-Score',
 'Mammography screening_% Screened (Black)',
 'Mammography screening_% Screened (Hispanic)',
 'Mammography screening_% Screened (White)',
 'Flu vaccinations_% Vaccinated',
 'Flu vaccinations_Z-Score',
 'Flu vaccinations_% Vaccinated (Black)',
 'Flu vaccinations_% Vaccinated (Hispanic)',
 'Flu vaccinations_% Vaccinated (White)',
 'High school graduation_Cohort Size',
 'High school graduation_Graduation Rate',
 'High school graduation_Z-Score',
 'Some college_95% CI - Low',
 'Some college_95% CI - High',
 'Some college_Z-Score',
 'Unemployment_Z-Score',
 'Children in poverty_% Children in Poverty',
 'Children in poverty_95% CI - Low',
 'Children in poverty_95% CI - High',
 'Children in poverty_Z-Score',
 'Children in poverty_% Children in Poverty (Black)',
 'Children in poverty_% Children in Poverty (Hispanic)',
 'Children in poverty_% Children in Poverty (White)',
 'Income inequality_80th Percentile Income',
 'Income inequality_20th Percentile Income',
 'Income inequality_Income Ratio',
 'Income inequality_Z-Score',
 'Children in single-parent households_# Single-Parent Households',
 'Children in single-parent households_# Households',
 'Children in single-parent households_% Single-Parent Households',
 'Children in single-parent households_95% CI - Low',
 'Children in single-parent households_95% CI - High',
 'Children in single-parent households_Z-Score',
 'Social associations_# Associations',
 'Social associations_Association Rate',
 'Social associations_Z-Score',
 'Violent crime_Z-Score',
 'Injury deaths_# Injury Deaths',
 'Injury deaths_Injury Death Rate',
 'Injury deaths_95% CI - Low',
 'Injury deaths_95% CI - High',
 'Injury deaths_Z-Score',
 'Air pollution - particulate matter_Average Daily PM2.5',
 'Air pollution - particulate matter_Z-Score',
 'Drinking water violations_Presence of violation',
 'Drinking water violations_Z-Score',
 'Severe housing problems_95% CI - Low',
 'Severe housing problems_95% CI - High',
 'Severe housing problems_Severe Housing Cost Burden',
 'Severe housing problems_Overcrowding',
 'Severe housing problems_Inadequate Facilities',
 'Severe housing problems_Z-Score',
 'Driving alone to work_% Drive Alone',
 'Driving alone to work_95% CI - Low',
 'Driving alone to work_95% CI - High',
 'Driving alone to work_Z-Score',
 'Driving alone to work_% Drive Alone (Black)',
 'Driving alone to work_% Drive Alone (Hispanic)',
 'Driving alone to work_% Drive Alone (White)',
 'Long commute - driving alone_# Workers who Drive Alone',
 'Long commute - driving alone_% Long Commute - Drives Alone',
 'Long commute - driving alone_95% CI - Low',
 'Long commute - driving alone_95% CI - High',
 'Long commute - driving alone_Z-Score',
 'Length of Life_Z-Score',
 'Length of Life_Rank',
 'Quality of Life_Z-Score',
 'Quality of Life_Rank',
 'Health Behaviors_Z-Score',
 'Clinical Care_Z-Score',
 'Social & Economic Factors_Z-Score',
 'Physical Environment_Z-Score']

In [24]:
# Dropping columns in-place
chr19.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr19)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  71.0   
2    29003  Missouri          Andrew                   8.0   
3    29005  Missouri        Atchison                  24.0   
4    29007  Missouri         Audrain                  50.0   
..     ...       ...             ...                   ...   
111  29223  Missouri           Wayne                 103.0   
112  29225  Missouri         Webster                  43.0   
113  29227  Missouri           Worth                  72.0   
114  29229  Missouri          Wright                  97.0   
115  29510  Missouri  St. Louis City                 112.0   

     Health Factors_Rank  Life expectancy_Life Expectancy  \
0                    NaN                        77.403222   
1                   36.0                        77.290893   
2                    6.0    

In [25]:
chr19.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Life expectancy_Life Expectancy',
 'Premature age-adjusted mortality_# Deaths',
 'Premature age-adjusted mortality_Age-Adjusted Mortality',
 'Frequent physical distress_% Frequent Physical Distress',
 'Frequent mental distress_% Frequent Mental Distress',
 'Diabetes prevalence_% Diabetic',
 'Food insecurity_# Food Insecure',
 'Food insecurity_% Food Insecure',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Uninsured adults_# Uninsured',
 'Uninsured adults_% Uninsured',
 'Other primary care providers_Other PCP Rate',
 'Other primary care providers_Other PCP Ratio',
 'Median household income_Household Income',
 'Demographics_Population',
 'Demographics_% 65 and over',
 'Demographics_# Rural',
 'Demographics_% Rural',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unheal

In [114]:
# Columns to be dropped
columns_to_drop = [
 'Life expectancy_Life Expectancy',
 'Premature age-adjusted mortality_# Deaths',
 'Premature age-adjusted mortality_Age-Adjusted Mortality',
 'Frequent physical distress_% Frequent Physical Distress',
 'Frequent mental distress_% Frequent Mental Distress',
 'Diabetes prevalence_% Diabetic',
 'Food insecurity_# Food Insecure',
 'Food insecurity_% Food Insecure',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Uninsured adults_# Uninsured',
 'Other primary care providers_Other PCP Rate',
 'Other primary care providers_Other PCP Ratio',
 'Median household income_Household Income',
 'Demographics_Population',
 'Demographics_% 65 and over',
 'Demographics_# Rural',
 'Demographics_% Rural',
 'Food environment index_Food Environment Index',
 'Access to exercise opportunities_% With Access',
 'Uninsured_# Uninsured',
 'Uninsured_% Uninsured',
 'Primary care physicians_# Primary Care Physicians',
 'Primary care physicians_PCP Rate',
 'Primary care physicians_PCP Ratio',
 'Mental health providers_# Mental Health Providers',
 'Mental health providers_MHP Rate',
 'Mental health providers_MHP Ratio',
 'Some college_# Some College',
 'Some college_Population',
 'Some college_% Some College',
 'Unemployment_# Unemployed',
 'Unemployment_Labor Force',
 'Violent crime_Annual Average Violent Crimes',
 'Severe housing problems_% Severe Housing Problems']

In [115]:
# Dropping columns in-place
chr19.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr19)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  71.0   
2    29003  Missouri          Andrew                   8.0   
3    29005  Missouri        Atchison                  24.0   
4    29007  Missouri         Audrain                  50.0   
5    29009  Missouri           Barry                  86.0   
6    29011  Missouri          Barton                  65.0   
7    29013  Missouri           Bates                  48.0   
8    29015  Missouri          Benton                  90.0   
9    29017  Missouri       Bollinger                  76.0   
10   29019  Missouri           Boone                   9.0   
11   29021  Missouri        Buchanan                  75.0   
12   29023  Missouri          Butler                 106.0   
13   29025  Missouri        Caldwell                  27.0   
14   29027  Missouri      

In [203]:
chr19.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Uninsured adults_% Uninsured',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Unemployment_% Unemployed',
 'Violent Crime Rate',
 'Health Behaviors_Rank',
 'Clinical Care_Rank',
 'Social & Economic Factors_Rank',
 'Physical Environment_Rank',
 'year']

In [202]:
chr19.rename(columns={'Violent crime_Violent Crime Rate': 'Violent Crime Rate'}, inplace=True)

In [26]:
data_18_1 = pd.read_excel('2018 County Health Rankings Missouri Data - v3.xls',sheet_name = 'Outcomes & Factors Rankings', header = [0,1])
data_18_2 = pd.read_excel('2018 County Health Rankings Missouri Data - v3.xls',sheet_name = 'Additional Measure Data', header = [0,1])
data_18_3 = pd.read_excel('2018 County Health Rankings Missouri Data - v3.xls',sheet_name = 'Ranked Measure Data', header = [0,1])
data_18_4 = pd.read_excel('2018 County Health Rankings Missouri Data - v3.xls',sheet_name = 'Outcomes & Factors SubRankings', header = [0,1])

In [27]:
data_18_1 = add_index_prefix(data_18_1)
data_18_2 = add_index_prefix(data_18_2)
data_18_3 = add_index_prefix(data_18_3)
data_18_4 = add_index_prefix(data_18_4)

display(data_18_1.columns,data_18_2.columns, data_18_3.columns , data_18_4.columns)

Index(['FIPS', 'State', 'County', 'Health Outcomes_Z-Score',
       'Health Outcomes_Rank', 'Health Factors_Z-Score',
       'Health Factors_Rank'],
      dtype='object')

Index(['FIPS', 'State', 'County', 'Premature age-adjusted mortality_# Deaths',
       'Premature age-adjusted mortality_Age-Adjusted Mortality',
       'Premature age-adjusted mortality_95% CI - Low',
       'Premature age-adjusted mortality_95% CI - High',
       'Premature age-adjusted mortality_Age-Adjusted Mortality (Black)',
       'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic)',
       'Premature age-adjusted mortality_Age-Adjusted Mortality (White)',
       'Child mortality_# Deaths', 'Child mortality_Child Mortality Rate',
       'Child mortality_95% CI - Low', 'Child mortality_95% CI - High',
       'Child mortality_Child Mortality Rate (Black)',
       'Child mortality_Child Mortality Rate (Hispanic)',
       'Child mortality_Child Mortality Rate (White)',
       'Infant mortality_# Deaths', 'Infant mortality_Infant Mortality Rate',
       'Infant mortality_95% CI - Low', 'Infant mortality_95% CI - High',
       'Infant mortality_Infant Mortality Rate (Bl

Index(['FIPS', 'State', 'County',
       'Premature death_Years of Potential Life Lost Rate',
       'Premature death_95% CI - Low', 'Premature death_95% CI - High',
       'Premature death_Z-Score',
       'Premature death_Years of Potential Life Lost Rate (Black)',
       'Premature death_Years of Potential Life Lost Rate (Hispanic)',
       'Premature death_Years of Potential Life Lost Rate (White)',
       ...
       'Driving alone to work_95% CI - High', 'Driving alone to work_Z-Score',
       'Driving alone to work_% Drive Alone (Black)',
       'Driving alone to work_% Drive Alone (Hispanic)',
       'Driving alone to work_% Drive Alone (White)',
       'Long commute - driving alone_# Workers who Drive Alone',
       'Long commute - driving alone_% Long Commute - Drives Alone',
       'Long commute - driving alone_95% CI - Low',
       'Long commute - driving alone_95% CI - High',
       'Long commute - driving alone_Z-Score'],
      dtype='object', length=164)

Index(['FIPS', 'State', 'County', 'Length of Life_Z-Score',
       'Length of Life_Rank', 'Quality of Life_Z-Score',
       'Quality of Life_Rank', 'Health Behaviors_Z-Score',
       'Health Behaviors_Rank', 'Clinical Care_Z-Score', 'Clinical Care_Rank',
       'Social & Economic Factors_Z-Score', 'Social & Economic Factors_Rank',
       'Physical Environment_Z-Score', 'Physical Environment_Rank'],
      dtype='object')

In [28]:
chr18 = pd.merge(data_18_1 , data_18_2 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr18 = pd.merge(chr18 , data_18_3 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr18 = pd.merge(chr18 , data_18_4 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr18

,FIPS,State,County,Health Outcomes_Z-Score,Health Outcomes_Rank,Health Factors_Z-Score,Health Factors_Rank,Premature age-adjusted mortality_# Deaths,Premature age-adjusted mortality_Age-Adjusted Mortality,Premature age-adjusted mortality_95% CI - Low,...,Quality of Life_Z-Score,Quality of Life_Rank,Health Behaviors_Z-Score,Health Behaviors_Rank,Clinical Care_Z-Score,Clinical Care_Rank,Social & Economic Factors_Z-Score,Social & Economic Factors_Rank,Physical Environment_Z-Score,Physical Environment_Rank
0,29000,Missouri,NaN,NaN,NaN,NaN,NaN,81491,390.4,387.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29001,Missouri,Adair,-0.027039,69.0,-0.285616,27.0,298,402.8,355.1,...,0.205034,87.0,-0.109545,26.0,-0.203694,7.0,0.049941,70.0,-0.022318,41.0
2,29003,Missouri,Andrew,-1.109202,8.0,-0.647800,8.0,217,332.1,286.3,...,-0.524103,10.0,-0.239185,9.0,-0.104706,22.0,-0.325093,10.0,0.021184,75.0
3,29005,Missouri,Atchison,-1.120573,7.0,-0.475412,14.0,96,410.9,325.8,...,-0.588248,5.0,-0.245472,5.0,0.139964,102.0,-0.291468,12.0,-0.078436,6.0
4,29007,Missouri,Audrain,-0.376861,42.0,-0.154913,43.0,327,354.4,314.9,...,-0.003355,64.0,0.029217,66.0,-0.108203,21.0,-0.016730,58.0,-0.059196,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,29223,Missouri,Wayne,1.515640,108.0,0.410425,100.0,294,523.7,458.6,...,0.524260,104.0,0.150860,103.0,0.069025,84.0,0.228607,99.0,-0.038068,28.0
112,29225,Missouri,Webster,-0.228662,55.0,0.044289,63.0,449,352.0,318.7,...,-0.071392,50.0,0.070131,79.0,0.001078,58.0,-0.022174,57.0,-0.004746,56.0
113,29227,Missouri,Worth,-0.030367,67.0,-0.613672,9.0,39,469.7,327.1,...,-0.030367,58.0,-0.151158,19.0,-0.062136,35.0,-0.304385,11.0,-0.095993,2.0
114,29229,Missouri,Wright,0.668493,93.0,0.355668,97.0,327,485.2,429.8,...,0.403362,99.0,0.160938,104.0,0.065235,82.0,0.184459,92.0,-0.054965,17.0


In [29]:
chr18['year'] = 2018

In [30]:
chr18

,FIPS,State,County,Health Outcomes_Z-Score,Health Outcomes_Rank,Health Factors_Z-Score,Health Factors_Rank,Premature age-adjusted mortality_# Deaths,Premature age-adjusted mortality_Age-Adjusted Mortality,Premature age-adjusted mortality_95% CI - Low,...,Quality of Life_Rank,Health Behaviors_Z-Score,Health Behaviors_Rank,Clinical Care_Z-Score,Clinical Care_Rank,Social & Economic Factors_Z-Score,Social & Economic Factors_Rank,Physical Environment_Z-Score,Physical Environment_Rank,year
0,29000,Missouri,NaN,NaN,NaN,NaN,NaN,81491,390.4,387.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
1,29001,Missouri,Adair,-0.027039,69.0,-0.285616,27.0,298,402.8,355.1,...,87.0,-0.109545,26.0,-0.203694,7.0,0.049941,70.0,-0.022318,41.0,2018
2,29003,Missouri,Andrew,-1.109202,8.0,-0.647800,8.0,217,332.1,286.3,...,10.0,-0.239185,9.0,-0.104706,22.0,-0.325093,10.0,0.021184,75.0,2018
3,29005,Missouri,Atchison,-1.120573,7.0,-0.475412,14.0,96,410.9,325.8,...,5.0,-0.245472,5.0,0.139964,102.0,-0.291468,12.0,-0.078436,6.0,2018
4,29007,Missouri,Audrain,-0.376861,42.0,-0.154913,43.0,327,354.4,314.9,...,64.0,0.029217,66.0,-0.108203,21.0,-0.016730,58.0,-0.059196,16.0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,29223,Missouri,Wayne,1.515640,108.0,0.410425,100.0,294,523.7,458.6,...,104.0,0.150860,103.0,0.069025,84.0,0.228607,99.0,-0.038068,28.0,2018
112,29225,Missouri,Webster,-0.228662,55.0,0.044289,63.0,449,352.0,318.7,...,50.0,0.070131,79.0,0.001078,58.0,-0.022174,57.0,-0.004746,56.0,2018
113,29227,Missouri,Worth,-0.030367,67.0,-0.613672,9.0,39,469.7,327.1,...,58.0,-0.151158,19.0,-0.062136,35.0,-0.304385,11.0,-0.095993,2.0,2018
114,29229,Missouri,Wright,0.668493,93.0,0.355668,97.0,327,485.2,429.8,...,99.0,0.160938,104.0,0.065235,82.0,0.184459,92.0,-0.054965,17.0,2018


In [31]:
# Columns to be dropped
columns_to_drop = columns_to_drop = [
 'Health Outcomes_Z-Score',
 'Health Factors_Z-Score',
 'Premature age-adjusted mortality_# Deaths',
 'Premature age-adjusted mortality_Age-Adjusted Mortality',
 'Premature age-adjusted mortality_95% CI - Low',
 'Premature age-adjusted mortality_95% CI - High',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Black)',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (Hispanic)',
 'Premature age-adjusted mortality_Age-Adjusted Mortality (White)',
 'Child mortality_# Deaths',
 'Child mortality_Child Mortality Rate',
 'Child mortality_95% CI - Low',
 'Child mortality_95% CI - High',
 'Child mortality_Child Mortality Rate (Black)',
 'Child mortality_Child Mortality Rate (Hispanic)',
 'Child mortality_Child Mortality Rate (White)',
 'Infant mortality_# Deaths',
 'Infant mortality_Infant Mortality Rate',
 'Infant mortality_95% CI - Low',
 'Infant mortality_95% CI - High',
 'Infant mortality_Infant Mortality Rate (Black)',
 'Infant mortality_Infant Mortality Rate (Hispanic)',
 'Infant mortality_Infant Mortality Rate (White)',
 'Frequent physical distress_95% CI - Low',
 'Frequent physical distress_95% CI - High',
 'Frequent mental distress_95% CI - Low',
 'Frequent mental distress_95% CI - High',
 'Diabetes prevalence_95% CI - Low',
 'Diabetes prevalence_95% CI - High',
 'HIV prevalence_# HIV Cases',
 'HIV prevalence_HIV Prevalence Rate',
 'Drug overdose deaths_# Drug Overdose Deaths',
 'Drug overdose deaths_Drug Overdose Mortality Rate',
 'Drug overdose deaths - modeled_Range Drug Overdose Mortality Rate',
 'Motor vehicle crash deaths_# Motor Vehicle Deaths',
 'Motor vehicle crash deaths_MV Mortality Rate',
 'Motor vehicle crash deaths_95% CI - Low',
 'Motor vehicle crash deaths_95% CI - High',
 'Insufficient sleep_% Insufficient Sleep',
 'Insufficient sleep_95% CI - Low',
 'Insufficient sleep_95% CI - High',
 'Uninsured adults_95% CI - Low',
 'Uninsured adults_95% CI - High',
 'Uninsured children_# Uninsured',
 'Uninsured children_% Uninsured',
 'Uninsured children_95% CI - Low',
 'Uninsured children_95% CI - High',
 'Health care costs_Costs',
 'Disconnected youth_% Disconnected Youth',
 'Median household income_95% CI - Low',
 'Median household income_95% CI - High',
 'Median household income_Household income (Black)',
 'Median household income_Household income (Hispanic)',
 'Median household income_Household income (White)',
 'Children eligible for free or reduced price lunch_% Free or Reduced Lunch',
 'Residential segregation - black/white_Segregation index',
 'Residential segregation - non-white/white_Segregation Index',
 'Homicides_Homicide Rate',
 'Homicides_95% CI - Low',
 'Homicides_95% CI - High',
 'Firearm fatalities_# Firearm Fatalities',
 'Firearm fatalities_Firearm Fatalities Rate',
 'Firearm fatalities_95% CI - Low',
 'Firearm fatalities_95% CI - High',
 'Demographics_# African American',
 'Demographics_% African American',
 'Demographics_# American Indian/Alaskan Native',
 'Demographics_% American Indian/Alaskan Native',
 'Demographics_# Asian',
 'Demographics_% Asian',
 'Demographics_# Native Hawaiian/Other Pacific Islander',
 'Demographics_% Native Hawaiian/Other Pacific Islander',
 'Demographics_# Hispanic',
 'Demographics_% Hispanic',
 'Demographics_# Non-Hispanic White',
 'Demographics_% Non-Hispanic White',
 'Demographics_# Not Proficient in English',
 'Demographics_% Not Proficient in English',
 'Demographics_95% CI - Low',
 'Demographics_95% CI - High',
 'Demographics_% Female',
 'Premature death_Years of Potential Life Lost Rate',
 'Premature death_95% CI - Low',
 'Premature death_95% CI - High',
 'Premature death_Z-Score',
 'Premature death_Years of Potential Life Lost Rate (Black)',
 'Premature death_Years of Potential Life Lost Rate (Hispanic)',
 'Premature death_Years of Potential Life Lost Rate (White)',
 'Poor or fair health_95% CI - Low',
 'Poor or fair health_95% CI - High',
 'Poor or fair health_Z-Score',
 'Poor physical health days_95% CI - Low',
 'Poor physical health days_95% CI - High',
 'Poor physical health days_Z-Score',
 'Poor mental health days_95% CI - Low',
 'Poor mental health days_95% CI - High',
 'Poor mental health days_Z-Score',
 'Low birthweight_Unreliable',
 'Low birthweight_% LBW',
 'Low birthweight_95% CI - Low',
 'Low birthweight_95% CI - High',
 'Low birthweight_Z-Score',
 'Low birthweight_% LBW (Black)',
 'Low birthweight_% LBW (Hispanic)',
 'Low birthweight_% LBW (White)',
 'Adult smoking_95% CI - Low',
 'Adult smoking_95% CI - High',
 'Adult smoking_Z-Score',
 'Adult obesity_95% CI - Low',
 'Adult obesity_95% CI - High',
 'Adult obesity_Z-Score',
 'Food environment index_Z-Score',
 'Physical inactivity_95% CI - Low',
 'Physical inactivity_95% CI - High',
 'Physical inactivity_Z-Score',
 'Access to exercise opportunities_Z-Score',
 'Excessive drinking_% Excessive Drinking',
 'Excessive drinking_95% CI - Low',
 'Excessive drinking_95% CI - High',
 'Excessive drinking_Z-Score',
 'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths',
 'Alcohol-impaired driving deaths_# Driving Deaths',
 'Alcohol-impaired driving deaths_% Alcohol-Impaired',
 'Alcohol-impaired driving deaths_95% CI - Low',
 'Alcohol-impaired driving deaths_95% CI - High',
 'Alcohol-impaired driving deaths_Z-Score',
 'Sexually transmitted infections_# Chlamydia Cases',
 'Sexually transmitted infections_Chlamydia Rate',
 'Sexually transmitted infections_Z-Score',
 'Teen births_Teen Birth Rate',
 'Teen births_95% CI - Low',
 'Teen births_95% CI - High',
 'Teen births_Z-Score',
 'Teen births_Teen Birth Rate (Black)',
 'Teen births_Teen Birth Rate (Hispanic)',
 'Teen births_Teen Birth Rate (White)',
 'Uninsured_95% CI - Low',
 'Uninsured_95% CI - High',
 'Uninsured_Z-Score',
 'Primary care physicians_Z-Score',
 'Dentists_# Dentists',
 'Dentists_Dentist Rate',
 'Dentists_Dentist Ratio',
 'Dentists_Z-Score',
 'Mental health providers_Z-Score',
 'Preventable hospital stays_# Medicare Enrollees',
 'Preventable hospital stays_Preventable Hosp. Rate',
 'Preventable hospital stays_95% CI - Low',
 'Preventable hospital stays_95% CI - High',
 'Preventable hospital stays_Z-Score',
 'Diabetes monitoring_# Diabetics',
 'Diabetes monitoring_% Receiving HbA1c',
 'Diabetes monitoring_95% CI - Low',
 'Diabetes monitoring_95% CI - High',
 'Diabetes monitoring_Z-Score',
 'Diabetes monitoring_% Receiving HbA1c (Black)',
 'Diabetes monitoring_% Receiving HbA1c (White)',
 'Mammography screening_# Medicare Enrollees',
 'Mammography screening_% Mammography',
 'Mammography screening_95% CI - Low',
 'Mammography screening_95% CI - High',
 'Mammography screening_Z-Score',
 'Mammography screening_% Mammography (Black)',
 'Mammography screening_% Mammography (White)',
 'High school graduation_Cohort Size',
 'High school graduation_Graduation Rate',
 'High school graduation_Z-Score',
 'Some college_95% CI - Low',
 'Some college_95% CI - High',
 'Some college_Z-Score',
 'Unemployment_Z-Score',
 'Children in poverty_% Children in Poverty',
 'Children in poverty_95% CI - Low',
 'Children in poverty_95% CI - High',
 'Children in poverty_Z-Score',
 'Children in poverty_% Children in Poverty (Black)',
 'Children in poverty_% Children in Poverty (Hispanic)',
 'Children in poverty_% Children in Poverty (White)',
 'Income inequality_80th Percentile Income',
 'Income inequality_20th Percentile Income',
 'Income inequality_Income Ratio',
 'Income inequality_Z-Score',
 'Children in single-parent households_# Single-Parent Households',
 'Children in single-parent households_# Households',
 'Children in single-parent households_% Single-Parent Households',
 'Children in single-parent households_95% CI - Low',
 'Children in single-parent households_95% CI - High',
 'Children in single-parent households_Z-Score',
 'Social associations_# Associations',
 'Social associations_Association Rate',
 'Social associations_Z-Score',
 'Violent crime_Z-Score',
 'Injury deaths_# Injury Deaths',
 'Injury deaths_Injury Death Rate',
 'Injury deaths_95% CI - Low',
 'Injury deaths_95% CI - High',
 'Injury deaths_Z-Score',
 'Air pollution - particulate matter_Average Daily PM2.5',
 'Air pollution - particulate matter_Z-Score',
 'Drinking water violations_Presence of violation',
 'Drinking water violations_Z-Score',
 'Severe housing problems_95% CI - Low',
 'Severe housing problems_95% CI - High',
 'Severe housing problems_Z-Score',
 'Driving alone to work_% Drive Alone',
 'Driving alone to work_95% CI - Low',
 'Driving alone to work_95% CI - High',
 'Driving alone to work_Z-Score',
 'Driving alone to work_% Drive Alone (Black)',
 'Driving alone to work_% Drive Alone (Hispanic)',
 'Driving alone to work_% Drive Alone (White)',
 'Long commute - driving alone_# Workers who Drive Alone',
 'Long commute - driving alone_% Long Commute - Drives Alone',
 'Long commute - driving alone_95% CI - Low',
 'Long commute - driving alone_95% CI - High',
 'Long commute - driving alone_Z-Score',
 'Length of Life_Z-Score',
 'Quality of Life_Z-Score',
 'Health Behaviors_Z-Score',
 'Clinical Care_Z-Score',
 'Social & Economic Factors_Z-Score',
 'Physical Environment_Z-Score']


In [32]:
# Dropping columns in-place
chr18.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr18)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  69.0   
2    29003  Missouri          Andrew                   8.0   
3    29005  Missouri        Atchison                   7.0   
4    29007  Missouri         Audrain                  42.0   
..     ...       ...             ...                   ...   
111  29223  Missouri           Wayne                 108.0   
112  29225  Missouri         Webster                  55.0   
113  29227  Missouri           Worth                  67.0   
114  29229  Missouri          Wright                  93.0   
115  29510  Missouri  St. Louis City                 113.0   

     Health Factors_Rank  \
0                    NaN   
1                   27.0   
2                    8.0   
3                   14.0   
4                   43.0   
..                   ...   
111            

In [33]:
chr18.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Frequent physical distress_% Frequent Physical Distress',
 'Frequent mental distress_% Frequent Mental Distress',
 'Diabetes prevalence_% Diabetic',
 'Food insecurity_# Food Insecure',
 'Food insecurity_% Food Insecure',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Uninsured adults_# Uninsured',
 'Uninsured adults_% Uninsured',
 'Other primary care providers_Other PCP Rate',
 'Other primary care providers_Other PCP Ratio',
 'Median household income_Household Income',
 'Demographics_Population',
 'Demographics_% < 18',
 'Demographics_% 65 and over',
 'Demographics_# Rural',
 'Demographics_% Rural',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Food environment index_Food Environment Index',
 'P

In [117]:
# Columns to be dropped
columns_to_drop = columns_to_drop = [
 'Frequent physical distress_% Frequent Physical Distress',
 'Frequent mental distress_% Frequent Mental Distress',
 'Diabetes prevalence_% Diabetic',
 'Food insecurity_# Food Insecure',
 'Food insecurity_% Food Insecure',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Uninsured adults_# Uninsured',
 'Other primary care providers_Other PCP Rate',
 'Other primary care providers_Other PCP Ratio',
 'Median household income_Household Income',
 'Demographics_Population',
 'Demographics_% < 18',
 'Demographics_% 65 and over',
 'Demographics_# Rural',
 'Demographics_% Rural',
 'Food environment index_Food Environment Index',
 'Physical inactivity_% Physically Inactive',
 'Access to exercise opportunities_% With Access',
 'Uninsured_# Uninsured',
 'Uninsured_% Uninsured',
 'Primary care physicians_# Primary Care Physicians',
 'Primary care physicians_PCP Rate',
 'Primary care physicians_PCP Ratio',
 'Mental health providers_# Mental Health Providers',
 'Mental health providers_MHP Rate',
 'Mental health providers_MHP Ratio',
 'Some college_# Some College',
 'Some college_Population',
 'Some college_% Some College',
 'Unemployment_# Unemployed',
 'Unemployment_Labor Force',
 'Violent crime_# Violent Crimes',
 'Severe housing problems_# Households with Severe Problems',
 'Severe housing problems_% Severe Housing Problems',
 'Length of Life_Rank',
 'Quality of Life_Rank']


In [118]:
# Dropping columns in-place
chr18.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr18)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  69.0   
2    29003  Missouri          Andrew                   8.0   
3    29005  Missouri        Atchison                   7.0   
4    29007  Missouri         Audrain                  42.0   
5    29009  Missouri           Barry                  87.0   
6    29011  Missouri          Barton                  78.0   
7    29013  Missouri           Bates                  58.0   
8    29015  Missouri          Benton                  90.0   
9    29017  Missouri       Bollinger                  68.0   
10   29019  Missouri           Boone                   9.0   
11   29021  Missouri        Buchanan                  81.0   
12   29023  Missouri          Butler                 111.0   
13   29025  Missouri        Caldwell                  53.0   
14   29027  Missouri      

In [201]:
chr18.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Uninsured adults_% Uninsured',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Unemployment_% Unemployed',
 'Violent Crime Rate',
 'Health Behaviors_Rank',
 'Clinical Care_Rank',
 'Social & Economic Factors_Rank',
 'Physical Environment_Rank',
 'year']

In [200]:
chr18.rename(columns={'Violent crime_Violent Crime Rate': 'Violent Crime Rate'}, inplace=True)

In [34]:
data_17_1 = pd.read_excel('2017 County Health Rankings Missouri Data - v2.xls',sheet_name = 'Outcomes & Factors Rankings', header = [0,1])
data_17_2 = pd.read_excel('2017 County Health Rankings Missouri Data - v2.xls',sheet_name = 'Additional Measure Data', header = [0,1])
data_17_3 = pd.read_excel('2017 County Health Rankings Missouri Data - v2.xls',sheet_name = 'Ranked Measure Data', header = [0,1])
data_17_4 = pd.read_excel('2017 County Health Rankings Missouri Data - v2.xls',sheet_name = 'Outcomes & Factors SubRankings', header = [0,1])

In [35]:
data_17_3.columns.to_list()

[('*Due to data limitations, data for Jasper and Newton counties were averaged for primary care physicians',
  'FIPS'),
 ('*Due to data limitations, data for Jasper and Newton counties were averaged for primary care physicians',
  'State'),
 ('*Due to data limitations, data for Jasper and Newton counties were averaged for primary care physicians',
  'County'),
 ('Premature death', '# Deaths'),
 ('Premature death', 'Years of Potential Life Lost Rate'),
 ('Premature death', '95% CI - Low'),
 ('Premature death', '95% CI - High'),
 ('Premature death', 'Z-Score'),
 ('Poor or fair health', '% Fair/Poor'),
 ('Poor or fair health', '95% CI - Low'),
 ('Poor or fair health', '95% CI - High'),
 ('Poor or fair health', 'Z-Score'),
 ('Poor physical health days', 'Physically Unhealthy Days'),
 ('Poor physical health days', '95% CI - Low'),
 ('Poor physical health days', '95% CI - High'),
 ('Poor physical health days', 'Z-Score'),
 ('Poor mental health days', 'Mentally Unhealthy Days'),
 ('Poor menta

In [36]:
data_17_1 = add_index_prefix(data_17_1)
data_17_2 = add_index_prefix(data_17_2)
data_17_3 = add_index_prefix(data_17_3)
data_17_4 = add_index_prefix(data_17_4)

display(data_17_1.columns,data_17_2.columns, data_17_3.columns , data_17_4.columns)

Index(['FIPS', 'State', 'County', 'Health Outcomes_Z-Score',
       'Health Outcomes_Rank', 'Health Factors_Z-Score',
       'Health Factors_Rank'],
      dtype='object')

Index(['FIPS', 'State', 'County', 'Premature age-adjusted mortality_# Deaths',
       'Premature age-adjusted mortality_Age-Adjusted Mortality',
       'Premature age-adjusted mortality_95% CI - Low',
       'Premature age-adjusted mortality_95% CI - High',
       'Child mortality_# Deaths', 'Child mortality_Child Mortality Rate',
       'Child mortality_95% CI - Low', 'Child mortality_95% CI - High',
       'Infant mortality_# Deaths', 'Infant mortality_Infant Mortality Rate',
       'Infant mortality_95% CI - Low', 'Infant mortality_95% CI - High',
       'Frequent physical distress_% Frequent Physical Distress',
       'Frequent physical distress_95% CI - Low',
       'Frequent physical distress_95% CI - High',
       'Frequent mental distress_% Frequent Mental Distress',
       'Frequent mental distress_95% CI - Low',
       'Frequent mental distress_95% CI - High',
       'Diabetes prevalence_% Diabetic', 'Diabetes prevalence_95% CI - Low',
       'Diabetes prevalence_95% CI - Hig

Index(['FIPS', 'State', 'County', 'Premature death_# Deaths',
       'Premature death_Years of Potential Life Lost Rate',
       'Premature death_95% CI - Low', 'Premature death_95% CI - High',
       'Premature death_Z-Score', 'Poor or fair health_% Fair/Poor',
       'Poor or fair health_95% CI - Low',
       ...
       'Driving alone to work_95% CI - High', 'Driving alone to work_Z-Score',
       'Driving alone to work_% Drive Alone - Black',
       'Driving alone to work_% Drive Alone - Hispanic',
       'Driving alone to work_% Drive Alone - White',
       'Long commute - driving alone_# Workers who Drive Alone',
       'Long commute - driving alone_% Long Commute - Drives Alone',
       'Long commute - driving alone_95% CI - Low',
       'Long commute - driving alone_95% CI - High',
       'Long commute - driving alone_Z-Score'],
      dtype='object', length=162)

Index(['FIPS', 'State', 'County', 'Length of Life_Z-Score',
       'Length of Life_Rank', 'Quality of Life_Z-Score',
       'Quality of Life_Rank', 'Health Behaviors_Z-Score',
       'Health Behaviors_Rank', 'Clinical Care_Z-Score', 'Clinical Care_Rank',
       'Social & Economic Factors_Z-Score', 'Social & Economic Factors_Rank',
       'Physical Environment_Z-Score', 'Physical Environment_Rank'],
      dtype='object')

In [37]:
display(data_17_1.shape,data_17_2.shape, data_17_3.shape , data_17_4.shape)

(116, 7)

(116, 89)

(116, 162)

(116, 15)

In [38]:
pd.set_option('display.max_rows', None)
display(data_17_3[['FIPS', 'State' , 'County']])

,FIPS,State,County
0,29000,Missouri,NaN
1,29001,Missouri,Adair
2,29003,Missouri,Andrew
3,29005,Missouri,Atchison
4,29007,Missouri,Audrain
5,29009,Missouri,Barry
6,29011,Missouri,Barton
7,29013,Missouri,Bates
8,29015,Missouri,Benton
9,29017,Missouri,Bollinger


In [39]:
data_17_3['County'] = data_17_3['County'].str.replace("""*""" , '')
data_17_4['County'] = data_17_4['County'].str.replace("""*""" , '')


In [40]:
chr17 = pd.merge(data_17_1 , data_17_2 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
display(chr17.shape)
chr17 = pd.merge(chr17 , data_17_3 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
display(chr17.shape)
chr17 = pd.merge(chr17 , data_17_4 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
display(chr17.shape)
chr17

(116, 93)

(116, 252)

(116, 264)

,FIPS,State,County,Health Outcomes_Z-Score,Health Outcomes_Rank,Health Factors_Z-Score,Health Factors_Rank,Premature age-adjusted mortality_# Deaths,Premature age-adjusted mortality_Age-Adjusted Mortality,Premature age-adjusted mortality_95% CI - Low,...,Quality of Life_Z-Score,Quality of Life_Rank,Health Behaviors_Z-Score,Health Behaviors_Rank,Clinical Care_Z-Score,Clinical Care_Rank,Social & Economic Factors_Z-Score,Social & Economic Factors_Rank,Physical Environment_Z-Score,Physical Environment_Rank
0,29000,Missouri,NaN,NaN,NaN,NaN,NaN,79611,386.4,383.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29001,Missouri,Adair,0.128347,75.0,-0.325805,26.0,287,394.0,346.7,...,0.373928,100.0,-0.130016,21.0,-0.222724,7.0,-0.003876,61.0,0.030810,86.0
2,29003,Missouri,Andrew,-1.119926,7.0,-0.732145,7.0,217,332.4,286.7,...,-0.691611,2.0,-0.238928,6.0,-0.112561,18.0,-0.349129,8.0,-0.031527,33.0
3,29005,Missouri,Atchison,-1.261720,4.0,-0.554614,12.0,88,366.8,289.0,...,-0.541104,10.0,-0.227006,7.0,0.095986,95.0,-0.347574,9.0,-0.076021,7.0
4,29007,Missouri,Audrain,-0.163215,55.0,-0.062855,51.0,342,375.8,334.9,...,0.035889,65.0,0.080148,82.0,-0.076157,29.0,-0.008977,60.0,-0.057868,20.0
5,29009,Missouri,Barry,0.385615,86.0,0.069393,66.0,589,436.5,398.8,...,0.059636,68.0,0.024586,72.0,0.009985,59.0,0.038269,69.0,-0.003447,54.0
6,29011,Missouri,Barton,0.303517,84.0,0.247636,86.0,178,422.1,356.6,...,0.149374,82.0,0.135440,95.0,0.154718,103.0,0.007849,65.0,-0.050370,23.0
7,29013,Missouri,Bates,0.102706,74.0,0.225506,81.0,263,433.3,378.5,...,-0.004855,60.0,-0.028388,51.0,0.163264,106.0,0.089981,78.0,0.000649,57.0
8,29015,Missouri,Benton,0.446339,88.0,0.264118,88.0,408,466.6,411.6,...,0.018465,63.0,-0.125936,25.0,0.066074,82.0,0.312863,106.0,0.011117,65.0
9,29017,Missouri,Bollinger,0.036061,69.0,0.270888,90.0,195,415.2,354.0,...,-0.098164,51.0,0.111729,91.0,0.042572,72.0,0.146319,85.0,-0.029732,34.0


In [41]:
chr17['year'] = 2017
chr17

,FIPS,State,County,Health Outcomes_Z-Score,Health Outcomes_Rank,Health Factors_Z-Score,Health Factors_Rank,Premature age-adjusted mortality_# Deaths,Premature age-adjusted mortality_Age-Adjusted Mortality,Premature age-adjusted mortality_95% CI - Low,...,Quality of Life_Rank,Health Behaviors_Z-Score,Health Behaviors_Rank,Clinical Care_Z-Score,Clinical Care_Rank,Social & Economic Factors_Z-Score,Social & Economic Factors_Rank,Physical Environment_Z-Score,Physical Environment_Rank,year
0,29000,Missouri,NaN,NaN,NaN,NaN,NaN,79611,386.4,383.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
1,29001,Missouri,Adair,0.128347,75.0,-0.325805,26.0,287,394.0,346.7,...,100.0,-0.130016,21.0,-0.222724,7.0,-0.003876,61.0,0.030810,86.0,2017
2,29003,Missouri,Andrew,-1.119926,7.0,-0.732145,7.0,217,332.4,286.7,...,2.0,-0.238928,6.0,-0.112561,18.0,-0.349129,8.0,-0.031527,33.0,2017
3,29005,Missouri,Atchison,-1.261720,4.0,-0.554614,12.0,88,366.8,289.0,...,10.0,-0.227006,7.0,0.095986,95.0,-0.347574,9.0,-0.076021,7.0,2017
4,29007,Missouri,Audrain,-0.163215,55.0,-0.062855,51.0,342,375.8,334.9,...,65.0,0.080148,82.0,-0.076157,29.0,-0.008977,60.0,-0.057868,20.0,2017
5,29009,Missouri,Barry,0.385615,86.0,0.069393,66.0,589,436.5,398.8,...,68.0,0.024586,72.0,0.009985,59.0,0.038269,69.0,-0.003447,54.0,2017
6,29011,Missouri,Barton,0.303517,84.0,0.247636,86.0,178,422.1,356.6,...,82.0,0.135440,95.0,0.154718,103.0,0.007849,65.0,-0.050370,23.0,2017
7,29013,Missouri,Bates,0.102706,74.0,0.225506,81.0,263,433.3,378.5,...,60.0,-0.028388,51.0,0.163264,106.0,0.089981,78.0,0.000649,57.0,2017
8,29015,Missouri,Benton,0.446339,88.0,0.264118,88.0,408,466.6,411.6,...,63.0,-0.125936,25.0,0.066074,82.0,0.312863,106.0,0.011117,65.0,2017
9,29017,Missouri,Bollinger,0.036061,69.0,0.270888,90.0,195,415.2,354.0,...,51.0,0.111729,91.0,0.042572,72.0,0.146319,85.0,-0.029732,34.0,2017


In [42]:
chr17.shape

(116, 265)

In [44]:
# Columns to be dropped
columns_to_drop = [
 'Health Outcomes_Z-Score',
 'Health Factors_Z-Score',
 'Premature age-adjusted mortality_# Deaths',
 'Premature age-adjusted mortality_Age-Adjusted Mortality',
 'Premature age-adjusted mortality_95% CI - Low',
 'Premature age-adjusted mortality_95% CI - High',
 'Child mortality_# Deaths',
 'Child mortality_Child Mortality Rate',
 'Child mortality_95% CI - Low',
 'Child mortality_95% CI - High',
 'Infant mortality_# Deaths',
 'Infant mortality_Infant Mortality Rate',
 'Infant mortality_95% CI - Low',
 'Infant mortality_95% CI - High',
 'Frequent physical distress_95% CI - Low',
 'Frequent physical distress_95% CI - High',
 'Frequent mental distress_95% CI - Low',
 'Frequent mental distress_95% CI - High',
 'Diabetes prevalence_95% CI - Low',
 'Diabetes prevalence_95% CI - High',
 'HIV prevalence_# HIV Cases',
 'HIV prevalence_HIV Prevalence Rate',
 'Drug overdose deaths_# Drug Overdose Deaths',
 'Drug overdose deaths_Drug Overdose Mortality Rate',
 'Motor vehicle crash deaths_# Motor Vehicle Deaths',
 'Motor vehicle crash deaths_MV Mortality Rate',
 'Motor vehicle crash deaths_95% CI - Low',
 'Motor vehicle crash deaths_95% CI - High',
 'Insufficient sleep_% Insufficient Sleep',
 'Insufficient sleep_95% CI - Low',
 'Insufficient sleep_95% CI - High',
 'Uninsured adults_95% CI - Low',
 'Uninsured adults_95% CI - High',
 'Uninsured children_# Uninsured',
 'Uninsured children_% Uninsured',
 'Uninsured children_95% CI - Low',
 'Uninsured children_95% CI - High',
 'Health care costs_Costs',
 'Disconnected youth_% Disconnected Youth',
 'Median household income_95% CI - Low',
 'Median household income_95% CI - High',
 'Median household income_Household Income (white alone)',
 'Median household income_Household Income (black alone)',
 'Median household income_Household income (Hispanic)',
 'Children eligible for free or reduced price lunch_% Free or Reduced Lunch',
 'Residential segregation - black/white_Segregation index',
 'Residential segregation - non-white/white_Segregation Index',
 'Homicides_Homicide Rate',
 'Homicides_95% CI - Low',
 'Homicides_95% CI - High',
 'Firearm fatalities_# Firearm Fatalities',
 'Firearm fatalities_Firearm Fatalities Rate',
 'Firearm fatalities_95% CI - Low',
 'Firearm fatalities_95% CI - High',
 'Demographics_# African American',
 'Demographics_% African American',
 'Demographics_# American Indian/Alaskan Native',
 'Demographics_% American Indian/Alaskan Native',
 'Demographics_# Asian',
 'Demographics_% Asian',
 'Demographics_# Native Hawaiian/Other Pacific Islander',
 'Demographics_% Native Hawaiian/Other Pacific Islander',
 'Demographics_# Hispanic',
 'Demographics_% Hispanic',
 'Demographics_# Non-Hispanic White',
 'Demographics_% Non-Hispanic White',
 'Demographics_# Not Proficient in English',
 'Demographics_% Not Proficient in English',
 'Demographics_95% CI - Low',
 'Demographics_95% CI - High',
 'Demographics_% Female',
 'Premature death_# Deaths',
 'Premature death_Years of Potential Life Lost Rate',
 'Premature death_95% CI - Low',
 'Premature death_95% CI - High',
 'Premature death_Z-Score',
 'Poor or fair health_95% CI - Low',
 'Poor or fair health_95% CI - High',
 'Poor or fair health_Z-Score',
 'Poor physical health days_95% CI - Low',
 'Poor physical health days_95% CI - High',
 'Poor physical health days_Z-Score',
 'Poor mental health days_95% CI - Low',
 'Poor mental health days_95% CI - High',
 'Poor mental health days_Z-Score',
 'Low birthweight_Unreliable',
 'Low birthweight_# Low Birthweight Births',
 'Low birthweight_# Live Births',
 'Low birthweight_% LBW',
 'Low birthweight_95% CI - Low',
 'Low birthweight_95% CI - High',
 'Low birthweight_Z-Score',
 'Adult smoking_95% CI - Low',
 'Adult smoking_95% CI - High',
 'Adult smoking_Z-Score',
 'Adult obesity_95% CI - Low',
 'Adult obesity_95% CI - High',
 'Adult obesity_Z-Score',
 'Food environment index_Z-Score',
 'Physical inactivity_95% CI - Low',
 'Physical inactivity_95% CI - High',
 'Physical inactivity_Z-Score',
 'Access to exercise opportunities_Z-Score',
 'Excessive drinking_% Excessive Drinking',
 'Excessive drinking_95% CI - Low',
 'Excessive drinking_95% CI - High',
 'Excessive drinking_Z-Score',
 'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths',
 'Alcohol-impaired driving deaths_# Driving Deaths',
 'Alcohol-impaired driving deaths_% Alcohol-Impaired',
 'Alcohol-impaired driving deaths_95% CI - Low',
 'Alcohol-impaired driving deaths_95% CI - High',
 'Alcohol-impaired driving deaths_Z-Score',
 'Sexually transmitted infections_# Chlamydia Cases',
 'Sexually transmitted infections_Chlamydia Rate',
 'Sexually transmitted infections_Z-Score',
 'Teen births_Teen Births',
 'Teen births_Teen Population',
 'Teen births_Teen Birth Rate',
 'Teen births_95% CI - Low',
 'Teen births_95% CI - High',
 'Teen births_Z-Score',
 'Uninsured_95% CI - Low',
 'Uninsured_95% CI - High',
 'Uninsured_Z-Score',
 'Primary care physicians_Z-Score',
 'Dentists_# Dentists',
 'Dentists_Dentist Rate',
 'Dentists_Dentist Ratio',
 'Dentists_Z-Score',
 'Mental health providers_Z-Score',
 'Preventable hospital stays_# Medicare Enrollees',
 'Preventable hospital stays_Preventable Hosp. Rate',
 'Preventable hospital stays_95% CI - Low',
 'Preventable hospital stays_95% CI - High',
 'Preventable hospital stays_Z-Score',
 'Preventable hospital stays_Preventable Hosp. Rate (white)',
 'Preventable hospital stays_Preventable Hosp. Rate (black)',
 'Diabetes monitoring_# Diabetics',
 'Diabetes monitoring_% Receiving HbA1c',
 'Diabetes monitoring_95% CI - Low',
 'Diabetes monitoring_95% CI - High',
 'Diabetes monitoring_Z-Score',
 'Diabetes monitoring_% Receiving HbA1c (white)',
 'Diabetes monitoring_% Receiving HbA1c (black)',
 'Mammography screening_# Medicare Enrollees',
 'Mammography screening_% Mammography',
 'Mammography screening_95% CI - Low',
 'Mammography screening_95% CI - High',
 'Mammography screening_Z-Score',
 'Mammography screening_% Mammography (white)',
 'Mammography screening_% Mammography (black)',
 'High school graduation_Cohort Size',
 'High school graduation_Graduation Rate',
 'High school graduation_Z-Score',
 'Some college_95% CI - Low',
 'Some college_95% CI - High',
 'Some college_Z-Score',
 'Unemployment_Z-Score',
 'Children in poverty_% Children in Poverty',
 'Children in poverty_95% CI - Low',
 'Children in poverty_95% CI - High',
 'Children in poverty_Z-Score',
 'Children in poverty_% Children in Poverty - Black',
 'Children in poverty_% Children in Poverty - Hispanic',
 'Children in poverty_% Children in Poverty - White',
 'Income inequality_80th Percentile Income',
 'Income inequality_20th Percentile Income',
 'Income inequality_Income Ratio',
 'Income inequality_Z-Score',
 'Children in single-parent households_# Single-Parent Households',
 'Children in single-parent households_# Households',
 'Children in single-parent households_% Single-Parent Households',
 'Children in single-parent households_95% CI - Low',
 'Children in single-parent households_95% CI - High',
 'Children in single-parent households_Z-Score',
 'Social associations_# Associations',
 'Social associations_Association Rate',
 'Social associations_Z-Score',
 'Violent crime_Z-Score',
 'Injury deaths_# Injury Deaths',
 'Injury deaths_Injury Death Rate',
 'Injury deaths_95% CI - Low',
 'Injury deaths_95% CI - High',
 'Injury deaths_Z-Score',
 'Air pollution - particulate matter_Average Daily PM2.5',
 'Air pollution - particulate matter_Z-Score',
 'Drinking water violations_Presence of violation',
 'Drinking water violations_Z-Score',
 'Severe housing problems_95% CI - Low',
 'Severe housing problems_95% CI - High',
 'Severe housing problems_Z-Score',
 'Driving alone to work_% Drive Alone',
 'Driving alone to work_95% CI - Low',
 'Driving alone to work_95% CI - High',
 'Driving alone to work_Z-Score',
 'Driving alone to work_% Drive Alone - Black',
 'Driving alone to work_% Drive Alone - Hispanic',
 'Driving alone to work_% Drive Alone - White',
 'Long commute - driving alone_# Workers who Drive Alone',
 'Long commute - driving alone_% Long Commute - Drives Alone',
 'Long commute - driving alone_95% CI - Low',
 'Long commute - driving alone_95% CI - High',
 'Long commute - driving alone_Z-Score',
 'Length of Life_Z-Score',
 'Quality of Life_Z-Score',
 'Health Behaviors_Z-Score',
 'Clinical Care_Z-Score',
 'Social & Economic Factors_Z-Score',
 'Physical Environment_Z-Score']

In [45]:
# Dropping columns in-place
chr17.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr17)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  75.0   
2    29003  Missouri          Andrew                   7.0   
3    29005  Missouri        Atchison                   4.0   
4    29007  Missouri         Audrain                  55.0   
5    29009  Missouri           Barry                  86.0   
6    29011  Missouri          Barton                  84.0   
7    29013  Missouri           Bates                  74.0   
8    29015  Missouri          Benton                  88.0   
9    29017  Missouri       Bollinger                  69.0   
10   29019  Missouri           Boone                  19.0   
11   29021  Missouri        Buchanan                  76.0   
12   29023  Missouri          Butler                 110.0   
13   29025  Missouri        Caldwell                  87.0   
14   29027  Missouri      

In [46]:
chr17.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Frequent physical distress_% Frequent Physical Distress',
 'Frequent mental distress_% Frequent Mental Distress',
 'Diabetes prevalence_% Diabetic',
 'Food insecurity_# Food Insecure',
 'Food insecurity_% Food Insecure',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Uninsured adults_# Uninsured',
 'Uninsured adults_% Uninsured',
 'Other primary care providers_Other PCP Rate',
 'Other primary care providers_Other PCP Ratio',
 'Median household income_Household Income',
 'Demographics_Population',
 'Demographics_% < 18',
 'Demographics_% 65 and over',
 'Demographics_# Rural',
 'Demographics_% Rural',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Food environment index_Food Environment Index',
 'P

In [121]:
# Columns to be dropped
columns_to_drop = columns_to_drop = [
 'Frequent physical distress_% Frequent Physical Distress',
 'Frequent mental distress_% Frequent Mental Distress',
 'Diabetes prevalence_% Diabetic',
 'Food insecurity_# Food Insecure',
 'Food insecurity_% Food Insecure',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Uninsured adults_# Uninsured',
 'Other primary care providers_Other PCP Rate',
 'Other primary care providers_Other PCP Ratio',
 'Median household income_Household Income',
 'Demographics_Population',
 'Demographics_% < 18',
 'Demographics_% 65 and over',
 'Demographics_# Rural',
 'Demographics_% Rural',
 'Food environment index_Food Environment Index',
 'Physical inactivity_% Physically Inactive',
 'Access to exercise opportunities_% With Access',
 'Uninsured_# Uninsured',
 'Uninsured_% Uninsured',
 'Primary care physicians_# Primary Care Physicians',
 'Primary care physicians_PCP Rate',
 'Primary care physicians_PCP Ratio',
 'Mental health providers_# Mental Health Providers',
 'Mental health providers_MHP Rate',
 'Mental health providers_MHP Ratio',
 'Some college_# Some College',
 'Some college_Population',
 'Some college_% Some College',
 'Unemployment_# Unemployed',
 'Unemployment_Labor Force',
 'Violent crime_# Violent Crimes',
 'Severe housing problems_# Households with Severe Problems',
 'Severe housing problems_% Severe Housing Problems',
 'Length of Life_Rank',
 'Quality of Life_Rank']

In [122]:
# Dropping columns in-place
chr17.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr17)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  75.0   
2    29003  Missouri          Andrew                   7.0   
3    29005  Missouri        Atchison                   4.0   
4    29007  Missouri         Audrain                  55.0   
5    29009  Missouri           Barry                  86.0   
6    29011  Missouri          Barton                  84.0   
7    29013  Missouri           Bates                  74.0   
8    29015  Missouri          Benton                  88.0   
9    29017  Missouri       Bollinger                  69.0   
10   29019  Missouri           Boone                  19.0   
11   29021  Missouri        Buchanan                  76.0   
12   29023  Missouri          Butler                 110.0   
13   29025  Missouri        Caldwell                  87.0   
14   29027  Missouri      

In [199]:
chr17.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Uninsured adults_% Uninsured',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Unemployment_% Unemployed',
 'Violent Crime Rate',
 'Health Behaviors_Rank',
 'Clinical Care_Rank',
 'Social & Economic Factors_Rank',
 'Physical Environment_Rank',
 'year']

In [198]:
chr17.rename(columns={'Violent crime_Violent Crime Rate': 'Violent Crime Rate'}, inplace=True)

In [47]:
data_16_1 = pd.read_excel('2016 County Health Rankings Missouri Data - v3.xls',sheet_name = 'Outcomes & Factors Rankings', header = [0,1])
data_16_2 = pd.read_excel('2016 County Health Rankings Missouri Data - v3.xls',sheet_name = 'Additional Measure Data', header = [0,1])
data_16_3 = pd.read_excel('2016 County Health Rankings Missouri Data - v3.xls',sheet_name = 'Ranked Measure Data', header = [0,1])
data_16_4 = pd.read_excel('2016 County Health Rankings Missouri Data - v3.xls',sheet_name = 'Outcomes & Factors SubRankings', header = [0,1])

In [48]:
data_16_1 = add_index_prefix(data_16_1)
data_16_2 = add_index_prefix(data_16_2)
data_16_3 = add_index_prefix(data_16_3)
data_16_4 = add_index_prefix(data_16_4)

display(data_16_1.columns,data_16_2.columns, data_16_3.columns , data_16_4.columns)

Index(['FIPS', 'State', 'County', 'Health Outcomes_Z-Score',
       'Health Outcomes_Rank', 'Health Factors_Z-Score',
       'Health Factors_Rank'],
      dtype='object')

Index(['FIPS', 'State', 'County', 'Premature age-adjusted mortality_# Deaths',
       'Premature age-adjusted mortality_Age-Adjusted Mortality',
       'Premature age-adjusted mortality_95% CI - Low',
       'Premature age-adjusted mortality_95% CI - High',
       'Child mortality_# Deaths', 'Child mortality_Child Mortality Rate',
       'Child mortality_95% CI - Low', 'Child mortality_95% CI - High',
       'Infant mortality_# Deaths', 'Infant mortality_Infant Mortality Rate',
       'Infant mortality_95% CI - Low', 'Infant mortality_95% CI - High',
       'Frequent physical distress_% Frequent Physical Distress',
       'Frequent physical distress_95% CI - Low',
       'Frequent physical distress_95% CI - High',
       'Frequent mental distress_% Frequent Mental Distress',
       'Frequent mental distress_95% CI - Low',
       'Frequent mental distress_95% CI - High',
       'Diabetes prevalence_% Diabetic', 'Diabetes prevalence_95% CI - Low',
       'Diabetes prevalence_95% CI - Hig

Index(['FIPS', 'State', 'County', 'Premature death_# Deaths',
       'Premature death_Years of Potential Life Lost Rate',
       'Premature death_95% CI - Low', 'Premature death_95% CI - High',
       'Premature death_Z-Score', 'Poor or fair health_% Fair/Poor',
       'Poor or fair health_95% CI - Low',
       ...
       'Driving alone to work_# Workers',
       'Driving alone to work_% Drive Alone',
       'Driving alone to work_95% CI - Low',
       'Driving alone to work_95% CI - High', 'Driving alone to work_Z-Score',
       'Long commute - driving alone_# Workers who Drive Alone',
       'Long commute - driving alone_% Long Commute - Drives Alone',
       'Long commute - driving alone_95% CI - Low',
       'Long commute - driving alone_95% CI - High',
       'Long commute - driving alone_Z-Score'],
      dtype='object', length=156)

Index(['FIPS', 'State', 'County', 'Length of Life_Z-Score',
       'Length of Life_Rank', 'Quality of Life_Z-Score',
       'Quality of Life_Rank', 'Health Behaviors_Z-Score',
       'Health Behaviors_Rank', 'Clinical Care_Z-Score', 'Clinical Care_Rank',
       'Social & Economic Factors_Z-Score', 'Social & Economic Factors_Rank',
       'Physical Environment_Z-Score', 'Physical Environment_Rank'],
      dtype='object')

In [54]:
chr16 = pd.merge(data_16_1 , data_16_2 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr16 = pd.merge(chr16 , data_16_3 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr16 = pd.merge(chr16 , data_16_4 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr16

,FIPS,State,County,Health Outcomes_Z-Score,Health Outcomes_Rank,Health Factors_Z-Score,Health Factors_Rank,Premature age-adjusted mortality_# Deaths,Premature age-adjusted mortality_Age-Adjusted Mortality,Premature age-adjusted mortality_95% CI - Low,...,Quality of Life_Z-Score,Quality of Life_Rank,Health Behaviors_Z-Score,Health Behaviors_Rank,Clinical Care_Z-Score,Clinical Care_Rank,Social & Economic Factors_Z-Score,Social & Economic Factors_Rank,Physical Environment_Z-Score,Physical Environment_Rank
0,29000,Missouri,NaN,NaN,NaN,NaN,NaN,75686.0,381.1,378.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29001,Missouri,Adair,-0.134035,52.0,-0.188987,42.0,259.0,357.8,312.9,...,0.389978,100.0,0.040365,68.0,-0.202690,8.0,-0.059706,52.0,0.033045,84.0
2,29003,Missouri,Andrew,-0.992217,11.0,-0.830355,6.0,197.0,323.2,276.3,...,-0.689654,4.0,-0.278990,3.0,-0.107269,21.0,-0.372536,6.0,-0.071561,13.0
3,29005,Missouri,Atchison,-0.942970,17.0,-0.553804,13.0,81.0,367.5,287.6,...,-0.539196,10.0,-0.142454,21.0,0.056889,76.0,-0.344711,7.0,-0.123528,1.0
4,29007,Missouri,Audrain,-0.336119,39.0,-0.077271,53.0,314.0,365.8,324.6,...,-0.027267,57.0,0.057696,73.0,-0.083458,26.0,0.004397,66.0,-0.055906,22.0
5,29009,Missouri,Barry,0.471284,90.0,0.081157,64.0,567.0,439.6,401.2,...,0.008338,59.0,-0.030148,53.0,0.028823,68.0,0.067841,70.0,0.014641,64.0
6,29011,Missouri,Barton,0.063128,69.0,0.107339,70.0,154.0,376.2,314.2,...,0.106889,73.0,0.036515,66.0,0.165269,107.0,-0.031156,58.0,-0.063289,17.0
7,29013,Missouri,Bates,0.274824,83.0,0.280670,85.0,278.0,463.2,406.4,...,0.178586,87.0,0.062851,74.0,0.093135,92.0,0.134188,86.0,-0.009504,45.0
8,29015,Missouri,Benton,0.374548,87.0,0.167651,77.0,380.0,435.7,384.0,...,0.142539,82.0,-0.128040,25.0,0.110302,99.0,0.171211,89.0,0.014178,63.0
9,29017,Missouri,Bollinger,-0.177079,50.0,0.371941,93.0,188.0,407.2,346.2,...,-0.161698,41.0,0.142988,97.0,0.087775,89.0,0.109217,79.0,0.031960,82.0


In [55]:
chr16['year'] = 2016

In [56]:
chr16.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Z-Score',
 'Health Outcomes_Rank',
 'Health Factors_Z-Score',
 'Health Factors_Rank',
 'Premature age-adjusted mortality_# Deaths',
 'Premature age-adjusted mortality_Age-Adjusted Mortality',
 'Premature age-adjusted mortality_95% CI - Low',
 'Premature age-adjusted mortality_95% CI - High',
 'Child mortality_# Deaths',
 'Child mortality_Child Mortality Rate',
 'Child mortality_95% CI - Low',
 'Child mortality_95% CI - High',
 'Infant mortality_# Deaths',
 'Infant mortality_Infant Mortality Rate',
 'Infant mortality_95% CI - Low',
 'Infant mortality_95% CI - High',
 'Frequent physical distress_% Frequent Physical Distress',
 'Frequent physical distress_95% CI - Low',
 'Frequent physical distress_95% CI - High',
 'Frequent mental distress_% Frequent Mental Distress',
 'Frequent mental distress_95% CI - Low',
 'Frequent mental distress_95% CI - High',
 'Diabetes prevalence_% Diabetic',
 'Diabetes prevalence_95% CI - Low',
 'Diabetes prevale

In [58]:
# Columns to be dropped
columns_to_drop = [
 'Health Outcomes_Z-Score',
 'Health Factors_Z-Score',
 'Premature age-adjusted mortality_# Deaths',
 'Premature age-adjusted mortality_Age-Adjusted Mortality',
 'Premature age-adjusted mortality_95% CI - Low',
 'Premature age-adjusted mortality_95% CI - High',
 'Child mortality_# Deaths',
 'Child mortality_Child Mortality Rate',
 'Child mortality_95% CI - Low',
 'Child mortality_95% CI - High',
 'Infant mortality_# Deaths',
 'Infant mortality_Infant Mortality Rate',
 'Infant mortality_95% CI - Low',
 'Infant mortality_95% CI - High',
 'Frequent physical distress_95% CI - Low',
 'Frequent physical distress_95% CI - High',
 'Frequent mental distress_95% CI - Low',
 'Frequent mental distress_95% CI - High',
 'Diabetes prevalence_95% CI - Low',
 'Diabetes prevalence_95% CI - High',
 'HIV prevalence_# HIV Cases',
 'HIV prevalence_HIV Prevalence Rate',
 'Drug overdose deaths_# Drug Overdose Deaths',
 'Drug overdose deaths_Drug Overdose Mortality Rate',
 'Drug overdose deaths - modeled_Range Drug Overdose Mortality Rate',
 'Motor vehicle crash deaths_# Motor Vehicle Deaths',
 'Motor vehicle crash deaths_MV Mortality Rate',
 'Motor vehicle crash deaths_95% CI - Low',
 'Motor vehicle crash deaths_95% CI - High',
 'Insufficient sleep_% Insufficient Sleep',
 'Insufficient sleep_95% CI - Low',
 'Insufficient sleep_95% CI - High',
 'Uninsured adults_95% CI - Low',
 'Uninsured adults_95% CI - High',
 'Uninsured children_# Uninsured',
 'Uninsured children_% Uninsured',
 'Uninsured children_95% CI - Low',
 'Uninsured children_95% CI - High',
 'Health care costs_Costs',
 'Previous Other primary care providers data^_Other PCP Rate',
 'Previous Other primary care providers data^_Other PCP Ratio',
 'Median household income_95% CI - Low',
 'Median household income_95% CI - High',
 'Children eligible for free lunch_% Free Lunch',
 'Residential segregation - black/white_Segregation index',
 'Residential segregation - \nnon-white/white_Segregation Index',
 'Homicides_Homicide Rate',
 'Homicides_95% CI - Low',
 'Homicides_95% CI - High',
 'Demographics_# African American',
 'Demographics_% African American',
 'Demographics_# American Indian/Alaskan Native',
 'Demographics_% American Indian/Alaskan Native',
 'Demographics_# Asian',
 'Demographics_% Asian',
 'Demographics_# Native Hawaiian/Other Pacific Islander',
 'Demographics_% Native Hawaiian/Other Pacific Islander',
 'Demographics_# Hispanic',
 'Demographics_% Hispanic',
 'Demographics_# Non-Hispanic White',
 'Demographics_% Non-Hispanic White',
 'Demographics_# Not Proficient in English',
 'Demographics_% Not Proficient in English',
 'Demographics_95% CI - Low',
 'Demographics_95% CI - High',
 'Demographics_% Female',
 'Premature death_# Deaths',
 'Premature death_Years of Potential Life Lost Rate',
 'Premature death_95% CI - Low',
 'Premature death_95% CI - High',
 'Premature death_Z-Score',
 'Poor or fair health_95% CI - Low',
 'Poor or fair health_95% CI - High',
 'Poor or fair health_Z-Score',
 'Poor physical health days_95% CI - Low',
 'Poor physical health days_95% CI - High',
 'Poor physical health days_Z-Score',
 'Poor mental health days_95% CI - Low',
 'Poor mental health days_95% CI - High',
 'Poor mental health days_Z-Score',
 'Low birthweight_Unreliable',
 'Low birthweight_# Low Birthweight Births',
 'Low birthweight_# Live Births',
 'Low birthweight_% LBW',
 'Low birthweight_95% CI - Low',
 'Low birthweight_95% CI - High',
 'Low birthweight_Z-Score',
 'Adult smoking_95% CI - Low',
 'Adult smoking_95% CI - High',
 'Adult smoking_Z-Score',
 'Adult obesity_95% CI - Low',
 'Adult obesity_95% CI - High',
 'Adult obesity_Z-Score',
 'Food environment index_Z-Score',
 'Physical inactivity_95% CI - Low',
 'Physical inactivity_95% CI - High',
 'Physical inactivity_Z-Score',
 'Access to exercise opportunities_Z-Score',
 'Excessive drinking_% Excessive Drinking',
 'Excessive drinking_95% CI - Low',
 'Excessive drinking_95% CI - High',
 'Excessive drinking_Z-Score',
 'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths',
 'Alcohol-impaired driving deaths_# Driving Deaths',
 'Alcohol-impaired driving deaths_% Alcohol-Impaired',
 'Alcohol-impaired driving deaths_95% CI - Low',
 'Alcohol-impaired driving deaths_95% CI - High',
 'Alcohol-impaired driving deaths_Z-Score',
 'Sexually transmitted infections_# Chlamydia Cases',
 'Sexually transmitted infections_Chlamydia Rate',
 'Sexually transmitted infections_Z-Score',
 'Teen births_Teen Births',
 'Teen births_Teen Population',
 'Teen births_Teen Birth Rate',
 'Teen births_95% CI - Low',
 'Teen births_95% CI - High',
 'Teen births_Z-Score',
 'Uninsured_95% CI - Low',
 'Uninsured_95% CI - High',
 'Uninsured_Z-Score',
 'Primary care physicians_# Primary Care Physicians',
 'Primary care physicians_PCP Rate',
 'Primary care physicians_PCP Ratio',
 'Primary care physicians_Z-Score',
 'Dentists_# Dentists',
 'Dentists_Dentist Rate',
 'Dentists_Dentist Ratio',
 'Dentists_Z-Score',
 'Previous Mental health providers data -- used to calculate rankings^_# Mental Health Providers',
 'Previous Mental health providers data -- used to calculate rankings^_MHP Rate',
 'Previous Mental health providers data -- used to calculate rankings^_MHP Ratio',
 'Previous Mental health providers data -- used to calculate rankings^_Z-Score',
 'Preventable hospital stays_# Medicare Enrollees',
 'Preventable hospital stays_Preventable Hosp. Rate',
 'Preventable hospital stays_95% CI - Low',
 'Preventable hospital stays_95% CI - High',
 'Preventable hospital stays_Z-Score',
 'Diabetic monitoring_# Diabetics',
 'Diabetic monitoring_% Receiving HbA1c',
 'Diabetic monitoring_95% CI - Low',
 'Diabetic monitoring_95% CI - High',
 'Diabetic monitoring_Z-Score',
 'Mammography screening_# Medicare Enrollees',
 'Mammography screening_% Mammography',
 'Mammography screening_95% CI - Low',
 'Mammography screening_95% CI - High',
 'Mammography screening_Z-Score',
 'High school graduation_Cohort Size',
 'High school graduation_Graduation Rate',
 'High school graduation_Z-Score',
 'Some college_95% CI - Low',
 'Some college_95% CI - High',
 'Some college_Z-Score',
 'Unemployment_Z-Score',
 'Children in poverty_# Children in Poverty',
 'Children in poverty_% Children in Poverty',
 'Children in poverty_95% CI - Low',
 'Children in poverty_95% CI - High',
 'Children in poverty_Z-Score',
 'Income inequality_80th Percentile Income',
 'Income inequality_20th Percentile Income',
 'Income inequality_Income Ratio',
 'Income inequality_Z-Score',
 'Children in single-parent households_# Single-Parent Households',
 'Children in single-parent households_# Households',
 'Children in single-parent households_% Single-Parent Households',
 'Children in single-parent households_95% CI - Low',
 'Children in single-parent households_95% CI - High',
 'Children in single-parent households_Z-Score',
 'Social associations_# Associations',
 'Social associations_Association Rate',
 'Social associations_Z-Score',
 'Violent crime_Z-Score',
 'Injury deaths_# Injury Deaths',
 'Injury deaths_Injury Death Rate',
 'Injury deaths_95% CI - Low',
 'Injury deaths_95% CI - High',
 'Injury deaths_Z-Score',
 'Air pollution - particulate matter_Average Daily PM2.5',
 'Air pollution - particulate matter_Z-Score',
 'Drinking water violations_Presence of violation',
 'Drinking water violations_Z-Score',
 'Severe housing problems_95% CI - Low',
 'Severe housing problems_95% CI - High',
 'Severe housing problems_Z-Score',
 'Driving alone to work_# Drive Alone',
 'Driving alone to work_# Workers',
 'Driving alone to work_% Drive Alone',
 'Driving alone to work_95% CI - Low',
 'Driving alone to work_95% CI - High',
 'Driving alone to work_Z-Score',
 'Long commute - driving alone_# Workers who Drive Alone',
 'Long commute - driving alone_% Long Commute - Drives Alone',
 'Long commute - driving alone_95% CI - Low',
 'Long commute - driving alone_95% CI - High',
 'Long commute - driving alone_Z-Score',
 'Length of Life_Z-Score',
 'Length of Life_Rank',
 'Quality of Life_Z-Score',
 'Health Behaviors_Z-Score',
 'Clinical Care_Z-Score',
 'Social & Economic Factors_Z-Score',
 'Physical Environment_Z-Score']

In [59]:
# Dropping columns in-place
chr16.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr16)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  52.0   
2    29003  Missouri          Andrew                  11.0   
3    29005  Missouri        Atchison                  17.0   
4    29007  Missouri         Audrain                  39.0   
5    29009  Missouri           Barry                  90.0   
6    29011  Missouri          Barton                  69.0   
7    29013  Missouri           Bates                  83.0   
8    29015  Missouri          Benton                  87.0   
9    29017  Missouri       Bollinger                  50.0   
10   29019  Missouri           Boone                  14.0   
11   29021  Missouri        Buchanan                  73.0   
12   29023  Missouri          Butler                 106.0   
13   29025  Missouri        Caldwell                  66.0   
14   29027  Missouri      

In [60]:
chr16.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Frequent physical distress_% Frequent Physical Distress',
 'Frequent mental distress_% Frequent Mental Distress',
 'Diabetes prevalence_% Diabetic',
 'Food insecurity_# Food Insecure',
 'Food insecurity_% Food Insecure',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Uninsured adults_# Uninsured',
 'Uninsured adults_% Uninsured',
 'Other primary care providers^_Other PCP Rate',
 'Other primary care providers^_Other PCP Ratio',
 'Median household income_Household Income',
 'Demographics_Population',
 'Demographics_% < 18',
 'Demographics_% 65 and over',
 'Demographics_# Rural',
 'Demographics_% Rural',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Food environment index_Food Environment Index',
 

In [124]:
# Columns to be dropped
columns_to_drop = columns_to_drop = [
 'Frequent physical distress_% Frequent Physical Distress',
 'Frequent mental distress_% Frequent Mental Distress',
 'Diabetes prevalence_% Diabetic',
 'Food insecurity_# Food Insecure',
 'Food insecurity_% Food Insecure',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Uninsured adults_# Uninsured',
 'Other primary care providers^_Other PCP Rate',
 'Other primary care providers^_Other PCP Ratio',
 'Median household income_Household Income',
 'Demographics_Population',
 'Demographics_% < 18',
 'Demographics_% 65 and over',
 'Demographics_# Rural',
 'Demographics_% Rural',
 'Food environment index_Food Environment Index',
 'Physical inactivity_% Physically Inactive',
 'Access to exercise opportunities_% With Access',
 'Uninsured_# Uninsured',
 'Uninsured_% Uninsured',
 'Mental health providers^_# Mental Health Providers',
 'Mental health providers^_MHP Rate',
 'Mental health providers^_MHP Ratio',
 'Some college_# Some College',
 'Some college_Population',
 'Some college_% Some College',
 'Unemployment_# Unemployed',
 'Unemployment_Labor Force',
 'Violent crime_# Violent Crimes',
 'Severe housing problems_# Households with Severe Problems',
 'Severe housing problems_% Severe Housing Problems',
 'Quality of Life_Rank']

In [125]:
# Dropping columns in-place
chr16.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr16)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  52.0   
2    29003  Missouri          Andrew                  11.0   
3    29005  Missouri        Atchison                  17.0   
4    29007  Missouri         Audrain                  39.0   
5    29009  Missouri           Barry                  90.0   
6    29011  Missouri          Barton                  69.0   
7    29013  Missouri           Bates                  83.0   
8    29015  Missouri          Benton                  87.0   
9    29017  Missouri       Bollinger                  50.0   
10   29019  Missouri           Boone                  14.0   
11   29021  Missouri        Buchanan                  73.0   
12   29023  Missouri          Butler                 106.0   
13   29025  Missouri        Caldwell                  66.0   
14   29027  Missouri      

In [196]:
chr16.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Uninsured adults_% Uninsured',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Unemployment_% Unemployed',
 'Violent Crime Rate',
 'Health Behaviors_Rank',
 'Clinical Care_Rank',
 'Social & Economic Factors_Rank',
 'Physical Environment_Rank',
 'year']

In [195]:
chr16.rename(columns={'Violent crime_Violent Crime Rate': 'Violent Crime Rate'}, inplace=True)

In [127]:
data_15_1 = pd.read_excel('2015 County Health Rankings Missouri Data - v3.xls',sheet_name = 'Outcomes & Factors Rankings', header = [0,1])
data_15_2 = pd.read_excel('2015 County Health Rankings Missouri Data - v3.xls',sheet_name = 'Additional Measure Data', header = [0,1])
data_15_3 = pd.read_excel('2015 County Health Rankings Missouri Data - v3.xls',sheet_name = 'Ranked Measure Data', header = [0,1])
data_15_4 = pd.read_excel('2015 County Health Rankings Missouri Data - v3.xls',sheet_name = 'Outcomes & Factors SubRankings', header = [0,1])

In [128]:
data_15_1 = add_index_prefix(data_15_1)
data_15_2 = add_index_prefix(data_15_2)
data_15_3 = add_index_prefix(data_15_3)
data_15_4 = add_index_prefix(data_15_4)

display(data_15_1.columns,data_15_2.columns, data_15_3.columns , data_15_4.columns)

Index(['FIPS', 'State', 'County', 'Health Outcomes_Z-Score',
       'Health Outcomes_Rank', 'Health Factors_Z-Score',
       'Health Factors_Rank'],
      dtype='object')

Index(['FIPS', 'State', 'County', 'Demographics_Population',
       'Demographics_# < 18', 'Demographics_% < 18',
       'Demographics_# 65 and over', 'Demographics_% 65 and over',
       'Demographics_# African American', 'Demographics_% African American',
       'Demographics_# American Indian/ Alaskan Native',
       'Demographics_% American Indian/ Alaskan Native',
       'Demographics_# Asian', 'Demographics_% Asian',
       'Demographics_# Native Hawaiian/ Other Pacific Islander',
       'Demographics_% Native Hawaiian/ Other Pacific Islander',
       'Demographics_# Hispanic', 'Demographics_% Hispanic',
       'Demographics_# Non-Hispanic white',
       'Demographics_% Non-Hispanic white',
       'Demographics_# Not Proficient in English',
       'Demographics_% Not Proficient in English', 'Demographics_95% CI - Low',
       'Demographics_95% CI - High', 'Demographics_# Female',
       'Demographics_% Female', 'Demographics_# Rural', 'Demographics_% Rural',
       'Diabetes_# Di

Index(['FIPS', 'State', 'County', 'Premature death_# Deaths',
       'Premature death_Years of Potential Life Lost Rate',
       'Premature death_95% CI - Low', 'Premature death_95% CI - High',
       'Premature death_Z-Score', 'Poor or fair health_Sample Size',
       'Poor or fair health_% Fair/Poor',
       ...
       'Driving alone to work_# Workers',
       'Driving alone to work_% Drive Alone',
       'Driving alone to work_95% CI - Low',
       'Driving alone to work_95% CI - High', 'Driving alone to work_Z-Score',
       'Long commute - driving alone_# Workers who Drive Alone',
       'Long commute - driving alone_% Long Commute - Drives Alone',
       'Long commute - driving alone_95% CI - Low',
       'Long commute - driving alone_95% CI - High',
       'Long commute - driving alone_Z-Score'],
      dtype='object', length=161)

Index(['FIPS', 'State', 'County', 'Length of Life_Z-Score',
       'Length of Life_Rank', 'Quality of Life_Z-Score',
       'Quality of Life_Rank', 'Health Behaviors_Z-Score',
       'Health Behaviors_Rank', 'Clinical Care_Z-Score', 'Clinical Care_Rank',
       'Social & Economic Factors_Z-Score', 'Social & Economic Factors_Rank',
       'Physical Environment_Z-Score', 'Physical Environment_Rank'],
      dtype='object')

In [129]:
chr15 = pd.merge(data_15_1 , data_15_2 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr15 = pd.merge(chr15 , data_15_3 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr15 = pd.merge(chr15 , data_15_4 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr15

,FIPS,State,County,Health Outcomes_Z-Score,Health Outcomes_Rank,Health Factors_Z-Score,Health Factors_Rank,Demographics_Population,Demographics_# < 18,Demographics_% < 18,...,Quality of Life_Z-Score,Quality of Life_Rank,Health Behaviors_Z-Score,Health Behaviors_Rank,Clinical Care_Z-Score,Clinical Care_Rank,Social & Economic Factors_Z-Score,Social & Economic Factors_Rank,Physical Environment_Z-Score,Physical Environment_Rank
0,29000,Missouri,NaN,NaN,NaN,NaN,NaN,6044171.0,1397685.0,23.124511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29001,Missouri,Adair,-0.172722,53,-0.358675,23,25572.0,4753.0,18.586735,...,0.197451,88,-0.079604,32,-0.199515,8,-0.157436,26,0.077880,106
2,29003,Missouri,Andrew,-0.904375,10,-0.705849,7,17445.0,4035.0,23.129837,...,-0.638826,1,-0.176102,17,-0.088924,25,-0.385426,6,-0.055398,12
3,29005,Missouri,Atchison,-0.590034,25,-0.513316,15,5449.0,1091.0,20.022022,...,-0.494177,4,-0.029596,49,0.015269,65,-0.420515,2,-0.078474,5
4,29007,Missouri,Audrain,-0.206459,48,-0.021788,57,25661.0,6015.0,23.440240,...,0.187392,87,0.028036,66,-0.038167,36,0.017954,67,-0.029611,34
5,29009,Missouri,Barry,0.427370,88,0.028671,63,35572.0,8300.0,23.332959,...,0.079249,77,0.082273,86,-0.022755,47,-0.025285,56,-0.005562,58
6,29011,Missouri,Barton,-0.380227,39,0.118224,71,12275.0,3123.0,25.441955,...,-0.025224,66,-0.045470,41,0.080207,84,0.053306,72,0.030181,83
7,29013,Missouri,Bates,-0.097759,60,0.366954,91,16550.0,3971.0,23.993958,...,-0.249380,24,0.076216,83,0.176035,108,0.126604,86,-0.011901,52
8,29015,Missouri,Benton,0.122781,78,0.301854,85,18932.0,3335.0,17.615677,...,-0.098084,57,-0.036698,44,0.116776,96,0.240828,99,-0.019051,40
9,29017,Missouri,Bollinger,-0.142220,56,0.369106,92,12490.0,2844.0,22.770216,...,-0.080255,62,0.128451,93,0.072414,81,0.107695,81,0.060546,99


In [130]:
chr15['year'] = 2015

In [131]:
chr15.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Z-Score',
 'Health Outcomes_Rank',
 'Health Factors_Z-Score',
 'Health Factors_Rank',
 'Demographics_Population',
 'Demographics_# < 18',
 'Demographics_% < 18',
 'Demographics_# 65 and over',
 'Demographics_% 65 and over',
 'Demographics_# African American',
 'Demographics_% African American',
 'Demographics_# American Indian/ Alaskan Native',
 'Demographics_% American Indian/ Alaskan Native',
 'Demographics_# Asian',
 'Demographics_% Asian',
 'Demographics_# Native Hawaiian/ Other Pacific Islander',
 'Demographics_% Native Hawaiian/ Other Pacific Islander',
 'Demographics_# Hispanic',
 'Demographics_% Hispanic',
 'Demographics_# Non-Hispanic white',
 'Demographics_% Non-Hispanic white',
 'Demographics_# Not Proficient in English',
 'Demographics_% Not Proficient in English',
 'Demographics_95% CI - Low',
 'Demographics_95% CI - High',
 'Demographics_# Female',
 'Demographics_% Female',
 'Demographics_# Rural',
 'Demographics_% Rural',
 

In [132]:
# Columns to be dropped
columns_to_drop = [
 'Health Outcomes_Z-Score',
 'Health Factors_Z-Score',
 'Demographics_# African American',
 'Demographics_% African American',
 'Demographics_# American Indian/ Alaskan Native',
 'Demographics_% American Indian/ Alaskan Native',
 'Demographics_# Asian',
 'Demographics_% Asian',
 'Demographics_# Native Hawaiian/ Other Pacific Islander',
 'Demographics_% Native Hawaiian/ Other Pacific Islander',
 'Demographics_# Hispanic',
 'Demographics_% Hispanic',
 'Demographics_# Non-Hispanic white',
 'Demographics_% Non-Hispanic white',
 'Demographics_# Not Proficient in English',
 'Demographics_% Not Proficient in English',
 'Demographics_95% CI - Low',
 'Demographics_95% CI - High',
 'Demographics_# Female',
 'Demographics_% Female',
 'Diabetes_95% CI - Low',
 'Diabetes_95% CI - High',
 'HIV prevalence_# HIV Cases',
 'HIV prevalence_HIV Prevalence Rate',
 'Premature age-adjusted mortality_# Deaths',
 'Premature age-adjusted mortality_Age-Adjusted Mortality',
 'Premature age-adjusted mortality_95% CI - Low',
 'Premature age-adjusted mortality_95% CI - High',
 'Infant mortality_# Deaths',
 'Infant mortality_Infant Mortality Rate',
 'Infant mortality_95% CI - Low',
 'Infant mortality_95% CI - High',
 'Child mortality_# Deaths',
 'Child mortality_Child Mortality Rate',
 'Child mortality_95% CI - Low',
 'Child mortality_95% CI - High',
 'Motor vehicle crash deaths_# Motor Vehicle Deaths',
 'Motor vehicle crash deaths_MV Mortality Rate',
 'Motor vehicle crash deaths_95% CI - Low',
 'Motor vehicle crash deaths_95% CI - High',
 'Drug poisoning deaths_# Drug Poisoning Deaths',
 'Drug poisoning deaths_Drug Poisoning Mortality Rate',
 'Uninsured adults_95% CI - Low',
 'Uninsured adults_95% CI - High',
 'Uninsured children_# Uninsured',
 'Uninsured children_% Uninsured',
 'Uninsured children_95% CI - Low',
 'Uninsured children_95% CI - High',
 'Health care costs_Costs',
 'Could not see doctor due to cost_Sample Size',
 "Could not see doctor due to cost_% Couldn't Access",
 'Could not see doctor due to cost_95% CI - Low',
 'Could not see doctor due to cost_95% CI - High',
 'Previous Other primary care providers data^_Other PCP Rate',
 'Previous Other primary care providers data^_Other PCP Ratio',
 'Median household income_95% CI - Low',
 'Median household income_95% CI - High',
 'Children eligible for free lunch_% Free Lunch',
 'Homicides_Homicide Rate',
 'Homicides_95% CI - Low',
 'Homicides_95% CI - High',
 'Premature death_# Deaths',
 'Premature death_Years of Potential Life Lost Rate',
 'Premature death_95% CI - Low',
 'Premature death_95% CI - High',
 'Premature death_Z-Score',
 'Poor or fair health_Sample Size',
 'Poor or fair health_95% CI - Low',
 'Poor or fair health_95% CI - High',
 'Poor or fair health_Z-Score',
 'Poor physical health days_Sample Size',
 'Poor physical health days_95% CI - Low',
 'Poor physical health days_95% CI - High',
 'Poor physical health days_Z-Score',
 'Poor mental health days_Sample Size',
 'Poor mental health days_95% CI - Low',
 'Poor mental health days_95% CI - High',
 'Poor mental health days_Z-Score',
 'Low birthweight_Unreliable',
 'Low birthweight_# Low Birthweight Births',
 'Low birthweight_# Live births',
 'Low birthweight_% LBW',
 'Low birthweight_95% CI - Low',
 'Low birthweight_95% CI - High',
 'Low birthweight_Z-Score',
 'Adult smoking_Sample Size',
 'Adult smoking_95% CI - Low',
 'Adult smoking_95% CI - High',
 'Adult smoking_Z-Score',
 'Adult obesity_95% CI - Low',
 'Adult obesity_95% CI - High',
 'Adult obesity_Z-Score',
 'Food environment index_Z-Score',
 'Physical inactivity_95% CI - Low',
 'Physical inactivity_95% CI - High',
 'Physical inactivity_Z-Score',
 'Access to exercise opportunities_Z-Score',
 'Excessive drinking_Sample Size',
 'Excessive drinking_% Excessive Drinking',
 'Excessive drinking_95% CI - Low',
 'Excessive drinking_95% CI - High',
 'Excessive drinking_Z-Score',
 'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths',
 'Alcohol-impaired driving deaths_# Driving Deaths',
 'Alcohol-impaired driving deaths_% Alcohol-Impaired',
 'Alcohol-impaired driving deaths_Z-Score',
 'Sexually transmitted infections_# Chlamydia Cases',
 'Sexually transmitted infections_Chlamydia Rate',
 'Sexually transmitted infections_Z-Score',
 'Teen births_Teen Births ',
 'Teen births_Teen Population',
 'Teen births_Teen Birth Rate',
 'Teen births_95% CI - Low',
 'Teen births_95% CI - High',
 'Teen births_Z-Score',
 'Uninsured_95% CI - Low',
 'Uninsured_95% CI - High',
 'Uninsured_Z-Score',
 'Primary care physicians_Z-Score',
 'Dentists_# Dentists',
 'Dentists_Dentist Rate',
 'Dentists_Dentist Ratio',
 'Dentists_Z-Score',
 'Previous Mental health providers data -- used to calculate rankings^_# Mental Health Providers',
 'Previous Mental health providers data -- used to calculate rankings^_MHP Rate',
 'Previous Mental health providers data -- used to calculate rankings^_MHP Ratio',
 'Previous Mental health providers data -- used to calculate rankings^_Z-Score',
 'Preventable hospital stays_# Medicare Enrollees',
 'Preventable hospital stays_Preventable Hosp. Rate',
 'Preventable hospital stays_95% CI - Low',
 'Preventable hospital stays_95% CI - High',
 'Preventable hospital stays_Z-Score',
 'Diabetic monitoring_# Diabetics',
 'Diabetic monitoring_% Receiving HbA1c',
 'Diabetic monitoring_95% CI - Low',
 'Diabetic monitoring_95% CI - High',
 'Diabetic monitoring_Z-Score',
 'Mammography screening_# Medicare Enrollees',
 'Mammography screening_% Mammography',
 'Mammography screening_95% CI - Low',
 'Mammography screening_95% CI - High',
 'Mammography screening_Z-Score',
 'High school graduation_Cohort Size',
 'High school graduation_Graduation Rate',
 'High school graduation_Z-Score',
 'Some college_95% CI - Low',
 'Some college_95% CI - High',
 'Some college_Z-Score',
 'Unemployment_# Unemployed',
 'Unemployment_Labor Force',
 'Unemployment_Z-Score',
 'Children in poverty_# Children in Poverty',
 'Children in poverty_% Children in Poverty',
 'Children in poverty_95% CI - Low',
 'Children in poverty_95% CI - High',
 'Children in poverty_Z-Score',
 'Income inequality_80th Percentile Income',
 'Income inequality_20th Percentile Income',
 'Income inequality_Income Ratio',
 'Income inequality_Z-Score',
 'Children in single-parent households_# Single-Parent Households',
 'Children in single-parent households_# Households',
 'Children in single-parent households_% Single-Parent Households',
 'Children in single-parent households_95% CI - Low',
 'Children in single-parent households_95% CI - High',
 'Children in single-parent households_Z-Score',
 'Social associations_# Associations',
 'Social associations_Association Rate',
 'Social associations_Z-Score',
 'Violent crime_Z-Score',
 'Injury deaths_# Injury Deaths',
 'Injury deaths_Injury Death Rate',
 'Injury deaths_95% CI - Low',
 'Injury deaths_95% CI - High',
 'Injury deaths_Z-Score',
 'Air pollution - particulate matter_Average Daily PM2.5',
 'Air pollution - particulate matter_Z-Score',
 'Drinking water violations_Pop. In Viol',
 'Drinking water violations_% Pop in Viol',
 'Drinking water violations_Z-Score',
 'Severe housing problems_# Households with Severe Problems',
 'Severe housing problems_% Severe Housing Problems',
 'Severe housing problems_95% CI - Low',
 'Severe housing problems_95% CI - High',
 'Severe housing problems_Z-Score',
 'Driving alone to work_# Drive Alone',
 'Driving alone to work_# Workers',
 'Driving alone to work_% Drive Alone',
 'Driving alone to work_95% CI - Low',
 'Driving alone to work_95% CI - High',
 'Driving alone to work_Z-Score',
 'Long commute - driving alone_# Workers who Drive Alone',
 'Long commute - driving alone_% Long Commute - Drives Alone',
 'Long commute - driving alone_95% CI - Low',
 'Long commute - driving alone_95% CI - High',
 'Long commute - driving alone_Z-Score',
 'Length of Life_Z-Score',
 'Length of Life_Rank',
 'Quality of Life_Z-Score',
 'Health Behaviors_Z-Score',
 'Clinical Care_Z-Score',
 'Social & Economic Factors_Z-Score',
 'Physical Environment_Z-Score']

In [133]:
# Dropping columns in-place
chr15.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr15)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County Health Outcomes_Rank Health Factors_Rank  \
0    29000  Missouri             NaN                  NaN                 NaN   
1    29001  Missouri           Adair                   53                  23   
2    29003  Missouri          Andrew                   10                   7   
3    29005  Missouri        Atchison                   25                  15   
4    29007  Missouri         Audrain                   48                  57   
5    29009  Missouri           Barry                   88                  63   
6    29011  Missouri          Barton                   39                  71   
7    29013  Missouri           Bates                   60                  91   
8    29015  Missouri          Benton                   78                  85   
9    29017  Missouri       Bollinger                   56                  92   
10   29019  Missouri           Boone                    7        

In [134]:
chr15.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Demographics_Population',
 'Demographics_# < 18',
 'Demographics_% < 18',
 'Demographics_# 65 and over',
 'Demographics_% 65 and over',
 'Demographics_# Rural',
 'Demographics_% Rural',
 'Diabetes_# Diabetic',
 'Diabetes_% Diabetic',
 'Food insecurity_# Food Insecure',
 'Food insecurity_% Food Insecure',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Uninsured adults_# Uninsured',
 'Uninsured adults_% Uninsured',
 'Other primary care providers^_Other PCP Rate',
 'Other primary care providers^_Other PCP Ratio',
 'Median household income_Household Income',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Food environment index_Food Environment Index',
 'Physical inactivity_% Physically Inactive',
 'A

In [135]:
# Columns to be dropped
columns_to_drop = columns_to_drop = [
 'Demographics_Population',
 'Demographics_# < 18',
 'Demographics_% < 18',
 'Demographics_# 65 and over',
 'Demographics_% 65 and over',
 'Demographics_# Rural',
 'Demographics_% Rural',
 'Diabetes_# Diabetic',
 'Diabetes_% Diabetic',
 'Food insecurity_# Food Insecure',
 'Food insecurity_% Food Insecure',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Uninsured adults_# Uninsured',
 'Other primary care providers^_Other PCP Rate',
 'Other primary care providers^_Other PCP Ratio',
 'Median household income_Household Income',
 'Food environment index_Food Environment Index',
 'Physical inactivity_% Physically Inactive',
 'Access to exercise opportunities_# With Access',
 'Access to exercise opportunities_% With Access',
 'Uninsured_# Uninsured',
 'Uninsured_% Uninsured',
 'Primary care physicians_# Primary Care Physicians',
 'Primary care physicians_PCP Rate',
 'Primary care physicians_PCP Ratio',
 'Mental health providers^_# Mental Health Providers',
 'Mental health providers^_MHP Rate',
 'Mental health providers^_MHP Ratio',
 'Some college_# Some College',
 'Some college_Population',
 'Some college_% Some College',
 'Violent crime_# Violent Crimes',
 'Quality of Life_Rank']

In [136]:
# Dropping columns in-place
chr15.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr15)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County Health Outcomes_Rank Health Factors_Rank  \
0    29000  Missouri             NaN                  NaN                 NaN   
1    29001  Missouri           Adair                   53                  23   
2    29003  Missouri          Andrew                   10                   7   
3    29005  Missouri        Atchison                   25                  15   
4    29007  Missouri         Audrain                   48                  57   
5    29009  Missouri           Barry                   88                  63   
6    29011  Missouri          Barton                   39                  71   
7    29013  Missouri           Bates                   60                  91   
8    29015  Missouri          Benton                   78                  85   
9    29017  Missouri       Bollinger                   56                  92   
10   29019  Missouri           Boone                    7        

In [194]:
chr15.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Uninsured adults_% Uninsured',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Unemployment_% Unemployed',
 'Violent Crime Rate',
 'Health Behaviors_Rank',
 'Clinical Care_Rank',
 'Social & Economic Factors_Rank',
 'Physical Environment_Rank',
 'year']

In [193]:
chr15.rename(columns={'Violent crime_Violent Crime Rate': 'Violent Crime Rate'}, inplace=True)

In [69]:
data_14_1 = pd.read_excel('2014 County Health Rankings Missouri Data - v6.xls',sheet_name = 'Outcomes & Factors Rankings', header = [0,1])
data_14_2 = pd.read_excel('2014 County Health Rankings Missouri Data - v6.xls',sheet_name = 'Additional Measure Data', header = [0,1])
data_14_3 = pd.read_excel('2014 County Health Rankings Missouri Data - v6.xls',sheet_name = 'Ranked Measure Data', header = [0,1])
data_14_4 = pd.read_excel('2014 County Health Rankings Missouri Data - v6.xls',sheet_name = 'Outcomes & Factors SubRankings', header = [0,1])

In [70]:
data_14_1 = add_index_prefix(data_14_1)
data_14_2 = add_index_prefix(data_14_2)
data_14_3 = add_index_prefix(data_14_3)
data_14_4 = add_index_prefix(data_14_4)

display(data_14_1.columns,data_14_2.columns, data_14_3.columns , data_14_4.columns)

Index(['FIPS', 'State', 'County', 'Health Outcomes_Z-Score',
       'Health Outcomes_Rank', 'Health Factors_Z-Score',
       'Health Factors_Rank'],
      dtype='object')

Index(['FIPS', 'State', 'County', 'Demographics_Population',
       'Demographics_< 18', 'Demographics_65 and over',
       'Demographics_African American',
       'Demographics_American Indian/ Alaskan Native', 'Demographics_Asian',
       'Demographics_Native Hawaiian/ Other Pacific Islander',
       'Demographics_Hispanic', 'Demographics_Non-Hispanic white',
       'Demographics_# Not Proficient in English',
       'Demographics_% Not Proficient in English', 'Demographics_95% CI - Low',
       'Demographics_95% CI - High', 'Demographics_Female',
       'Demographics_Rural', 'Diabetes_% Diabetic', 'Diabetes_95% CI - Low',
       'Diabetes_95% CI - High', 'HIV prevalence rate_HIV Cases',
       'HIV prevalence rate_HIV Rate',
       'Premature age-adjusted mortality_Total deaths',
       'Premature age-adjusted mortality_Age-adjusted Mortality',
       'Premature age-adjusted mortality_95% CI - Low',
       'Premature age-adjusted mortality_95% CI - High',
       'Infant mortality_Inf

Index(['FIPS', 'State', 'County',
       'Premature death (Years of Potential Life Lost)_Deaths',
       'Premature death (Years of Potential Life Lost)_YPLL Rate',
       'Premature death (Years of Potential Life Lost)_95% CI - Low',
       'Premature death (Years of Potential Life Lost)_95% CI - High',
       'Premature death (Years of Potential Life Lost)_Z-Score',
       'Poor or fair health_Sample Size', 'Poor or fair health_% Fair/Poor',
       ...
       'Driving alone to work_Workers', 'Driving alone to work_% Drive Alone',
       'Driving alone to work_95% CI - Low',
       'Driving alone to work_95% CI - High', 'Driving alone to work_Z-Score',
       'Long commute - driving alone_Workers who Drive Alone',
       'Long commute - driving alone_Long Commute - Drives Alone',
       'Long commute - driving alone_95% CI - Low',
       'Long commute - driving alone_95% CI - High',
       'Long commute - driving alone_Z-Score'],
      dtype='object', length=161)

Index(['FIPS', 'State', 'County', 'Length of Life_Z-Score',
       'Length of Life_Rank', 'Quality of Life_Z-Score',
       'Quality of Life_Rank', 'Health Behaviors_Z-Score',
       'Health Behaviors_Rank', 'Clinical Care_Z-Score', 'Clinical Care_Rank',
       'Social & Economic Factors_Z-Score', 'Social & Economic Factors_Rank',
       'Physical Environment_Z-Score', 'Physical Environment_Rank'],
      dtype='object')

In [71]:
chr14 = pd.merge(data_14_1 , data_14_2 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr14 = pd.merge(chr14 , data_14_3 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr14 = pd.merge(chr14 , data_14_4 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr14

,FIPS,State,County,Health Outcomes_Z-Score,Health Outcomes_Rank,Health Factors_Z-Score,Health Factors_Rank,Demographics_Population,Demographics_< 18,Demographics_65 and over,...,Quality of Life_Z-Score,Quality of Life_Rank,Health Behaviors_Z-Score,Health Behaviors_Rank,Clinical Care_Z-Score,Clinical Care_Rank,Social & Economic Factors_Z-Score,Social & Economic Factors_Rank,Physical Environment_Z-Score,Physical Environment_Rank
0,29000,Missouri,NaN,NaN,NaN,NaN,NaN,6021988.0,23.305842,14.665938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29001,Missouri,Adair,-0.318116,42.0,-0.312390,23.0,25581.0,18.627106,13.259841,...,0.195929,90.0,-0.057934,37.0,-0.139325,13.0,-0.177918,31.0,0.062787,106.0
2,29003,Missouri,Andrew,-1.299846,2.0,-0.689512,10.0,17417.0,23.270368,16.696331,...,-0.676534,1.0,-0.166868,17.0,-0.021034,49.0,-0.462252,3.0,-0.039357,23.0
3,29005,Missouri,Atchison,-0.851891,13.0,-0.292278,27.0,5517.0,19.684611,22.820373,...,-0.415522,7.0,-0.087810,31.0,0.117997,99.0,-0.290124,13.0,-0.032341,33.0
4,29007,Missouri,Audrain,-0.014768,68.0,-0.149045,46.0,25621.0,24.538465,16.221069,...,0.146489,85.0,0.007740,61.0,-0.054192,32.0,-0.082503,49.0,-0.020091,43.0
5,29009,Missouri,Barry,0.217203,85.0,0.039201,65.0,35546.0,23.591965,19.248298,...,0.118324,81.0,0.049638,74.0,-0.053975,33.0,0.046846,70.0,-0.003308,57.0
6,29011,Missouri,Barton,-0.261708,48.0,0.151235,73.0,12337.0,25.622112,18.181081,...,-0.015095,67.0,-0.072758,34.0,0.157935,108.0,0.050176,72.0,0.015881,75.0
7,29013,Missouri,Bates,-0.488145,25.0,0.287236,90.0,16709.0,24.154647,18.678556,...,-0.251886,29.0,0.071588,79.0,0.147162,105.0,0.081320,80.0,-0.012835,50.0
8,29015,Missouri,Benton,0.171304,79.0,0.176050,76.0,18962.0,17.445417,27.201772,...,-0.136758,47.0,-0.026379,48.0,0.065940,84.0,0.150803,88.0,-0.014314,48.0
9,29017,Missouri,Bollinger,-0.342586,36.0,0.187929,77.0,12382.0,23.089969,17.024713,...,-0.034510,63.0,0.117090,94.0,0.046834,75.0,-0.036446,56.0,0.060451,105.0


In [72]:
chr14['year'] = 2014

In [73]:
chr14.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Z-Score',
 'Health Outcomes_Rank',
 'Health Factors_Z-Score',
 'Health Factors_Rank',
 'Demographics_Population',
 'Demographics_< 18',
 'Demographics_65 and over',
 'Demographics_African American',
 'Demographics_American Indian/ Alaskan Native',
 'Demographics_Asian',
 'Demographics_Native Hawaiian/ Other Pacific Islander',
 'Demographics_Hispanic',
 'Demographics_Non-Hispanic white',
 'Demographics_# Not Proficient in English',
 'Demographics_% Not Proficient in English',
 'Demographics_95% CI - Low',
 'Demographics_95% CI - High',
 'Demographics_Female',
 'Demographics_Rural',
 'Diabetes_% Diabetic',
 'Diabetes_95% CI - Low',
 'Diabetes_95% CI - High',
 'HIV prevalence rate_HIV Cases',
 'HIV prevalence rate_HIV Rate',
 'Premature age-adjusted mortality_Total deaths',
 'Premature age-adjusted mortality_Age-adjusted Mortality',
 'Premature age-adjusted mortality_95% CI - Low',
 'Premature age-adjusted mortality_95% CI - High',
 'Infant 

In [74]:
# Columns to be dropped
columns_to_drop = [
 'Health Outcomes_Z-Score',
 'Health Factors_Z-Score',
 'Demographics_African American',
 'Demographics_American Indian/ Alaskan Native',
 'Demographics_Asian',
 'Demographics_Native Hawaiian/ Other Pacific Islander',
 'Demographics_Hispanic',
 'Demographics_Non-Hispanic white',
 'Demographics_# Not Proficient in English',
 'Demographics_% Not Proficient in English',
 'Demographics_95% CI - Low',
 'Demographics_95% CI - High',
 'Demographics_Female',
 'Diabetes_95% CI - Low',
 'Diabetes_95% CI - High',
 'HIV prevalence rate_HIV Cases',
 'HIV prevalence rate_HIV Rate',
 'Premature age-adjusted mortality_Total deaths',
 'Premature age-adjusted mortality_Age-adjusted Mortality',
 'Premature age-adjusted mortality_95% CI - Low',
 'Premature age-adjusted mortality_95% CI - High',
 'Infant mortality_Infant mortality rate',
 'Infant mortality_95% CI - Low',
 'Infant mortality_95% CI - High',
 'Child mortality_Total deaths',
 'Child mortality_Child mortality rate',
 'Child mortality_95% CI - Low',
 'Child mortality_95% CI - High',
 'Motor vehicle crash death rate_Motor Vehicle Deaths',
 'Motor vehicle crash death rate_MV Mortality Rate',
 'Motor vehicle crash death rate_95% CI - Low',
 'Motor vehicle crash death rate_95% CI - High',
 'Drug poisoning deaths_Drug Poisoning Deaths',
 'Drug poisoning deaths_Drug Poisoning Mortality Rate',
 'Uninsured adults_95% CI - Low',
 'Uninsured adults_95% CI - High',
 'Uninsured children_# Uninsured',
 'Uninsured children_% Uninsured',
 'Uninsured children_95% CI - Low',
 'Uninsured children_95% CI - High',
 'Health care costs_Costs',
 'Health care costs_95% CI - Low',
 'Health care costs_95% CI - High',
 'Could not see doctor due to cost_Sample Size',
 "Could not see doctor due to cost_% Couldn't Access",
 'Could not see doctor due to cost_95% CI - Low',
 'Could not see doctor due to cost_95% CI - High',
 'Other primary care providers^_Other PCP Rate',
 'Other primary care providers^_Other PCP Ratio',
 'Previous Other primary care providers data^_Other PCP Rate',
 'Previous Other primary care providers data^_Other PCP Ratio',
 'Median household income_95% CI - Low',
 'Median household income_95% CI - High',
 'Children eligible for free lunch_% Free Lunch',
 'Homicide rate_Homicide Rate',
 'Homicide rate_95% CI - Low',
 'Homicide rate_95% CI - High',
 'Premature death (Years of Potential Life Lost)_Deaths',
 'Premature death (Years of Potential Life Lost)_YPLL Rate',
 'Premature death (Years of Potential Life Lost)_95% CI - Low',
 'Premature death (Years of Potential Life Lost)_95% CI - High',
 'Premature death (Years of Potential Life Lost)_Z-Score',
 'Poor or fair health_Sample Size',
 'Poor or fair health_95% CI - Low',
 'Poor or fair health_95% CI - High',
 'Poor or fair health_Z-Score',
 'Poor physical health days_Sample Size',
 'Poor physical health days_95% CI - Low',
 'Poor physical health days_95% CI - High',
 'Poor physical health days_Z-Score',
 'Poor mental health days_Sample Size',
 'Poor mental health days_95% CI - Low',
 'Poor mental health days_95% CI - High',
 'Poor mental health days_Z-Score',
 'Low birthweight_Unreliable',
 'Low birthweight_LBW Births',
 'Low birthweight_Live Births',
 'Low birthweight_% LBW',
 'Low birthweight_95% CI - Low',
 'Low birthweight_95% CI - High',
 'Low birthweight_Z-Score',
 'Adult smoking_Sample Size',
 'Adult smoking_95% CI - Low',
 'Adult smoking_95% CI - High',
 'Adult smoking_Z-Score',
 'Adult obesity_95% CI - Low',
 'Adult obesity_95% CI - High',
 'Adult obesity_Z-Score',
 'Food environment index_Z-Score',
 'Physical inactivity_95% CI - Low',
 'Physical inactivity_95% CI - High',
 'Physical inactivity_Z-Score',
 'Access to exercise opportunities_Z-Score',
 'Excessive drinking_Sample Size',
 'Excessive drinking_% Excessive Drinking',
 'Excessive drinking_95% CI - Low',
 'Excessive drinking_95% CI - High',
 'Excessive drinking_Z-Score',
 'Alcohol-impaired driving deaths_# Alcohol-Impaired Driving Deaths',
 'Alcohol-impaired driving deaths_# Driving Deaths',
 'Alcohol-impaired driving deaths_% Alcohol-Impaired',
 'Alcohol-impaired driving deaths_Z-Score',
 'Sexually transmitted infections_Chlamydia Cases',
 'Sexually transmitted infections_Chlamydia Rate',
 'Sexually transmitted infections_Z-Score',
 'Teen births_Teen Births ',
 'Teen births_Teen Population',
 'Teen births_Teen Birth Rate',
 'Teen births_95% CI - Low',
 'Teen births_95% CI - High',
 'Teen births_Z-Score',
 'Uninsured_# Uninsured',
 'Uninsured_% Uninsured',
 'Uninsured_95% CI - Low',
 'Uninsured_95% CI - High',
 'Uninsured_Z-Score',
 'Primary care physicians_Z-Score',
 'Dentists^_# Dentists',
 'Dentists^_Dentist Rate',
 'Dentists^_Dentist Ratio',
 'Previous Dentists data -- used to calculate rankings^_# Dentists',
 'Previous Dentists data -- used to calculate rankings^_Dentist Rate',
 'Previous Dentists data -- used to calculate rankings^_Dentist Ratio',
 'Previous Dentists data -- used to calculate rankings^_Z-Score',
 'Previous Mental health providers data -- used to calculate rankings^_# MHP',
 'Previous Mental health providers data -- used to calculate rankings^_MHP Rate',
 'Previous Mental health providers data -- used to calculate rankings^_MHP Ratio',
 'Previous Mental health providers data -- used to calculate rankings^_Z-Score',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_# Medicare enrollees',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_ACSC Rate',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_95% CI - Low',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_95% CI - High',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_Z-Score',
 'Diabetic screening_# Diabetics',
 'Diabetic screening_% HbA1c',
 'Diabetic screening_95% CI - Low',
 'Diabetic screening_95% CI - High',
 'Diabetic screening_Z-Score',
 'Mammography screening_# Medicare Enrollees',
 'Mammography screening_% Mammography',
 'Mammography screening_95% CI - Low',
 'Mammography screening_95% CI - High',
 'Mammography screening_Z-Score',
 'High school graduation*_Cohort size',
 'High school graduation*_Graduation Rate',
 'High school graduation*_Z-Score',
 'Some college (post-secondary education)_95% CI - Low',
 'Some college (post-secondary education)_95% CI - High',
 'Some college (post-secondary education)_Z-Score',
 'Unemployment_Z-Score',
 'Children in poverty_# Children in Poverty',
 'Children in poverty_% Children in Poverty',
 'Children in poverty_95% CI - Low',
 'Children in poverty_95% CI - High',
 'Children in poverty_Z-Score',
 'Inadequate social support_Sample Size',
 'Inadequate social support_% No Social-Emotional Support',
 'Inadequate social support_95% CI - Low',
 'Inadequate social support_95% CI - High',
 'Inadequate social support_Z-Score',
 'Children in single-parent households_# Single-Parent Households',
 'Children in single-parent households_# Households',
 'Children in single-parent households_% Single-Parent Households',
 'Children in single-parent households_95% CI - Low',
 'Children in single-parent households_95% CI - High',
 'Children in single-parent households_Z-Score',
 'Violent crime_Annual Violent Crimes',
 'Violent crime_Z-Score',
 'Injury deaths_Injury Deaths',
 'Injury deaths_Injury Death Rate',
 'Injury deaths_95% CI - Low',
 'Injury deaths_95% CI - High',
 'Injury deaths_Z-Score',
 'Air pollution - particulate matter_Average daily PM25',
 'Air pollution - particulate matter_Z-Score',
 'Drinking water violations_% pop in viol',
 'Drinking water violations_Pop. In Viol',
 'Drinking water violations_Z-Score',
 'Severe housing problems_95% CI - Low',
 'Severe housing problems_95% CI - High',
 'Severe housing problems_Z-Score',
 'Driving alone to work_# Drive Alone',
 'Driving alone to work_Workers',
 'Driving alone to work_% Drive Alone',
 'Driving alone to work_95% CI - Low',
 'Driving alone to work_95% CI - High',
 'Driving alone to work_Z-Score',
 'Long commute - driving alone_Workers who Drive Alone',
 'Long commute - driving alone_Long Commute - Drives Alone',
 'Long commute - driving alone_95% CI - Low',
 'Long commute - driving alone_95% CI - High',
 'Long commute - driving alone_Z-Score',
 'Length of Life_Z-Score',
 'Length of Life_Rank',
 'Quality of Life_Z-Score',
 'Health Behaviors_Z-Score',
 'Clinical Care_Z-Score',
 'Social & Economic Factors_Z-Score',
 'Physical Environment_Z-Score']

In [75]:
# Dropping columns in-place
chr14.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr14)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  42.0   
2    29003  Missouri          Andrew                   2.0   
3    29005  Missouri        Atchison                  13.0   
4    29007  Missouri         Audrain                  68.0   
5    29009  Missouri           Barry                  85.0   
6    29011  Missouri          Barton                  48.0   
7    29013  Missouri           Bates                  25.0   
8    29015  Missouri          Benton                  79.0   
9    29017  Missouri       Bollinger                  36.0   
10   29019  Missouri           Boone                   8.0   
11   29021  Missouri        Buchanan                  65.0   
12   29023  Missouri          Butler                 107.0   
13   29025  Missouri        Caldwell                  95.0   
14   29027  Missouri      

In [76]:
chr14.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Demographics_Population',
 'Demographics_< 18',
 'Demographics_65 and over',
 'Demographics_Rural',
 'Diabetes_% Diabetic',
 'Food insecurity_% Food Insecure',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Uninsured adults_# Uninsured',
 'Uninsured adults_% Uninsured',
 'Median household income_Household Income',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Food environment index_Food Environment Index',
 'Physical inactivity_% Physically Inactive',
 'Access to exercise opportunities_% With Access',
 'Primary care physicians_# PCP',
 'Primary care physicians_PCP Rate',
 'Primary care physicians_PCP Ratio',
 'Mental health providers^_# MHP',
 'Mental health providers^_MHP Rate',
 'Mental health

In [138]:
# Columns to be dropped
columns_to_drop = columns_to_drop = [
 'Demographics_Population',
 'Demographics_< 18',
 'Demographics_65 and over',
 'Demographics_Rural',
 'Diabetes_% Diabetic',
 'Food insecurity_% Food Insecure',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Uninsured adults_# Uninsured',
 'Median household income_Household Income',
 'Food environment index_Food Environment Index',
 'Physical inactivity_% Physically Inactive',
 'Access to exercise opportunities_% With Access',
 'Primary care physicians_# PCP',
 'Primary care physicians_PCP Rate',
 'Primary care physicians_PCP Ratio',
 'Mental health providers^_# MHP',
 'Mental health providers^_MHP Rate',
 'Mental health providers^_MHP Ratio',
 'Some college (post-secondary education)_# Some College',
 'Some college (post-secondary education)_Population',
 'Some college (post-secondary education)_% Some College',
 'Unemployment_# Unemployed',
 'Unemployment_Labor Force',
 'Severe housing problems_# Household with Severe Problems',
 'Severe housing problems_% Severe Housing Problems',
 'Quality of Life_Rank']

In [139]:
# Dropping columns in-place
chr14.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr14)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  42.0   
2    29003  Missouri          Andrew                   2.0   
3    29005  Missouri        Atchison                  13.0   
4    29007  Missouri         Audrain                  68.0   
5    29009  Missouri           Barry                  85.0   
6    29011  Missouri          Barton                  48.0   
7    29013  Missouri           Bates                  25.0   
8    29015  Missouri          Benton                  79.0   
9    29017  Missouri       Bollinger                  36.0   
10   29019  Missouri           Boone                   8.0   
11   29021  Missouri        Buchanan                  65.0   
12   29023  Missouri          Butler                 107.0   
13   29025  Missouri        Caldwell                  95.0   
14   29027  Missouri      

In [192]:
chr14.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Uninsured adults_% Uninsured',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Unemployment_% Unemployed',
 'Violent Crime Rate',
 'Health Behaviors_Rank',
 'Clinical Care_Rank',
 'Social & Economic Factors_Rank',
 'Physical Environment_Rank',
 'year']

In [191]:
chr14.rename(columns={'Violent crime_Violent Crime Rate': 'Violent Crime Rate'}, inplace=True)

In [77]:
data_13_1 = pd.read_excel('2013 County Health Ranking Missouri Data - v1_0.xls',sheet_name = 'Outcomes & Factors Rankings', header = [0,1])
data_13_2 = pd.read_excel('2013 County Health Ranking Missouri Data - v1_0.xls',sheet_name = 'Additional Measure Data', header = [0,1])
data_13_3 = pd.read_excel('2013 County Health Ranking Missouri Data - v1_0.xls',sheet_name = 'Ranked Measure Data', header = [0,1])
data_13_4 = pd.read_excel('2013 County Health Ranking Missouri Data - v1_0.xls',sheet_name = 'Outcomes & Factors SubRankings', header = [0,1])

In [78]:
data_13_1 = add_index_prefix(data_13_1)
data_13_2 = add_index_prefix(data_13_2)
data_13_3 = add_index_prefix(data_13_3)
data_13_4 = add_index_prefix(data_13_4)

display(data_13_1.columns,data_13_2.columns, data_13_3.columns , data_13_4.columns)

Index(['FIPS', 'State', 'County', 'Health Outcomes_Z-Score',
       'Health Outcomes_Rank', 'Health Factors_Z-Score',
       'Health Factors_Rank'],
      dtype='object')

Index(['FIPS', 'State', 'County', 'Demographics_Population',
       'Demographics_< 18', 'Demographics_65 and over',
       'Demographics_African American',
       'Demographics_American Indian/ Alaskan Native', 'Demographics_Asian',
       'Demographics_Native Hawaiian/ Other Pacific Islander',
       'Demographics_Hispanic', 'Demographics_Non-hispanic white',
       'Demographics_# not proficient in English',
       'Demographics_% not proficient in English', 'Demographics_Female',
       'Demographics_Rural', 'Diabetes_% diabetic', 'Diabetes_95% CI - Low',
       'Diabetes_95% CI - High', 'HIV prevalence rate_HIV Cases',
       'HIV prevalence rate_HIV Rate',
       'Premature age-adjusted mortality_Total deaths',
       'Premature age-adjusted mortality_Age-adjusted Mortality',
       'Premature age-adjusted mortality_95% CI - Low',
       'Premature age-adjusted mortality_95% CI - High',
       'Child mortality_Total deaths', 'Child mortality_Mortality rate',
       'Child mortali

Index(['FIPS', 'State', 'County',
       'Premature death (Years of Potential Life Lost)_Deaths',
       'Premature death (Years of Potential Life Lost)_YPLL Rate',
       'Premature death (Years of Potential Life Lost)_95% CI - Low',
       'Premature death (Years of Potential Life Lost)_95% CI - High',
       'Premature death (Years of Potential Life Lost)_Z-Score',
       'Poor or fair health_Sample Size', 'Poor or fair health_% Fair/Poor',
       ...
       'Drinking water safety_Z-Score',
       'Access to recreational facilities_# Rec Facilities',
       'Access to recreational facilities_Rec Facility Rate',
       'Access to recreational facilities_Z-Score',
       'Limited access to healthy foods*_# Limited Access',
       'Limited access to healthy foods*_% Limited Access',
       'Limited access to healthy foods*_Z-Score',
       'Fast food restaurants_# Fast Foods',
       'Fast food restaurants_% Fast Foods', 'Fast food restaurants_Z-Score'],
      dtype='object', length=13

Index(['FIPS', 'State', 'County', 'Mortality_Z-Score', 'Mortality_Rank',
       'Morbidity_Z-Score', 'Morbidity_Rank', 'Health Behaviors_Z-Score',
       'Health Behaviors_Rank', 'Clinical Care_Z-Score', 'Clinical Care_Rank',
       'Social & Economic Factors_Z-Score', 'Social & Economic Factors_Rank',
       'Physical Environment_Z-Score', 'Physical Environment_Rank'],
      dtype='object')

In [79]:
chr13 = pd.merge(data_13_1 , data_13_2 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr13 = pd.merge(chr13 , data_13_3 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr13 = pd.merge(chr13 , data_13_4 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr13

,FIPS,State,County,Health Outcomes_Z-Score,Health Outcomes_Rank,Health Factors_Z-Score,Health Factors_Rank,Demographics_Population,Demographics_< 18,Demographics_65 and over,...,Morbidity_Z-Score,Morbidity_Rank,Health Behaviors_Z-Score,Health Behaviors_Rank,Clinical Care_Z-Score,Clinical Care_Rank,Social & Economic Factors_Z-Score,Social & Economic Factors_Rank,Physical Environment_Z-Score,Physical Environment_Rank
0,29000,Missouri,NaN,NaN,NaN,NaN,NaN,6010688,23.493500,14.218871,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29001,Missouri,Adair,-0.331526,40.0,-0.446178,18.0,25552,18.378209,12.820914,...,0.182519,85.0,-0.182762,13.0,-0.138352,14.0,-0.145624,36.0,0.020560,82.0
2,29003,Missouri,Andrew,-1.272348,2.0,-0.456085,16.0,17196,23.389160,15.817632,...,-0.649036,1.0,-0.088781,33.0,-0.004293,54.0,-0.350483,8.0,-0.012527,46.0
3,29005,Missouri,Atchison,-0.946633,10.0,-0.543302,12.0,5569,20.650027,22.032681,...,-0.510264,5.0,-0.192274,12.0,0.070696,84.0,-0.352467,7.0,-0.069257,5.0
4,29007,Missouri,Audrain,0.002364,74.0,-0.135937,43.0,25566,24.802472,15.958695,...,0.163621,83.0,-0.026503,48.0,-0.075581,29.0,-0.066372,50.0,0.032519,93.0
5,29009,Missouri,Barry,0.303663,83.0,0.043162,59.0,35282,23.881866,18.131058,...,0.204784,86.0,0.119362,93.0,-0.036922,42.0,0.015585,66.0,-0.054863,13.0
6,29011,Missouri,Barton,-0.362403,35.0,0.108167,67.0,12327,25.634785,17.311592,...,-0.115791,50.0,0.027070,70.0,0.026077,66.0,0.046687,69.0,0.008333,69.0
7,29013,Missouri,Bates,-0.445573,27.0,0.231219,83.0,17008,24.717780,17.791627,...,-0.209314,33.0,-0.020475,53.0,0.228078,111.0,0.088077,77.0,-0.064460,9.0
8,29015,Missouri,Benton,0.149078,79.0,0.130671,69.0,19115,17.358096,26.717238,...,-0.158985,44.0,-0.014653,57.0,0.074236,87.0,0.144271,89.0,-0.073184,4.0
9,29017,Missouri,Bollinger,-0.073487,66.0,0.131494,70.0,12356,23.195209,16.558757,...,0.234590,90.0,0.155332,103.0,-0.003015,56.0,-0.026501,61.0,0.005678,64.0


In [80]:
chr13['year'] = 2013

In [81]:
chr13.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Z-Score',
 'Health Outcomes_Rank',
 'Health Factors_Z-Score',
 'Health Factors_Rank',
 'Demographics_Population',
 'Demographics_< 18',
 'Demographics_65 and over',
 'Demographics_African American',
 'Demographics_American Indian/ Alaskan Native',
 'Demographics_Asian',
 'Demographics_Native Hawaiian/ Other Pacific Islander',
 'Demographics_Hispanic',
 'Demographics_Non-hispanic white',
 'Demographics_# not proficient in English',
 'Demographics_% not proficient in English',
 'Demographics_Female',
 'Demographics_Rural',
 'Diabetes_% diabetic',
 'Diabetes_95% CI - Low',
 'Diabetes_95% CI - High',
 'HIV prevalence rate_HIV Cases',
 'HIV prevalence rate_HIV Rate',
 'Premature age-adjusted mortality_Total deaths',
 'Premature age-adjusted mortality_Age-adjusted Mortality',
 'Premature age-adjusted mortality_95% CI - Low',
 'Premature age-adjusted mortality_95% CI - High',
 'Child mortality_Total deaths',
 'Child mortality_Mortality rate',
 '

In [83]:
# Columns to be dropped
columns_to_drop = [
 'Health Outcomes_Z-Score',
 'Health Factors_Z-Score',
 'Demographics_African American',
 'Demographics_American Indian/ Alaskan Native',
 'Demographics_Asian',
 'Demographics_Native Hawaiian/ Other Pacific Islander',
 'Demographics_Hispanic',
 'Demographics_Non-hispanic white',
 'Demographics_# not proficient in English',
 'Demographics_% not proficient in English',
 'Demographics_Female',
 'Diabetes_95% CI - Low',
 'Diabetes_95% CI - High',
 'HIV prevalence rate_HIV Cases',
 'HIV prevalence rate_HIV Rate',
 'Premature age-adjusted mortality_Total deaths',
 'Premature age-adjusted mortality_Age-adjusted Mortality',
 'Premature age-adjusted mortality_95% CI - Low',
 'Premature age-adjusted mortality_95% CI - High',
 'Child mortality_Total deaths',
 'Child mortality_Mortality rate',
 'Child mortality_95% CI - Low',
 'Child mortality_95% CI - High',
 'Infant mortality_Total deaths',
 'Infant mortality_Mortality rate',
 'Infant mortality_95% CI - Low',
 'Infant mortality_95% CI - High',
 'Health care costs_Costs',
 'Uninsured adults_95% CI - Low',
 'Uninsured adults_95% CI - High',
 'Uninsured children_# Uninsured',
 'Uninsured children_% Uninsured',
 'Uninsured children_95% CI - Low',
 'Uninsured children_95% CI - High',
 'Could not see doctor due to cost_Sample Size',
 "Could not see doctor due to cost_% Couldn't Access",
 'Median household income_95% CI - Low',
 'Median household income_95% CI - High',
 'High housing costs_# high housing costs',
 'High housing costs_Households',
 'High housing costs_% high housing costs',
 'Children eligible for free lunch_% Free lunch',
 'Homicide rate_Homicide Rate',
 'Homicide rate_95% CI - Low',
 'Homicide rate_95% CI - High',
 'Commuting alone_# Drive Alone',
 'Commuting alone_Workers',
 'Commuting alone_% Drive Alone',
 'Access to Parks_% park access',
 'Premature death (Years of Potential Life Lost)_Deaths',
 'Premature death (Years of Potential Life Lost)_YPLL Rate',
 'Premature death (Years of Potential Life Lost)_95% CI - Low',
 'Premature death (Years of Potential Life Lost)_95% CI - High',
 'Premature death (Years of Potential Life Lost)_Z-Score',
 'Poor or fair health_Sample Size',
 'Poor or fair health_95% CI - Low',
 'Poor or fair health_95% CI - High',
 'Poor or fair health_Z-Score',
 'Poor physical health days_Sample Size',
 'Poor physical health days_95% CI - Low',
 'Poor physical health days_95% CI - High',
 'Poor physical health days_Z-Score',
 'Poor mental health days_Sample Size',
 'Poor mental health days_95% CI - Low',
 'Poor mental health days_95% CI - High',
 'Poor mental health days_Z-Score',
 'Low birthweight_Unreliable',
 'Low birthweight_LBW Births',
 'Low birthweight_Live Births',
 'Low birthweight_% LBW',
 'Low birthweight_95% CI - Low',
 'Low birthweight_95% CI - High',
 'Low birthweight_Z-Score',
 'Adult smoking_Sample Size',
 'Adult smoking_95% CI - Low',
 'Adult smoking_95% CI - High',
 'Adult smoking_Z-Score',
 'Adult obesity_95% CI - Low',
 'Adult obesity_95% CI - High',
 'Adult obesity_Z-Score',
 'Physical inactivity_95% CI - Low',
 'Physical inactivity_95% CI - High',
 'Physical inactivity_Z-Score',
 'Excessive drinking_Sample Size',
 'Excessive drinking_% Excessive Drinking',
 'Excessive drinking_95% CI - Low',
 'Excessive drinking_95% CI - High',
 'Excessive drinking_Z-Score',
 'Motor vehicle crash death rate_Motor Vehicle Deaths',
 'Motor vehicle crash death rate_MV Mortality Rate',
 'Motor vehicle crash death rate_95% CI - Low',
 'Motor vehicle crash death rate_95% CI - High',
 'Motor vehicle crash death rate_Z-Score',
 'Sexually transmitted infections_Chlamydia Cases',
 'Sexually transmitted infections_Chlamydia Rate',
 'Sexually transmitted infections_Z-Score',
 'Teen birth rate_Teen Births ',
 'Teen birth rate_Teen Population',
 'Teen birth rate_Teen Birth Rate',
 'Teen birth rate_95% CI - Low',
 'Teen birth rate_95% CI - High',
 'Teen birth rate_Z-Score',
 'Uninsured_95% CI - Low',
 'Uninsured_95% CI - High',
 'Uninsured_Z-Score',
 'Primary care physicians*_Z-Score',
 'Dentists*_# Dentists',
 'Dentists*_Dentist Rate',
 'Dentists*_Dentist Ratio',
 'Dentists*_Z-Score',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_# Medicare enrollees',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_ACSC Rate',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_95% CI - Low',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_95% CI - High',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_Z-Score',
 'Diabetic screening_# Diabetics',
 'Diabetic screening_% HbA1c',
 'Diabetic screening_95% CI - Low',
 'Diabetic screening_95% CI - High',
 'Diabetic screening_Z-Score',
 'Mammography screening_# Medicare Enrollees',
 'Mammography screening_% Mammography',
 'Mammography screening_95% CI - Low',
 'Mammography screening_95% CI - High',
 'Mammography screening_Z-Score',
 'High school graduation*_AFGR',
 'High school graduation*_Z-Score',
 'Some college (post-secondary education)_95% CI - Low',
 'Some college (post-secondary education)_95% CI - High',
 'Some college (post-secondary education)_Z-Score',
 'Unemployment_Z-Score',
 'Children in poverty_# Children in Poverty',
 'Children in poverty_% Children in Poverty',
 'Children in poverty_95% CI - Low',
 'Children in poverty_95% CI - High',
 'Children in poverty_Z-Score',
 'Inadequate social support_Sample Size',
 'Inadequate social support_% No Social-Emotional Support',
 'Inadequate social support_95% CI - Low',
 'Inadequate social support_95% CI - High',
 'Inadequate social support_Z-Score',
 'Children in single-parent households_# Single-Parent Households',
 'Children in single-parent households_# Households',
 'Children in single-parent households_% Single-Parent Households',
 'Children in single-parent households_95% CI - Low',
 'Children in single-parent households_95% CI - High',
 'Children in single-parent households_Z-Score',
 'Violent crime rate_Annual Violent Crimes',
 'Violent crime rate_Z-Score',
 'Daily fine particulate matter_Average daily PM25',
 'Daily fine particulate matter_Z-Score',
 'Drinking water safety_% pop in viol',
 'Drinking water safety_Pop. In Viol',
 'Drinking water safety_Z-Score',
 'Access to recreational facilities_# Rec Facilities',
 'Access to recreational facilities_Rec Facility Rate',
 'Access to recreational facilities_Z-Score',
 'Limited access to healthy foods*_Z-Score',
 'Fast food restaurants_# Fast Foods',
 'Fast food restaurants_% Fast Foods',
 'Fast food restaurants_Z-Score',
 'Mortality_Z-Score',
 'Mortality_Rank',
 'Morbidity_Z-Score',
 'Morbidity_Rank',
 'Health Behaviors_Z-Score',
 'Clinical Care_Z-Score',
 'Social & Economic Factors_Z-Score',
 'Physical Environment_Z-Score']

In [84]:
# Dropping columns in-place
chr13.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr13)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  40.0   
2    29003  Missouri          Andrew                   2.0   
3    29005  Missouri        Atchison                  10.0   
4    29007  Missouri         Audrain                  74.0   
5    29009  Missouri           Barry                  83.0   
6    29011  Missouri          Barton                  35.0   
7    29013  Missouri           Bates                  27.0   
8    29015  Missouri          Benton                  79.0   
9    29017  Missouri       Bollinger                  66.0   
10   29019  Missouri           Boone                   6.0   
11   29021  Missouri        Buchanan                  70.0   
12   29023  Missouri          Butler                 109.0   
13   29025  Missouri        Caldwell                  86.0   
14   29027  Missouri      

In [85]:
chr13.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Demographics_Population',
 'Demographics_< 18',
 'Demographics_65 and over',
 'Demographics_Rural',
 'Diabetes_% diabetic',
 'Mental health providers_# MHP',
 'Mental health providers_MHP Rate',
 'Mental health providers_MHP Ratio',
 'Uninsured adults_# Uninsured',
 'Uninsured adults_% Uninsured',
 'Median household income_Household Income',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Physical inactivity_% Physically Inactive',
 'Uninsured_# Uninsured',
 'Uninsured_% Uninsured',
 'Primary care physicians*_# PCP',
 'Primary care physicians*_PCP Rate',
 'Primary care physicians*_PCP Ratio',
 'Some college (post-secondary education)_# Some College',
 'Some college (post-secondary education)_Population',
 'Some college (post-secondary education)_% Some Coll

In [141]:
# Columns to be dropped
columns_to_drop = columns_to_drop = [
 'Demographics_Population',
 'Demographics_< 18',
 'Demographics_65 and over',
 'Demographics_Rural',
 'Diabetes_% diabetic',
 'Mental health providers_# MHP',
 'Mental health providers_MHP Rate',
 'Mental health providers_MHP Ratio',
 'Uninsured adults_# Uninsured',
 'Median household income_Household Income',
 'Physical inactivity_% Physically Inactive',
 'Uninsured_# Uninsured',
 'Uninsured_% Uninsured',
 'Primary care physicians*_# PCP',
 'Primary care physicians*_PCP Rate',
 'Primary care physicians*_PCP Ratio',
 'Some college (post-secondary education)_# Some College',
 'Some college (post-secondary education)_Population',
 'Some college (post-secondary education)_% Some College',
 'Unemployment_# Unemployed',
 'Unemployment_Labor Force',
 'Limited access to healthy foods*_# Limited Access',
 'Limited access to healthy foods*_% Limited Access']

In [142]:
# Dropping columns in-place
chr13.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr13)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  40.0   
2    29003  Missouri          Andrew                   2.0   
3    29005  Missouri        Atchison                  10.0   
4    29007  Missouri         Audrain                  74.0   
5    29009  Missouri           Barry                  83.0   
6    29011  Missouri          Barton                  35.0   
7    29013  Missouri           Bates                  27.0   
8    29015  Missouri          Benton                  79.0   
9    29017  Missouri       Bollinger                  66.0   
10   29019  Missouri           Boone                   6.0   
11   29021  Missouri        Buchanan                  70.0   
12   29023  Missouri          Butler                 109.0   
13   29025  Missouri        Caldwell                  86.0   
14   29027  Missouri      

In [190]:
chr13.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Uninsured adults_% Uninsured',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Unemployment_% Unemployed',
 'Violent Crime Rate',
 'Health Behaviors_Rank',
 'Clinical Care_Rank',
 'Social & Economic Factors_Rank',
 'Physical Environment_Rank',
 'year']

In [189]:
chr13.rename(columns={'Violent crime rate_Violent Crime Rate': 'Violent Crime Rate'}, inplace=True)

In [86]:
data_12_1 = pd.read_excel('2012 County Health Ranking Missouri Data - v4.xls',sheet_name = 'Outcomes & Factors Rankings', header = [0,1])
data_12_2 = pd.read_excel('2012 County Health Ranking Missouri Data - v4.xls',sheet_name = 'Additional Measure Data', header = [0,1])
data_12_3 = pd.read_excel('2012 County Health Ranking Missouri Data - v4.xls',sheet_name = 'Ranked Measure Data', header = [0,1])
data_12_4 = pd.read_excel('2012 County Health Ranking Missouri Data - v4.xls',sheet_name = 'Outcomes & Factors SubRankings', header = [0,1])

In [87]:
data_12_1 = add_index_prefix(data_12_1)
data_12_2 = add_index_prefix(data_12_2)
data_12_3 = add_index_prefix(data_12_3)
data_12_4 = add_index_prefix(data_12_4)

display(data_12_1.columns,data_12_2.columns, data_12_3.columns , data_12_4.columns)

Index(['FIPS', 'State', 'County', 'Health Outcomes_Z-Score',
       'Health Outcomes_Rank', 'Health Factors_Z-Score',
       'Health Factors_Rank'],
      dtype='object')

Index(['FIPS', 'State', 'County', 'Demographics_Population',
       'Demographics_< 18', 'Demographics_65 and over',
       'Demographics_African American',
       'Demographics_American Indian/ Alaskan Native', 'Demographics_Asian',
       'Demographics_Native Hawaiian/ Other Pacific Islander',
       'Demographics_Hispanic', 'Demographics_Population.1',
       'Demographics_# not proficient in English',
       'Demographics_% not proficient in English', 'Demographics_Female',
       'Demographics_Rural', 'Diabetes_% diabetic', 'Diabetes_95% CI - Low',
       'Diabetes_95% CI - High', 'HIV prevalence rate_Population',
       'HIV prevalence rate_HIV Rate', 'Mental health providers_# MPH',
       'Mental health providers_MPH Rate', 'Mental health providers_MPH Ratio',
       'Health care costs_Costs', 'Uninsured adults_# Uninsured',
       'Uninsured adults_% Uninsured', 'Uninsured adults_95% CI - Low',
       'Uninsured adults_95% CI - High',
       'Could not see doctor due to cost_S

Index(['FIPS', 'State', 'County',
       'Premature death (Years of Potential Life Lost)_Unreliable',
       'Premature death (Years of Potential Life Lost)_Deaths',
       'Premature death (Years of Potential Life Lost)_YPLL Rate',
       'Premature death (Years of Potential Life Lost)_95% CI - Low',
       'Premature death (Years of Potential Life Lost)_95% CI - High',
       'Premature death (Years of Potential Life Lost)_Z-Score',
       'Poor or fair health_Sample Size',
       ...
       'Limited access to healthy foods_# Limited Access',
       'Limited access to healthy foods_% Limited Access',
       'Limited access to healthy foods_Z-Score',
       'Access to healthy foods_Zip Codes with Healthy Foods',
       'Access to healthy foods_# Zip Codes',
       'Access to healthy foods_% Healthy Foods',
       'Access to healthy foods_Z-Score', 'Fast food restaurants_# Fast Foods',
       'Fast food restaurants_% Fast Foods', 'Fast food restaurants_Z-Score'],
      dtype='object', 

Index(['FIPS', 'State', 'County', 'Mortality_Z-Score', 'Mortality_Rank',
       'Morbidity_Z-Score', 'Morbidity_Rank', 'Health Behaviors_Z-Score',
       'Health Behaviors_Rank', 'Clinical Care_Z-Score', 'Clinical Care_Rank',
       'Social & Economic Factors_Z-Score', 'Social & Economic Factors_Rank',
       'Physical Environment_Z-Score', 'Physical Environment_Rank'],
      dtype='object')

In [88]:
chr12 = pd.merge(data_12_1 , data_12_2 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr12 = pd.merge(chr12 , data_12_3 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr12 = pd.merge(chr12 , data_12_4 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr12

,FIPS,State,County,Health Outcomes_Z-Score,Health Outcomes_Rank,Health Factors_Z-Score,Health Factors_Rank,Demographics_Population,Demographics_< 18,Demographics_65 and over,...,Morbidity_Z-Score,Morbidity_Rank,Health Behaviors_Z-Score,Health Behaviors_Rank,Clinical Care_Z-Score,Clinical Care_Rank,Social & Economic Factors_Z-Score,Social & Economic Factors_Rank,Physical Environment_Z-Score,Physical Environment_Rank
0,29000,Missouri,NaN,NaN,NaN,NaN,NaN,5987580,23.9,13.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29001,Missouri,Adair,-0.158460,58.0,-0.480600,14.0,25135,18.9,12.3,...,0.144961,84.0,-0.210956,12.0,-0.024028,48.0,-0.241226,17.0,-0.004391,56.0
2,29003,Missouri,Andrew,-0.819723,13.0,-0.499769,12.0,17052,24.1,14.0,...,-0.503085,5.0,-0.084653,33.0,-0.074731,29.0,-0.375168,5.0,0.034783,93.0
3,29005,Missouri,Atchison,-1.209557,4.0,-0.426363,16.0,6036,22.4,20.3,...,-0.306924,22.0,-0.133830,21.0,0.126832,102.0,-0.330971,9.0,-0.088393,3.0
4,29007,Missouri,Audrain,-0.349703,43.0,0.019145,59.0,25556,25.1,15.6,...,0.055355,73.0,0.087095,85.0,-0.055832,36.0,0.003966,63.0,-0.016084,42.0
5,29009,Missouri,Barry,0.607363,96.0,-0.055039,52.0,35881,26.1,16.2,...,0.313861,98.0,0.075664,83.0,-0.099939,21.0,0.017022,65.0,-0.047786,15.0
6,29011,Missouri,Barton,-0.300282,46.0,0.256045,88.0,12386,25.3,16.5,...,-0.222254,34.0,0.039509,71.0,0.077304,87.0,0.119996,84.0,0.019236,82.0
7,29013,Missouri,Bates,-0.472819,31.0,0.269435,92.0,16761,24.5,16.9,...,-0.404125,13.0,-0.049713,42.0,0.210847,110.0,0.096736,78.0,0.011565,75.0
8,29015,Missouri,Benton,0.099803,74.0,0.266092,91.0,18461,18.4,25.6,...,-0.161199,43.0,-0.047244,43.0,0.065546,82.0,0.282587,102.0,-0.034798,22.0
9,29017,Missouri,Bollinger,0.037523,70.0,0.255160,87.0,11841,23.3,15.3,...,0.216114,92.0,0.145724,97.0,0.089455,90.0,0.023658,67.0,-0.003677,58.0


In [89]:
chr12['year'] = 2012

In [90]:
chr12.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Z-Score',
 'Health Outcomes_Rank',
 'Health Factors_Z-Score',
 'Health Factors_Rank',
 'Demographics_Population',
 'Demographics_< 18',
 'Demographics_65 and over',
 'Demographics_African American',
 'Demographics_American Indian/ Alaskan Native',
 'Demographics_Asian',
 'Demographics_Native Hawaiian/ Other Pacific Islander',
 'Demographics_Hispanic',
 'Demographics_Population.1',
 'Demographics_# not proficient in English',
 'Demographics_% not proficient in English',
 'Demographics_Female',
 'Demographics_Rural',
 'Diabetes_% diabetic',
 'Diabetes_95% CI - Low',
 'Diabetes_95% CI - High',
 'HIV prevalence rate_Population',
 'HIV prevalence rate_HIV Rate',
 'Mental health providers_# MPH',
 'Mental health providers_MPH Rate',
 'Mental health providers_MPH Ratio',
 'Health care costs_Costs',
 'Uninsured adults_# Uninsured',
 'Uninsured adults_% Uninsured',
 'Uninsured adults_95% CI - Low',
 'Uninsured adults_95% CI - High',
 'Could not se

In [91]:
# Columns to be dropped
columns_to_drop = [
 'Health Outcomes_Z-Score',
 'Health Factors_Z-Score',
 'Demographics_African American',
 'Demographics_American Indian/ Alaskan Native',
 'Demographics_Asian',
 'Demographics_Native Hawaiian/ Other Pacific Islander',
 'Demographics_Hispanic',
 'Demographics_Population.1',
 'Demographics_# not proficient in English',
 'Demographics_% not proficient in English',
 'Demographics_Female',
 'Diabetes_95% CI - Low',
 'Diabetes_95% CI - High',
 'HIV prevalence rate_Population',
 'HIV prevalence rate_HIV Rate',
 'Health care costs_Costs',
 'Uninsured adults_95% CI - Low',
 'Uninsured adults_95% CI - High',
 'Could not see doctor due to cost_Sample Size',
 "Could not see doctor due to cost_% Couldn't Access",
 'Dentists_# Dentists',
 'Dentists_Dentist Rate',
 'Dentists_Dentist Ratio',
 'Median household income_95% CI - Low',
 'Median household income_95% CI - High',
 'High housing costs_# high housing costs',
 'High housing costs_Households',
 'High housing costs_% high housing costs',
 'Children eligible for free lunch_% Free lunch',
 'Illiteracy_% illiterate',
 'Illiteracy_95% CI - Low',
 'Illiteracy_95% CI - High',
 'Homicide rate_Homicide Rate',
 'Homicide rate_95% CI - Low',
 'Homicide rate_95% CI - High',
 'Commuting alone_# Drive Alone',
 'Commuting alone_Workers',
 'Commuting alone_% Drive Alone',
 'Access to healthy foods_Zip Codes with Healthy Foods_x',
 'Access to healthy foods_# Zip Codes_x',
 'Access to healthy foods_% Healthy Foods_x',
 'Premature death (Years of Potential Life Lost)_Unreliable',
 'Premature death (Years of Potential Life Lost)_Deaths',
 'Premature death (Years of Potential Life Lost)_YPLL Rate',
 'Premature death (Years of Potential Life Lost)_95% CI - Low',
 'Premature death (Years of Potential Life Lost)_95% CI - High',
 'Premature death (Years of Potential Life Lost)_Z-Score',
 'Poor or fair health_Sample Size',
 'Poor or fair health_95% CI - Low',
 'Poor or fair health_95% CI - High',
 'Poor or fair health_Z-Score',
 'Poor physical health days_Sample Size',
 'Poor physical health days_95% CI - Low',
 'Poor physical health days_95% CI - High',
 'Poor physical health days_Z-Score',
 'Poor mental health days_Sample Size',
 'Poor mental health days_95% CI - Low',
 'Poor mental health days_95% CI - High',
 'Poor mental health days_Z-Score',
 'Low birthweight_Unreliable',
 'Low birthweight_LBW Births',
 'Low birthweight_Sample Size',
 'Low birthweight_% LBW',
 'Low birthweight_95% CI - Low',
 'Low birthweight_95% CI - High',
 'Low birthweight_Z-Score',
 'Adult smoking_Sample Size',
 'Adult smoking_95% CI - Low',
 'Adult smoking_95% CI - High',
 'Adult smoking_Z-Score',
 'Adult obesity_95% CI - Low',
 'Adult obesity_95% CI - High',
 'Adult obesity_Z-Score',
 'Physical inactivity_95% CI - Low',
 'Physical inactivity_95% CI - High',
 'Physical inactivity_Z-Score',
 'Excessive drinking_Sample Size',
 'Excessive drinking_% Excessive Drinking',
 'Excessive drinking_95% CI - Low',
 'Excessive drinking_95% CI - High',
 'Excessive drinking_Z-Score',
 'Motor vehicle crash death rate_Unreliable',
 'Motor vehicle crash death rate_Motor Vehicle Deaths',
 'Motor vehicle crash death rate_MV Mortality Rate',
 'Motor vehicle crash death rate_95% CI - Low',
 'Motor vehicle crash death rate_95% CI - High',
 'Motor vehicle crash death rate_Z-Score',
 'Sexually transmitted infections_Cases',
 'Sexually transmitted infections_Rates per 100,000',
 'Sexually transmitted infections_Z-Score',
 'Teen birth rate_Teen Births ',
 'Teen birth rate_Teen Population',
 'Teen birth rate_Teen Birth Rate',
 'Teen birth rate_95% CI - Low',
 'Teen birth rate_95% CI - High',
 'Teen birth rate_Z-Score',
 'Uninsured_95% CI - Low',
 'Uninsured_95% CI - High',
 'Uninsured_Z-Score',
 'Previous Primary care physician data--used to calculate rankings*_# PCP',
 'Previous Primary care physician data--used to calculate rankings*_PCP Rate',
 'Previous Primary care physician data--used to calculate rankings*_PCP Ratio',
 'Previous Primary care physician data--used to calculate rankings*_Z-Score',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_# Medicare enrollees',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_ACSC Rate',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_95% CI - Low',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_95% CI - High',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_Z-Score',
 'Diabetic screening_# Diabetics',
 'Diabetic screening_% HbA1c',
 'Diabetic screening_95% CI - Low',
 'Diabetic screening_95% CI - High',
 'Diabetic screening_Z-Score',
 'Mammography screening_# Medicare Enrollees',
 'Mammography screening_Mammography Rate',
 'Mammography screening_95% CI - Low',
 'Mammography screening_95% CI - High',
 'Mammography screening_Z-Score',
 'High school graduation_AFGR',
 'High school graduation_Z-Score',
 'Some college (post-secondary education)_PSED Num',
 'Some college (post-secondary education)_Population',
 'Some college (post-secondary education)_PSED',
 'Some college (post-secondary education)_95% CI - Low',
 'Some college (post-secondary education)_95% CI - High',
 'Some college (post-secondary education)_Z-Score',
 'Unemployment_Z-Score',
 'Children in poverty_# Children in Poverty',
 'Children in poverty_% Children in Poverty',
 'Children in poverty_95% CI - Low',
 'Children in poverty_95% CI - High',
 'Children in poverty_Z-Score',
 'Inadequate social support_Sample Size',
 'Inadequate social support_% No Social-Emotional Support',
 'Inadequate social support_95% CI - Low',
 'Inadequate social support_95% CI - High',
 'Inadequate social support_Z-Score',
 'Children in single-parent households_# Single-Parent Households',
 'Children in single-parent households_# Households',
 'Children in single-parent households_% Single-Parent Households',
 'Children in single-parent households_95% CI - Low',
 'Children in single-parent households_95% CI - High',
 'Children in single-parent households_Z-Score',
 'Violent crime rate_Violent Crimes',
 'Violent crime rate_Z-Score',
 'Air pollution-particulate matter days_PM Days',
 'Air pollution-particulate matter days_Z-Score',
 'Air pollution-ozone days_Ozone Days',
 'Air pollution-ozone days_Z-Score',
 'Access to recreational facilities_Rec Facs',
 'Access to recreational facilities_Rec Fac Rate',
 'Access to recreational facilities_Z-Score',
 'Limited access to healthy foods_Z-Score',
 'Access to healthy foods_Zip Codes with Healthy Foods_y',
 'Access to healthy foods_# Zip Codes_y',
 'Access to healthy foods_Z-Score',
 'Fast food restaurants_# Fast Foods',
 'Fast food restaurants_% Fast Foods',
 'Fast food restaurants_Z-Score',
 'Mortality_Z-Score',
 'Mortality_Rank',
 'Morbidity_Z-Score',
 'Morbidity_Rank',
 'Health Behaviors_Z-Score',
 'Clinical Care_Z-Score',
 'Social & Economic Factors_Z-Score',
 'Physical Environment_Z-Score']

In [92]:
# Dropping columns in-place
chr12.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr12)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  58.0   
2    29003  Missouri          Andrew                  13.0   
3    29005  Missouri        Atchison                   4.0   
4    29007  Missouri         Audrain                  43.0   
5    29009  Missouri           Barry                  96.0   
6    29011  Missouri          Barton                  46.0   
7    29013  Missouri           Bates                  31.0   
8    29015  Missouri          Benton                  74.0   
9    29017  Missouri       Bollinger                  70.0   
10   29019  Missouri           Boone                   9.0   
11   29021  Missouri        Buchanan                  62.0   
12   29023  Missouri          Butler                 110.0   
13   29025  Missouri        Caldwell                  69.0   
14   29027  Missouri      

In [93]:
chr12.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Demographics_Population',
 'Demographics_< 18',
 'Demographics_65 and over',
 'Demographics_Rural',
 'Diabetes_% diabetic',
 'Mental health providers_# MPH',
 'Mental health providers_MPH Rate',
 'Mental health providers_MPH Ratio',
 'Uninsured adults_# Uninsured',
 'Uninsured adults_% Uninsured',
 'Median household income_Household Income',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Physical inactivity_Physical Inactivity',
 'Uninsured_# Uninsured',
 'Uninsured_% Uninsured',
 'Primary care physicians*_# PCP',
 'Primary care physicians*_PCP Rate',
 'Primary care physicians*_PCP Ratio',
 'Unemployment_# Unemployed',
 'Unemployment_Labor Force',
 'Unemployment_% Unemployed',
 'Violent crime rate_Violent Crime Rate',
 'Limited access to healthy foods_# Li

In [144]:
# Columns to be dropped
columns_to_drop = columns_to_drop = [
 'Demographics_Population',
 'Demographics_< 18',
 'Demographics_65 and over',
 'Demographics_Rural',
 'Diabetes_% diabetic',
 'Mental health providers_# MPH',
 'Mental health providers_MPH Rate',
 'Mental health providers_MPH Ratio',
 'Uninsured adults_# Uninsured',
 'Median household income_Household Income',
 'Physical inactivity_Physical Inactivity',
 'Uninsured_# Uninsured',
 'Uninsured_% Uninsured',
 'Primary care physicians*_# PCP',
 'Primary care physicians*_PCP Rate',
 'Primary care physicians*_PCP Ratio',
 'Unemployment_# Unemployed',
 'Unemployment_Labor Force',
 'Limited access to healthy foods_# Limited Access',
 'Limited access to healthy foods_% Limited Access',
 'Access to healthy foods_% Healthy Foods_y']

In [145]:
# Dropping columns in-place
chr12.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr12)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  58.0   
2    29003  Missouri          Andrew                  13.0   
3    29005  Missouri        Atchison                   4.0   
4    29007  Missouri         Audrain                  43.0   
5    29009  Missouri           Barry                  96.0   
6    29011  Missouri          Barton                  46.0   
7    29013  Missouri           Bates                  31.0   
8    29015  Missouri          Benton                  74.0   
9    29017  Missouri       Bollinger                  70.0   
10   29019  Missouri           Boone                   9.0   
11   29021  Missouri        Buchanan                  62.0   
12   29023  Missouri          Butler                 110.0   
13   29025  Missouri        Caldwell                  69.0   
14   29027  Missouri      

In [188]:
chr12.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Uninsured adults_% Uninsured',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Unemployment_% Unemployed',
 'Violent Crime Rate',
 'Health Behaviors_Rank',
 'Clinical Care_Rank',
 'Social & Economic Factors_Rank',
 'Physical Environment_Rank',
 'year']

In [187]:
chr12.rename(columns={'Violent crime rate_Violent Crime Rate': 'Violent Crime Rate'}, inplace=True)

In [94]:
data_11_1 = pd.read_excel('2011 County Health Ranking Missouri Data - v4.xls',sheet_name = 'Outcomes & Factors Rankings', header = [0,1])
data_11_2 = pd.read_excel('2011 County Health Ranking Missouri Data - v4.xls',sheet_name = 'Additional Measure Data', header = [0,1])
data_11_3 = pd.read_excel('2011 County Health Ranking Missouri Data - v4.xls',sheet_name = 'Ranked Measure Data', header = [0,1])
data_11_4 = pd.read_excel('2011 County Health Ranking Missouri Data - v4.xls',sheet_name = 'Outcomes & Factors SubRankings', header = [0,1])

In [95]:
data_11_1 = add_index_prefix(data_11_1)
data_11_2 = add_index_prefix(data_11_2)
data_11_3 = add_index_prefix(data_11_3)
data_11_4 = add_index_prefix(data_11_4)

display(data_11_1.columns,data_11_2.columns, data_11_3.columns , data_11_4.columns)

Index(['FIPS', 'State', 'County', 'Health Outcomes_Z-Score',
       'Health Outcomes_Rank', 'Health Factors_Z-Score',
       'Health Factors_Rank'],
      dtype='object')

Index(['FIPS', 'State', 'County', 'Demographics_Population',
       'Demographics_< 18', 'Demographics_65 and over',
       'Demographics_African American',
       'Demographics_American Indian/ Alaskan Native', 'Demographics_Asian',
       'Demographics_Native Hawaiian/ Other Pacific Islander',
       'Demographics_Hispanic', 'Demographics_Population.1',
       'Demographics_# not proficient in English',
       'Demographics_% not proficient in English', 'Demographics_Female',
       'Demographics_Rural', '% diabetic_Diabetes', '% diabetic_95% CI - Low',
       '% diabetic_95% CI - High', 'HIV rate_Population', 'HIV rate_HIV rate',
       'Binge drinking_Sample Size', 'Binge drinking_% Binge Drinking',
       'Binge drinking_95% CI - Low', 'Binge drinking_95% CI - High',
       'Physical inactivity_Physical Inactivity',
       'Physical inactivity_95% CI - Low', 'Physical inactivity_95% CI - High',
       'Mental health providers_MPH No.', 'Mental health providers_Population',
       

Index(['FIPS', 'State', 'County',
       'Premature death (Years of Potential Life Lost)_Unreliable',
       'Premature death (Years of Potential Life Lost)_Deaths',
       'Premature death (Years of Potential Life Lost)_YPLL Rate',
       'Premature death (Years of Potential Life Lost)_95% CI - Low',
       'Premature death (Years of Potential Life Lost)_95% CI - High',
       'Premature death (Years of Potential Life Lost)_Z-Score',
       'Poor or fair health_Sample Size',
       ...
       'Air pollution-ozone days_Z-Score',
       'Access to healthy foods_Unreliable',
       'Access to healthy foods_Zip Codes with Healthy Food',
       'Access to healthy foods_# Zip Codes',
       'Access to healthy foods_% Healthy Food',
       'Access to healthy foods_Z-Score',
       'Access to recreational facilities_Rec Facs',
       'Access to recreational facilities_Population',
       'Access to recreational facilities_Rec Fac Rate',
       'Access to recreational facilities_Z-Score'],
   

Index(['FIPS', 'State', 'County', 'Mortality_Z-Score', 'Mortality_Rank',
       'Morbidity_Z-Score', 'Morbidity_Rank', 'Health Behaviors_Z-Score',
       'Health Behaviors_Rank', 'Clinical Care_Z-Score', 'Clinical Care_Rank',
       'Social & Economic Factors_Z-Score', 'Social & Economic Factors_Rank',
       'Physical Environment_Z-Score', 'Physical Environment_Rank'],
      dtype='object')

In [96]:
chr11 = pd.merge(data_11_1 , data_11_2 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr11 = pd.merge(chr11 , data_11_3 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr11 = pd.merge(chr11 , data_11_4 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr11

,FIPS,State,County,Health Outcomes_Z-Score,Health Outcomes_Rank,Health Factors_Z-Score,Health Factors_Rank,Demographics_Population,Demographics_< 18,Demographics_65 and over,...,Morbidity_Z-Score,Morbidity_Rank,Health Behaviors_Z-Score,Health Behaviors_Rank,Clinical Care_Z-Score,Clinical Care_Rank,Social & Economic Factors_Z-Score,Social & Economic Factors_Rank,Physical Environment_Z-Score,Physical Environment_Rank
0,29000,Missouri,NaN,NaN,NaN,NaN,NaN,5987580,23.9,13.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29001,Missouri,Adair,-0.073578,60,-0.435384,14,25135,18.9,12.3,...,0.030213,67,-0.266131,8,0.079340,84,-0.274994,13,0.026400,90
2,29003,Missouri,Andrew,-0.704369,19,-0.479694,11,17052,24.1,14.0,...,-0.534357,5,-0.019644,57,-0.108264,22,-0.373524,7,0.021738,82
3,29005,Missouri,Atchison,-1.180373,5,-0.237768,32,6036,22.4,20.3,...,-0.407813,14,-0.143334,22,0.230884,112,-0.221686,20,-0.103632,1
4,29007,Missouri,Audrain,-0.261647,45,-0.167915,38,25556,25.1,15.6,...,0.159157,86,0.002378,63,-0.112072,21,-0.041552,54,-0.016668,39
5,29009,Missouri,Barry,0.760075,100,-0.047647,50,35881,26.1,16.2,...,0.540874,104,0.061673,80,-0.093177,24,0.030080,63,-0.046223,10
6,29011,Missouri,Barton,-0.052656,61,0.078060,64,12386,25.3,16.5,...,0.034089,68,-0.086693,32,0.095350,90,0.053472,66,0.015931,76
7,29013,Missouri,Bates,-0.432543,34,0.119426,72,16761,24.5,16.9,...,-0.224421,32,-0.078749,38,0.153247,105,0.055675,67,-0.010747,46
8,29015,Missouri,Benton,-0.170069,51,0.088157,65,18461,18.4,25.6,...,-0.320307,22,-0.122251,27,0.097121,92,0.134361,83,-0.021074,34
9,29017,Missouri,Bollinger,0.269072,81,0.370449,101,11841,23.3,15.3,...,0.231178,93,0.038083,70,0.143073,103,0.145212,85,0.044081,106


In [97]:
chr11['year'] = 2011

In [98]:
chr11.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Z-Score',
 'Health Outcomes_Rank',
 'Health Factors_Z-Score',
 'Health Factors_Rank',
 'Demographics_Population',
 'Demographics_< 18',
 'Demographics_65 and over',
 'Demographics_African American',
 'Demographics_American Indian/ Alaskan Native',
 'Demographics_Asian',
 'Demographics_Native Hawaiian/ Other Pacific Islander',
 'Demographics_Hispanic',
 'Demographics_Population.1',
 'Demographics_# not proficient in English',
 'Demographics_% not proficient in English',
 'Demographics_Female',
 'Demographics_Rural',
 '% diabetic_Diabetes',
 '% diabetic_95% CI - Low',
 '% diabetic_95% CI - High',
 'HIV rate_Population',
 'HIV rate_HIV rate',
 'Binge drinking_Sample Size',
 'Binge drinking_% Binge Drinking',
 'Binge drinking_95% CI - Low',
 'Binge drinking_95% CI - High',
 'Physical inactivity_Physical Inactivity',
 'Physical inactivity_95% CI - Low',
 'Physical inactivity_95% CI - High',
 'Mental health providers_MPH No.',
 'Mental health p

In [99]:
# Columns to be dropped
columns_to_drop = [
 'Health Outcomes_Z-Score',
 'Health Factors_Z-Score',
 'Demographics_African American',
 'Demographics_American Indian/ Alaskan Native',
 'Demographics_Asian',
 'Demographics_Native Hawaiian/ Other Pacific Islander',
 'Demographics_Hispanic',
 'Demographics_Population.1',
 'Demographics_# not proficient in English',
 'Demographics_% not proficient in English',
 'Demographics_Female',
 '% diabetic_95% CI - Low',
 '% diabetic_95% CI - High',
 'HIV rate_Population',
 'HIV rate_HIV rate',
 'Binge drinking_Sample Size',
 'Binge drinking_% Binge Drinking',
 'Binge drinking_95% CI - Low',
 'Binge drinking_95% CI - High',
 'Physical inactivity_95% CI - Low',
 'Physical inactivity_95% CI - High',
 'Mental health providers_MPH No.',
 'Mental health providers_Population',
 'Median household income_Household Income',
 'Median household income_95% CI - Low',
 'Median household income_95% CI - High',
 '% with high housing costs_# high housing costs',
 '% with high housing costs_Households',
 '% with high housing costs_% high housing costs',
 '% of children eligible for free lunch_% Free lunch',
 '% illiterate_Population',
 '% illiterate_Illiteracy',
 '% illiterate_95% CI - Low',
 '% illiterate_95% CI - High',
 'Liquor store density_Population',
 'Liquor store density_Liquor Stores',
 'Liquor store density_Liquor Store Rate',
 '% of labor force that drives alone to work_No. Drive Alone',
 '% of labor force that drives alone to work_Workers',
 '% of labor force that drives alone to work_% Drive Alone',
 'Premature death (Years of Potential Life Lost)_Unreliable',
 'Premature death (Years of Potential Life Lost)_Deaths',
 'Premature death (Years of Potential Life Lost)_YPLL Rate',
 'Premature death (Years of Potential Life Lost)_95% CI - Low',
 'Premature death (Years of Potential Life Lost)_95% CI - High',
 'Premature death (Years of Potential Life Lost)_Z-Score',
 'Poor or fair health_Sample Size',
 'Poor or fair health_95% CI - Low',
 'Poor or fair health_95% CI - High',
 'Poor or fair health_Z-Score',
 'Poor physical health days_Sample Size',
 'Poor physical health days_95% CI - Low',
 'Poor physical health days_95% CI - High',
 'Poor physical health days_Z-Score',
 'Poor mental health days_Sample Size',
 'Poor mental health days_95% CI - Low',
 'Poor mental health days_95% CI - High',
 'Poor mental health days_Z-Score',
 'Low birthweight_Unreliable',
 'Low birthweight_Sample Size',
 'Low birthweight_% LBW',
 'Low birthweight_95% CI - Low',
 'Low birthweight_95% CI - High',
 'Low birthweight_Z-Score',
 'Adult smoking_Sample Size',
 'Adult smoking_95% CI - Low',
 'Adult smoking_95% CI - High',
 'Adult smoking_Z-Score',
 'Adult obesity_95% CI - Low',
 'Adult obesity_95% CI - High',
 'Adult obesity_Z-Score',
 'Excessive drinking_Sample Size',
 'Excessive drinking_% Excessive Drinking',
 'Excessive drinking_95% CI - Low',
 'Excessive drinking_95% CI - High',
 'Excessive drinking_Z-Score',
 'Motor vehicle crash death rate_Unreliable',
 'Motor vehicle crash death rate_MV mortality rate',
 'Motor vehicle crash death rate_95% CI - Low',
 'Motor vehicle crash death rate_95% CI - High',
 'Motor vehicle crash death rate_Z-Score',
 'Sexually transmitted infections_Under review',
 'Sexually transmitted infections_Cases',
 'Sexually transmitted infections_Population',
 'Sexually transmitted infections_Rates per 100,000',
 'Sexually transmitted infections_Z-Score',
 'Teen birth rate_Teen Births ',
 'Teen birth rate_Teen Birth Rate',
 'Teen birth rate_95% CI - Low',
 'Teen birth rate_95% CI - High',
 'Teen birth rate_Z-Score',
 'Uninsured adults_95% CI - Low',
 'Uninsured adults_95% CI - High',
 'Uninsured adults_Z-Score',
 'Primary care physicians*_PCP No',
 'Primary care physicians*_PCP Rate',
 'Primary care physicians*_PCP Ratio',
 'Previous Primary care physician data--used to calculate rankings*_Unreliable',
 'Previous Primary care physician data--used to calculate rankings*_Population',
 'Previous Primary care physician data--used to calculate rankings*_PCP No',
 'Previous Primary care physician data--used to calculate rankings*_PCP Rate',
 'Previous Primary care physician data--used to calculate rankings*_PCP Ratio',
 'Previous Primary care physician data--used to calculate rankings*_Z-Score',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_No of Medicare enrollees',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_ACSC Rate',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_95% CI - Low',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_95% CI - High',
 'Preventable hospital stays (Ambulatory Care Sensitive Conditions)_Z-Score',
 'Diabetic screening_No of Diabetics',
 'Diabetic screening_% HbA1c',
 'Diabetic screening_95% CI - Low',
 'Diabetic screening_95% CI - High',
 'Diabetic screening_Z-Score',
 'Mammography screening_Mammography Rate',
 'Mammography screening_95% CI - Low',
 'Mammography screening_95% CI - High',
 'Mammography screening_Z-Score',
 'High school graduation_Under review',
 'High school graduation_AFGR',
 'High school graduation_Z-Score',
 'Some college (post-secondary education)_PSED Num',
 'Some college (post-secondary education)_Population',
 'Some college (post-secondary education)_PSED',
 'Some college (post-secondary education)_Z-Score',
 'Unemployment_95% CI - Low',
 'Unemployment_95% CI - High',
 'Unemployment_Z-Score',
 'Children in poverty_% Children in Poverty',
 'Children in poverty_95% CI - Low',
 'Children in poverty_95% CI - High',
 'Children in poverty_Z-Score',
 'Inadequate social support_Sample Size',
 'Inadequate social support_% No Social-Emotional Support',
 'Inadequate social support_95% CI - Low',
 'Inadequate social support_95% CI - High',
 'Inadequate social support_Z-Score',
 'Single-parent households_% Single-Parent Households',
 'Single-parent households_Z-Score',
 'Violent crime rate_Aggregate Population',
 'Violent crime rate_Violent Crimes',
 'Violent crime rate_Z-Score',
 'Homicide rate_Unreliable',
 'Homicide rate_Homicide Rate',
 'Homicide rate_95% CI - Low',
 'Homicide rate_95% CI - High',
 'Homicide rate_Z-Score',
 'Air pollution-particulate matter days_PM Days',
 'Air pollution-particulate matter days_Z-Score',
 'Air pollution-ozone days_Ozone Days',
 'Air pollution-ozone days_Z-Score',
 'Access to healthy foods_Unreliable',
 'Access to healthy foods_Zip Codes with Healthy Food',
 'Access to healthy foods_# Zip Codes',
 'Access to healthy foods_% Healthy Food',
 'Access to healthy foods_Z-Score',
 'Access to recreational facilities_Rec Facs',
 'Access to recreational facilities_Population',
 'Access to recreational facilities_Rec Fac Rate',
 'Access to recreational facilities_Z-Score',
 'Mortality_Z-Score',
 'Mortality_Rank',
 'Morbidity_Z-Score',
 'Morbidity_Rank',
 'Health Behaviors_Z-Score',
 'Clinical Care_Z-Score',
 'Social & Economic Factors_Z-Score',
 'Physical Environment_Z-Score']

In [100]:
# Dropping columns in-place
chr11.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr11)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County Health Outcomes_Rank Health Factors_Rank  \
0    29000  Missouri             NaN                  NaN                 NaN   
1    29001  Missouri           Adair                   60                  14   
2    29003  Missouri          Andrew                   19                  11   
3    29005  Missouri        Atchison                    5                  32   
4    29007  Missouri         Audrain                   45                  38   
5    29009  Missouri           Barry                  100                  50   
6    29011  Missouri          Barton                   61                  64   
7    29013  Missouri           Bates                   34                  72   
8    29015  Missouri          Benton                   51                  65   
9    29017  Missouri       Bollinger                   81                 101   
10   29019  Missouri           Boone                   10        

In [101]:
chr11.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Demographics_Population',
 'Demographics_< 18',
 'Demographics_65 and over',
 'Demographics_Rural',
 '% diabetic_Diabetes',
 'Physical inactivity_Physical Inactivity',
 'Mental health providers_MPH Rate',
 'Mental health providers_MPH Ratio',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Uninsured adults_% Uninsured',
 'Unemployment_% unemployed',
 'Violent crime rate_Violent Crime Rate',
 'Health Behaviors_Rank',
 'Clinical Care_Rank',
 'Social & Economic Factors_Rank',
 'Physical Environment_Rank',
 'year']

In [147]:
# Columns to be dropped
columns_to_drop = columns_to_drop = [
 'Demographics_Population',
 'Demographics_< 18',
 'Demographics_65 and over',
 'Demographics_Rural',
 '% diabetic_Diabetes',
 'Physical inactivity_Physical Inactivity',
 'Mental health providers_MPH Rate',
 'Mental health providers_MPH Ratio']

In [148]:
# Dropping columns in-place
chr11.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr11)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County Health Outcomes_Rank Health Factors_Rank  \
0    29000  Missouri             NaN                  NaN                 NaN   
1    29001  Missouri           Adair                   60                  14   
2    29003  Missouri          Andrew                   19                  11   
3    29005  Missouri        Atchison                    5                  32   
4    29007  Missouri         Audrain                   45                  38   
5    29009  Missouri           Barry                  100                  50   
6    29011  Missouri          Barton                   61                  64   
7    29013  Missouri           Bates                   34                  72   
8    29015  Missouri          Benton                   51                  65   
9    29017  Missouri       Bollinger                   81                 101   
10   29019  Missouri           Boone                   10        

In [186]:
chr11.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Uninsured adults_% Uninsured',
 'Unemployment_% Unemployed',
 'Violent Crime Rate',
 'Health Behaviors_Rank',
 'Clinical Care_Rank',
 'Social & Economic Factors_Rank',
 'Physical Environment_Rank',
 'year']

In [167]:
chr11.rename(columns={'Unemployment_% unemployed': 'Unemployment_% Unemployed'}, inplace=True)

In [185]:
chr11.rename(columns={'Violent crime rate_Violent Crime Rate': 'Violent Crime Rate'}, inplace=True)

In [102]:
data_10_1 = pd.read_excel('2010 County Health Ranking Missouri Data - v2.xls',sheet_name = 'Outcomes & Factors Rankings', header = [0,1])
data_10_3 = pd.read_excel('2010 County Health Ranking Missouri Data - v2.xls',sheet_name = 'Measure Data', header = [0,1])
data_10_4 = pd.read_excel('2010 County Health Ranking Missouri Data - v2.xls',sheet_name = 'Outcomes & Factors SubRankings', header = [0,1])

In [103]:
data_10_1 = add_index_prefix(data_10_1)
#data_10_2 = add_index_prefix(data_10_2)
data_10_3 = add_index_prefix(data_10_3)
data_10_4 = add_index_prefix(data_10_4)

display(data_10_1.columns, data_10_3.columns , data_10_4.columns)

Index(['FIPS', 'State', 'County', 'Health Outcomes_Z-Score',
       'Health Outcomes_Rank', 'Health Factors_Z-Score',
       'Health Factors_Rank'],
      dtype='object')

Index(['FIPS', 'State', 'County', 'Premature death_Deaths',
       'Premature death_Aggregate Population', 'Premature death_YPLL Rate',
       'Premature death_95% CI - Low', 'Premature death_95% CI - High',
       'Premature death_Z-Score', 'Poor or fair health_Sample Size',
       ...
       'Air pollution-ozone days_Ozone Days',
       'Air pollution-ozone days_Z-Score',
       'Access to healthy foods_Zip Codes with Healthy Food',
       'Access to healthy foods_# Zip Codes',
       'Access to healthy foods_% Healthy Food',
       'Access to healthy foods_Z-Score', 'Liquor store density_Population',
       'Liquor store density_Liquor Stores',
       'Liquor store density_Liquor Store Rate',
       'Liquor store density_Z-Score'],
      dtype='object', length=127)

Index(['FIPS', 'State', 'County', 'Mortality_Z-Score', 'Mortality_Rank',
       'Morbidity_Z-Score', 'Morbidity_Rank', 'Health Behaviors_Z-Score',
       'Health Behaviors_Rank', 'Clinical Care_Z-Score', 'Clinical Care_Rank',
       'Social & Economic Factors_Z-Score', 'Social & Economic Factors_Rank',
       'Physical Environment_Z-Score', 'Physical Environment_Rank'],
      dtype='object')

In [104]:
chr10 = pd.merge(data_10_1 , data_10_3 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr10 = pd.merge(chr10 , data_10_4 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
#chr10 = pd.merge(chr10 , data_10_4 , on = ['FIPS', 'State' , 'County'] , how = 'inner')
chr10

,FIPS,State,County,Health Outcomes_Z-Score,Health Outcomes_Rank,Health Factors_Z-Score,Health Factors_Rank,Premature death_Deaths,Premature death_Aggregate Population,Premature death_YPLL Rate,...,Morbidity_Z-Score,Morbidity_Rank,Health Behaviors_Z-Score,Health Behaviors_Rank,Clinical Care_Z-Score,Clinical Care_Rank,Social & Economic Factors_Z-Score,Social & Economic Factors_Rank,Physical Environment_Z-Score,Physical Environment_Rank
0,29000,Missouri,NaN,NaN,NaN,NaN,NaN,70181.0,16227245.0,8111.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29001,Missouri,Adair,-0.141168,54.0,-0.328472,19.0,261.0,69445.0,8430.0,...,0.015521,69.0,-0.247746,9.0,0.113686,100.0,-0.185171,23.0,-0.009241,56.0
2,29003,Missouri,Andrew,-0.678787,21.0,-0.533086,9.0,205.0,46380.0,8393.5,...,-0.512310,7.0,-0.086881,33.0,-0.050488,36.0,-0.374594,5.0,-0.021124,39.0
3,29005,Missouri,Atchison,-0.946562,11.0,-0.195290,33.0,57.0,16354.0,7259.2,...,-0.475924,8.0,-0.172469,16.0,0.091394,94.0,-0.083666,45.0,-0.030549,27.0
4,29007,Missouri,Audrain,-0.106226,57.0,-0.094959,46.0,322.0,70530.0,7933.2,...,0.183680,88.0,0.024332,68.0,-0.084706,26.0,-0.017385,60.0,-0.017201,43.0
5,29009,Missouri,Barry,0.840857,102.0,-0.109446,43.0,511.0,98860.0,9678.5,...,0.662764,107.0,0.036314,73.0,-0.113120,20.0,-0.017697,59.0,-0.014943,46.0
6,29011,Missouri,Barton,-0.042845,61.0,0.247913,93.0,161.0,35501.0,8490.9,...,0.097514,78.0,-0.111665,23.0,0.022305,63.0,0.331644,107.0,0.005630,69.0
7,29013,Missouri,Bates,-0.533539,26.0,0.347105,101.0,226.0,46538.0,7989.5,...,-0.258730,29.0,0.103229,90.0,0.245283,115.0,0.038151,74.0,-0.039559,19.0
8,29015,Missouri,Benton,-0.359803,36.0,0.127501,77.0,363.0,49318.0,8961.0,...,-0.345500,21.0,-0.100832,26.0,0.126259,105.0,0.165656,88.0,-0.063581,9.0
9,29017,Missouri,Bollinger,0.540006,96.0,0.206233,88.0,182.0,33946.0,10164.6,...,0.231566,93.0,0.085476,87.0,0.055832,78.0,0.029524,70.0,0.035400,96.0


In [105]:
chr10['year'] = 2010

In [106]:
chr10.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Z-Score',
 'Health Outcomes_Rank',
 'Health Factors_Z-Score',
 'Health Factors_Rank',
 'Premature death_Deaths',
 'Premature death_Aggregate Population',
 'Premature death_YPLL Rate',
 'Premature death_95% CI - Low',
 'Premature death_95% CI - High',
 'Premature death_Z-Score',
 'Poor or fair health_Sample Size',
 'Poor or fair health_% Fair/Poor',
 'Poor or fair health_95% CI - Low',
 'Poor or fair health_95% CI - High',
 'Poor or fair health_Z-Score',
 'Poor physical health days_Sample Size',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor physical health days_95% CI - Low',
 'Poor physical health days_95% CI - High',
 'Poor physical health days_Z-Score',
 'Poor mental health days_Sample Size',
 'Poor mental health days_Mentally Unhealthy Days',
 'Poor mental health days_95% CI - Low',
 'Poor mental health days_95% CI - High',
 'Poor mental health days_Z-Score',
 'Low birthweight_Sample Size',
 'Low birthweight_% LBW',
 '

In [107]:
# Columns to be dropped
columns_to_drop = [
 'Health Outcomes_Z-Score',
 'Health Factors_Z-Score',
 'Premature death_Deaths',
 'Premature death_Aggregate Population',
 'Premature death_YPLL Rate',
 'Premature death_95% CI - Low',
 'Premature death_95% CI - High',
 'Premature death_Z-Score',
 'Poor or fair health_Sample Size',
 'Poor or fair health_95% CI - Low',
 'Poor or fair health_95% CI - High',
 'Poor or fair health_Z-Score',
 'Poor physical health days_Sample Size',
 'Poor physical health days_95% CI - Low',
 'Poor physical health days_95% CI - High',
 'Poor physical health days_Z-Score',
 'Poor mental health days_Sample Size',
 'Poor mental health days_95% CI - Low',
 'Poor mental health days_95% CI - High',
 'Poor mental health days_Z-Score',
 'Low birthweight_Sample Size',
 'Low birthweight_% LBW',
 'Low birthweight_95% CI - Low',
 'Low birthweight_95% CI - High',
 'Low birthweight_Z-Score',
 'Adult smoking_Sample Size',
 'Adult smoking_95% CI - Low',
 'Adult smoking_95% CI - High',
 'Adult smoking_Z-Score',
 'Adult obesity_95% CI - Low',
 'Adult obesity_95% CI - High',
 'Adult obesity_Z-Score',
 'Binge drinking_Sample Size',
 'Binge drinking_% Binge Drinking',
 'Binge drinking_95% CI - Low',
 'Binge drinking_95% CI - High',
 'Binge drinking_Z-Score',
 'Motor vehicle crash death rate_MV mortality rate',
 'Motor vehicle crash death rate_95% CI - Low',
 'Motor vehicle crash death rate_95% CI - High',
 'Motor vehicle crash death rate_Z-Score',
 'Chlamydia rate_Cases',
 'Chlamydia rate_Population',
 'Chlamydia rate_Rates per 100,000',
 'Chlamydia rate_Z-Score',
 'Teen birth rate_Teen Births ',
 'Teen birth rate_Teen Birth Rate',
 'Teen birth rate_95% CI - Low',
 'Teen birth rate_95% CI - High',
 'Teen birth rate_Z-Score',
 'Uninsured adults_95% CI - Low',
 'Uninsured adults_95% CI - High',
 'Uninsured adults_Z-Score',
 'Primary care provider rate*_PCP No',
 'Primary care provider rate*_PCP Rate',
 'Previous Primary care provider rate data--used to calculate rankings*_Population',
 'Previous Primary care provider rate data--used to calculate rankings*_PCP No',
 'Previous Primary care provider rate data--used to calculate rankings*_PCP Rate',
 'Previous Primary care provider rate data--used to calculate rankings*_Z-Score',
 'Preventable hospital stays_No of Medicare enrollees',
 'Preventable hospital stays_ACSC Rate',
 'Preventable hospital stays_95% CI - Low',
 'Preventable hospital stays_95% CI - High',
 'Preventable hospital stays_Z-Score',
 'Diabetic screening_No of Diabetics',
 'Diabetic screening_% HbA1c',
 'Diabetic screening_95% CI - Low',
 'Diabetic screening_95% CI - High',
 'Diabetic screening_Z-Score',
 'Hospice use_% Hospice Use',
 'Hospice use_95% CI - Low',
 'Hospice use_95% CI - High',
 'Hospice use_Z-Score',
 'High school graduation_Enrollment',
 'High school graduation_Diplomas',
 'High school graduation_AFGR',
 'High school graduation_Z-Score',
 'College degrees_% College',
 'College degrees_95% CI - Low',
 'College degrees_95% CI - High',
 'College degrees_Z-Score',
 'Unemployment_95% CI - Low',
 'Unemployment_95% CI - High',
 'Unemployment_Z-Score',
 'Children in poverty_% Children in Poverty',
 'Children in poverty_95% CI - Low',
 'Children in poverty_95% CI - High',
 'Children in poverty_Z-Score',
 'Income inequality_GINI',
 'Income inequality_Z-Score',
 'Inadequate social support_Sample Size',
 'Inadequate social support_% No Social-Emotional Support',
 'Inadequate social support_95% CI - Low',
 'Inadequate social support_95% CI - High',
 'Inadequate social support_Z-Score',
 'Single-parent households_% Single-Parent Households',
 'Single-parent households_95% CI - Low',
 'Single-parent households_95% CI - High',
 'Single-parent households_Z-Score',
 'Violent crime rate_Aggregate Population',
 'Violent crime rate_Violent Crimes',
 'Violent crime rate_Z-Score',
 'Homicide rate_Homicide Rate',
 'Homicide rate_95% CI - Low',
 'Homicide rate_95% CI - High',
 'Homicide rate_Z-Score',
 'Air pollution-particulate matter days_PM Days',
 'Air pollution-particulate matter days_Z-Score',
 'Air pollution-ozone days_Ozone Days',
 'Air pollution-ozone days_Z-Score',
 'Access to healthy foods_Zip Codes with Healthy Food',
 'Access to healthy foods_# Zip Codes',
 'Access to healthy foods_% Healthy Food',
 'Access to healthy foods_Z-Score',
 'Liquor store density_Population',
 'Liquor store density_Liquor Stores',
 'Liquor store density_Liquor Store Rate',
 'Liquor store density_Z-Score',
 'Mortality_Z-Score',
 'Mortality_Rank',
 'Morbidity_Z-Score',
 'Morbidity_Rank',
 'Health Behaviors_Z-Score',
 'Clinical Care_Z-Score',
 'Social & Economic Factors_Z-Score',
 'Physical Environment_Z-Score']

In [108]:
# Dropping columns in-place
chr10.drop(columns=columns_to_drop, inplace=True)

print("\nDataFrame After Dropping Columns In-Place:")
print(chr10)


DataFrame After Dropping Columns In-Place:
      FIPS     State          County  Health Outcomes_Rank  \
0    29000  Missouri             NaN                   NaN   
1    29001  Missouri           Adair                  54.0   
2    29003  Missouri          Andrew                  21.0   
3    29005  Missouri        Atchison                  11.0   
4    29007  Missouri         Audrain                  57.0   
5    29009  Missouri           Barry                 102.0   
6    29011  Missouri          Barton                  61.0   
7    29013  Missouri           Bates                  26.0   
8    29015  Missouri          Benton                  36.0   
9    29017  Missouri       Bollinger                  96.0   
10   29019  Missouri           Boone                  13.0   
11   29021  Missouri        Buchanan                  59.0   
12   29023  Missouri          Butler                 109.0   
13   29025  Missouri        Caldwell                  83.0   
14   29027  Missouri      

In [150]:
chr10.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Uninsured adults_% Uninsured',
 'Unemployment_% Unemployed',
 'Violent crime rate_Violent Crime Rate',
 'Health Behaviors_Rank',
 'Clinical Care_Rank',
 'Social & Economic Factors_Rank',
 'Physical Environment_Rank',
 'year']

In [112]:
chr10.rename(columns={'Unemployment_% unemployed': 'Unemployment_% Unemployed'}, inplace=True)

In [183]:
chr10.rename(columns={'Violent crime rate_Violent Crime Rate': 'Violent Crime Rate'}, inplace=True)

In [184]:
chr10.columns.to_list()

['FIPS',
 'State',
 'County',
 'Health Outcomes_Rank',
 'Health Factors_Rank',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Poor mental health days_Mentally Unhealthy Days',
 'Adult smoking_% Smokers',
 'Adult obesity_% Obese',
 'Uninsured adults_% Uninsured',
 'Unemployment_% Unemployed',
 'Violent Crime Rate',
 'Health Behaviors_Rank',
 'Clinical Care_Rank',
 'Social & Economic Factors_Rank',
 'Physical Environment_Rank',
 'year']

In [ ]:
#SHAPES

In [204]:
display(chr19.shape,chr18.shape,chr17.shape,chr16.shape,chr15.shape,chr14.shape,chr13.shape,chr12.shape , chr11.shape,chr10.shape)

(116, 18)

(116, 18)

(116, 18)

(116, 18)

(116, 18)

(116, 18)

(116, 18)

(116, 18)

(116, 18)

(116, 18)

In [ ]:
#SAVE

In [205]:
chr19.to_csv('chr19.csv')
chr18.to_csv('chr18.csv')
chr17.to_csv('chr17.csv')
chr16.to_csv('chr16.csv')
chr15.to_csv('chr15.csv')
chr14.to_csv('chr14.csv')
chr13.to_csv('chr13.csv')
chr12.to_csv('chr12.csv')
chr11.to_csv('chr11.csv')
chr10.to_csv('chr10.csv')


In [206]:
df = pd.concat([chr19,chr18,chr17,chr16,chr15,chr14,chr13,chr12 , chr11,chr10] )

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [207]:
import gc
gc.collect()

20

In [208]:
df.columns.to_list()

['Adult obesity_% Obese',
 'Adult smoking_% Smokers',
 'Clinical Care_Rank',
 'County',
 'FIPS',
 'Health Behaviors_Rank',
 'Health Factors_Rank',
 'Health Outcomes_Rank',
 'Physical Environment_Rank',
 'Poor mental health days_Mentally Unhealthy Days',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Social & Economic Factors_Rank',
 'State',
 'Unemployment_% Unemployed',
 'Uninsured adults_% Uninsured',
 'Violent Crime Rate',
 'year']

In [209]:
df.dtypes

Adult obesity_% Obese                                  float64
Adult smoking_% Smokers                                float64
Clinical Care_Rank                                      object
County                                                  object
FIPS                                                     int64
Health Behaviors_Rank                                   object
Health Factors_Rank                                     object
Health Outcomes_Rank                                    object
Physical Environment_Rank                               object
Poor mental health days_Mentally Unhealthy Days        float64
Poor or fair health_% Fair/Poor                        float64
Poor physical health days_Physically Unhealthy Days    float64
Social & Economic Factors_Rank                          object
State                                                   object
Unemployment_% Unemployed                              float64
Uninsured adults_% Uninsured                           

In [210]:
df.shape

(1160, 18)

In [211]:
df.to_csv('chr_csv.csv')

In [212]:
df1 = df[df['FIPS'].astype(str) != '29000']

In [213]:
df1.isna().sum()

Adult obesity_% Obese                                    0
Adult smoking_% Smokers                                184
Clinical Care_Rank                                       0
County                                                   0
FIPS                                                     0
Health Behaviors_Rank                                    0
Health Factors_Rank                                      0
Health Outcomes_Rank                                     0
Physical Environment_Rank                                0
Poor mental health days_Mentally Unhealthy Days         66
Poor or fair health_% Fair/Poor                         73
Poor physical health days_Physically Unhealthy Days     23
Social & Economic Factors_Rank                           0
State                                                    0
Unemployment_% Unemployed                                0
Uninsured adults_% Uninsured                             0
Violent Crime Rate                                      

In [214]:
df1.to_csv('chr_csv1.csv')

In [215]:
df1.shape

(1150, 18)

In [216]:
df1.columns.to_list()

['Adult obesity_% Obese',
 'Adult smoking_% Smokers',
 'Clinical Care_Rank',
 'County',
 'FIPS',
 'Health Behaviors_Rank',
 'Health Factors_Rank',
 'Health Outcomes_Rank',
 'Physical Environment_Rank',
 'Poor mental health days_Mentally Unhealthy Days',
 'Poor or fair health_% Fair/Poor',
 'Poor physical health days_Physically Unhealthy Days',
 'Social & Economic Factors_Rank',
 'State',
 'Unemployment_% Unemployed',
 'Uninsured adults_% Uninsured',
 'Violent Crime Rate',
 'year']

In [217]:
df1.head(13)

,Adult obesity_% Obese,Adult smoking_% Smokers,Clinical Care_Rank,County,FIPS,Health Behaviors_Rank,Health Factors_Rank,Health Outcomes_Rank,Physical Environment_Rank,Poor mental health days_Mentally Unhealthy Days,Poor or fair health_% Fair/Poor,Poor physical health days_Physically Unhealthy Days,Social & Economic Factors_Rank,State,Unemployment_% Unemployed,Uninsured adults_% Uninsured,Violent Crime Rate,year
1,30.7,22.311543,5,Adair,29001,36,36,71,83,4.740753,20.080050,4.946602,80,Missouri,4.361035,14.361997,247.607444,2019
2,32.9,18.135443,19,Andrew,29003,4,6,8,44,4.033326,14.848926,4.049459,6,Missouri,3.135397,12.512124,184.177962,2019
3,33.3,18.418132,75,Atchison,29005,6,9,24,11,4.194421,16.246522,4.189911,7,Missouri,3.514377,12.555900,103.412616,2019
4,35.0,21.175243,27,Audrain,29007,79,58,50,20,4.509256,18.028495,4.596011,66,Missouri,3.538447,14.368068,182.945617,2019
5,32.5,23.477209,60,Barry,29009,84,65,86,37,4.715085,22.362148,4.955659,54,Missouri,3.435039,19.712709,242.193136,2019
6,33.6,22.340861,92,Barton,29011,67,83,65,63,4.742786,21.323682,4.864516,77,Missouri,4.202502,18.093820,282.979609,2019
7,32.2,21.387283,94,Bates,29013,34,60,48,15,4.448123,18.740584,4.652697,67,Missouri,4.361755,15.426908,473.249132,2019
8,33.9,21.614194,67,Benton,29015,41,86,90,67,4.663769,20.760119,4.836018,97,Missouri,5.182584,16.108260,435.084348,2019
9,32.0,22.953879,79,Bollinger,29017,89,81,76,49,4.540038,19.551573,4.736196,70,Missouri,4.422148,16.706612,437.725449,2019
10,26.5,18.221026,1,Boone,29019,2,4,9,95,4.198763,16.849735,4.062777,8,Missouri,2.608312,10.868166,361.894534,2019


In [1]:
import pandas as pd

# Load your data
data_path = 'chr_csv1.csv'  # Update the path to where your data is stored
data = pd.read_csv(data_path)

# Display initial missing values
print("Initial missing values:\n", data.isnull().sum())

# Impute numerical columns with the median
numerical_cols = ['Adult smoking_% Smokers', 'Poor mental health days_Mentally Unhealthy Days', 
                  'Poor or fair health_% Fair/Poor', 'Poor physical health days_Physically Unhealthy Days',
                  'Unemployment_% Unemployed', 'Uninsured adults_% Uninsured', 'Violent Crime Rate']
for col in numerical_cols:
    median = data[col].median()
    data[col].fillna(median, inplace=True)


# Check missing values after imputation
print("\nMissing values after imputation:\n", data.isnull().sum())

# Save the modified data if needed
output_path = 'new_chr_csv1.csv'
data.to_csv(output_path, index=False)


Initial missing values:
 Unnamed: 0                                               0
Adult obesity_% Obese                                    0
Adult smoking_% Smokers                                184
Clinical Care_Rank                                       0
County                                                   0
FIPS                                                     0
Health Behaviors_Rank                                    0
Health Factors_Rank                                      0
Health Outcomes_Rank                                     0
Physical Environment_Rank                                0
Poor mental health days_Mentally Unhealthy Days         66
Poor or fair health_% Fair/Poor                         73
Poor physical health days_Physically Unhealthy Days     23
Social & Economic Factors_Rank                           0
State                                                    0
Unemployment_% Unemployed                                0
Uninsured adults_% Uninsured   